# Modular Financial RAG System

NEW USER (no database):
Section 1 (Setup) → Section 3-4 (Process docs) → Section 5+ (RAG)

RETURNING USER (existing database):
Section 1 (Setup) → Section 5+ (RAG)

## 1. Core Setup

#### 1.1. Install Required Libraries

In [2]:
# Install neccessary libraries
!pip install --quiet sentence-transformers transformers chromadb pymupdf rank-bm25 python-docx ipywidgets spacy

#### 1.2 Enable jupyter extensions

In [71]:
# Enable widgets
!jupyter nbextension enable --py widgetsnbextension

python(55633) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


usage: jupyter [-h] [--version] [--config-dir] [--data-dir] [--runtime-dir]
               [--paths] [--json] [--debug]
               [subcommand]

Jupyter: Interactive Computing

positional arguments:
  subcommand     the subcommand to launch

options:
  -h, --help     show this help message and exit
  --version      show the versions of core jupyter packages and exit
  --config-dir   show Jupyter config dir
  --data-dir     show Jupyter data dir
  --runtime-dir  show Jupyter runtime dir
  --paths        show all Jupyter paths. Add --json for machine-readable
                 format.
  --json         output paths as machine-readable json
  --debug        output debug information about paths

Available subcommands: kernel kernelspec migrate run troubleshoot trust

Jupyter command `jupyter-nbextension` not found.


#### 1.3 Import Libraries

In [15]:

# Essential imports
import warnings
warnings.filterwarnings("ignore", category=UserWarning)

from pathlib import Path
import os, json, re, threading
from typing import Dict, List, Optional, Any, Tuple
import numpy as np
import pandas as pd
from tqdm.auto import tqdm
import requests
import docx

from sentence_transformers import SentenceTransformer
from sentence_transformers import CrossEncoder
from transformers import pipeline
import chromadb
from chromadb.config import Settings
import fitz  # PyMuPDF

from rank_bm25 import BM25Okapi
import pickle
import re
from IPython.display import display, clear_output
import ipywidgets as widgets
import datetime
import spacy
from collections import defaultdict
import shutil
import time
from concurrent.futures import ThreadPoolExecutor
import subprocess
from dotenv import load_dotenv
from huggingface_hub import snapshot_download
from mlx_lm import load, generate
import traceback
import threading

#### Section 1.4. Local Config

This section establishes the core directory structure and configuration settings for the RAG system. It defines the foundational paths and validates the environment setup.

Run this cell once. Make sure the path to the desired data directory is correct and uncommented. This will be the dataset processed in the following sections. If another dataset have already been processed and stored, then comment out the path to those files. This allows to create different collections in the vector database.

**Directory Configuration:**
- Sets `ROOT` as the current working directory using `Path.cwd()`
- Defines three critical directories: `DATA_DIR` (source documents), `STORE_DIR` (ChromaDB storage), and `MODELS_DIR` (model cache)
- Creates `file_directories` list specifying which subdirectories to scan for documents

**Environment Validation:**
- Performs existence checks on all defined paths
- Scans and reports ChromaDB files if the store directory exists
- Provides diagnostic output showing directory status and file counts

**System Configuration:**
- Establishes `CONFIG` dictionary with key parameters:
  - `embedding_model`: BGE large model for semantic search
  - `finbert_model`: Financial sentiment analysis model
  - `llm_model`: Ollama model for response generation
  - `chunk_size` and `overlap`: Text processing parameters

In [16]:
# Core configuration to work from the current directory
ROOT = Path.cwd() 
DATA_DIR = ROOT / 'data'
STORE_DIR = ROOT / 'chroma_store'
MODELS_DIR = ROOT / 'models'

# Verify paths exist
print("Path verification:")
print(f"   ROOT: {ROOT}")
print(f"   DATA_DIR exists: {DATA_DIR.exists()}")
print(f"   STORE_DIR exists: {STORE_DIR.exists()}")
print(f"   MODELS_DIR exists: {MODELS_DIR.exists()}")

if STORE_DIR.exists():
    chroma_files = list(STORE_DIR.glob('*'))
    print(f"   ChromaDB files found: {len(chroma_files)}")
    for file in chroma_files[:5]:  # Show first 5 files
        print(f"      {file.name}")

file_directories = [
    # DATA_DIR / 'pra_rulebook',
    # Add back other directories when needed:
    DATA_DIR / 'earnings_transcripts',
    DATA_DIR / 'transcripts_analyst_meeting',
]

CONFIG = {
    'embedding_model': 'BAAI/bge-large-en-v1.5',
    'finbert_model': 'yiyanghkust/finbert-tone',
    'mistral_7b_4bit_mlx': "mlx-community/Mistral-7B-Instruct-v0.3-4bit",
    'llama_8b_4bit_mlx': "mlx-community/Llama-3.1-8B-Instruct-4bit",
    'llama_3_2_3b_4bit_mlx': "mlx-community/Llama-3.2-3B-Instruct-4bit",
    'chunk_size': 600,
    'overlap': 100
}

print(f"Config: {len(CONFIG)} settings loaded")

Path verification:
   ROOT: /Users/rodrigograndy/Desktop/coding_projects/posts_website_drafts/rag_boe
   DATA_DIR exists: True
   STORE_DIR exists: True
   MODELS_DIR exists: True
   ChromaDB files found: 4
      3f516e47-f1a9-48bb-adb7-79543f55aa64
      bm25_indices
      chroma.sqlite3
      c82d869e-4e75-4e96-af7b-b45927f70131
Config: 7 settings loaded


#### 1.5.1 Function to Download Models if Required

This section checks for the presence of required models in the MODELS_DIR and downloads them if they are not already available. It ensures that all necessary models are locally cached for offline use and faster loading times.

In [3]:
# Function to download model from HuggingFace Hub
def download_hf_model(repo_id, local_dir="models"):
    """
    Download a model from HuggingFace Hub.
    
    Args:
        repo_id (str): HuggingFace model repository ID (e.g., 'mlx-community/Mixtral-8x7B-Instruct-v0.1-4bit')
        local_dir (str): Local directory to save the model (default: 'models')
    
    Returns:
        str: Path to the downloaded model
    """
    # Load environment variables from .env file
    load_dotenv()
    
    # Get HuggingFace token from environment
    hf_token = os.getenv('HF_TOKEN')
    
    if not hf_token:
        print("Warning: HF_TOKEN not found in .env file. Attempting download without authentication.")
        print("Some models may require authentication to download.")
    
    # Create models directory if it doesn't exist
    models_path = Path(local_dir)
    models_path.mkdir(parents=True, exist_ok=True)
    
    # Extract model name from repo_id for the subdirectory
    model_name = repo_id.split('/')[-1]
    model_path = models_path / model_name
    
    try:
        print(f"Downloading model: {repo_id}")
        print(f"Destination: {model_path}")
        
        # Download the model
        downloaded_path = snapshot_download(
            repo_id=repo_id,
            local_dir=str(model_path),
            token=hf_token,
            resume_download=True,  # Resume if interrupted
            local_dir_use_symlinks=False  # Avoid symlinks for compatibility
        )
        
        print(f"✓ Model downloaded successfully to: {downloaded_path}")
        return downloaded_path
        
    except Exception as e:
        print(f"✗ Error downloading model: {str(e)}")
        print(f"\nTroubleshooting tips:")
        print(f"1. Verify the repo_id is correct: {repo_id}")
        print(f"2. Check if the model exists at: https://huggingface.co/{repo_id}")
        print(f"3. Ensure your HF_TOKEN has the necessary permissions")
        print(f"4. Some models require accepting a license agreement on HuggingFace first")
        raise


#### 1.5.2 Download Models

In [4]:
# Uncomment to download desired model.
# Models repo IDs:
# mistral_7b_4bit_mlx = "mlx-community/Mistral-7B-Instruct-v0.3-4bit"
# llama_8b_4bit_mlx = "mlx-community/Llama-3.1-8B-Instruct-4bit"
# llama_3_2_3b_4bit_mlx = "mlx-community/Llama-3.2-3B-Instruct-4bit"

# # Usage
# model_repo = llama_3_2_3b_4bit_mlx
# try:
#     model_path = download_hf_model(model_repo, local_dir='models')
#     print(f"\nModel ready at: {model_path}")
# except Exception as e:
#     print(f"\nFailed to download model: {e}")

#### 1.5.3 Model Loading



In [17]:
# Load embedding models (prefer local models/ directory)
from typing import Optional
def resolve_local_model_path(model_identifier: str, models_dir: Path = MODELS_DIR) -> Optional[str]:
    if not model_identifier:
        return None
    # last segment of HF id or simple folder name
    name = model_identifier.split('/')[-1]
    try:
        # find first directory under MODELS_DIR with matching folder name
        for candidate in models_dir.rglob(name):
            if candidate.is_dir():
                return str(candidate)
    except Exception:
        pass
    return None

def load_embedding_model(model_name: str) -> SentenceTransformer:
    local = resolve_local_model_path(model_name)
    if local:
        print(f"Loading local embedding model from: {local}")
        return SentenceTransformer(local)
    print(f"Loading embedding model from HF Hub identifier: {model_name}")
    return SentenceTransformer(model_name)

def load_finbert_model(model_name: str):
    local = resolve_local_model_path(model_name)
    if local:
        print(f"Loading local FinBERT model from: {local}")
        return pipeline('sentiment-analysis', model=local, return_all_scores=True)
    print(f"Loading FinBERT model from HF Hub identifier: {model_name}")
    return pipeline('sentiment-analysis', model=model_name, return_all_scores=True)

# Instantiate (will prefer local cached models under `models/` if present)
embedding_model = load_embedding_model(CONFIG['embedding_model'])
finbert = load_finbert_model(CONFIG['finbert_model'])

print("Models loaded: embedding + FinBERT")

Loading local embedding model from: /Users/rodrigograndy/Desktop/coding_projects/posts_website_drafts/rag_boe/models/embeddings/BAAI/bge-large-en-v1.5


Device set to use mps:0


Loading local FinBERT model from: /Users/rodrigograndy/Desktop/coding_projects/posts_website_drafts/rag_boe/models/financial/finbert-tone
Models loaded: embedding + FinBERT


#### 1.5.4 Local LLM Engine 


In [18]:
# Local LLM Engine - MLX Optimized for Apple Silicon
class LocalLLMEngine:
    """Local LLM engine using MLX for Apple Silicon optimization"""

    RECOMMENDED_MODELS = {
        # Keys align with CONFIG keys in section 1.4
        'llama_8b_4bit_mlx': {
            'name': 'Meta-Llama-3.1-8B-Instruct-4bit',
            'params': '8B',
            'quant': '4bit mlx',
            'size_gb': 4.3,
            'mlx_repo': 'mlx-community/Llama-3.1-8B-Instruct-4bit'
        },
        'mistral_7b_4bit_mlx': {
            'name': 'Mistral-7B-Instruct-v0.3-4bit',
            'params': '7B',
            'quant': '4bit mlx',
            'size_gb': 3.9,
            'mlx_repo': 'mlx-community/Mistral-7B-Instruct-v0.3-4bit'
        },
        'llama_3_2_3b_4bit_mlx': {
            'name': 'Llama-3.2-3B-Instruct-4bit',
            'params': '3B',
            'quant': '4bit mlx',
            'size_gb': 1.6,
            'mlx_repo': 'mlx-community/Llama-3.2-3B-Instruct-4bit'
        },
    }
    MODEL_FILE_EXTS = ("*.safetensors", "*.pt", "*.bin", "model.safetensors")

    def __init__(self, models_dir: Path, model_key: str, auto_warm: bool = True):
        """Initialize and load the model. Set auto_warm=False to skip the background warmup.
        """
        self.models_dir = Path(models_dir)
        if not self.models_dir.exists():
            raise FileNotFoundError(f"Models directory not found: {self.models_dir}")

        self.model_key = model_key
        # Accept either a recommendation key (like 'llama_3_2_3b_4bit_mlx')
        # or a repo_id / folder name (like 'mlx-community/Llama-3.2-3B- Instruct-4bit' or 'Llama-3.2-3B-Instruct-4bit')
        self.model_info = self.RECOMMENDED_MODELS.get(model_key)
        if not self.model_info:
            # Allow passing a direct directory name or repo id as model_key
            self.model_info = {
                'name': model_key,
                'params': 'unknown',
                'quant': 'unknown',
                'size_gb': 0.0
            }

        # Try to resolve the actual model path (may be nested)
        self.model_path = self._resolve_model_path()

        if not self.model_path or not self.model_path.exists():
            # Provide helpful diagnostic output
            candidates = self._list_candidate_model_dirs()
            msg = (
                f"Model not found: {self.models_dir / (self.model_info.get('name') or self.model_key)}\n"
                "Expected MLX model directory with safetensors files.\n\n"
                f"Attempted to locate model for key '{self.model_key}' (name='{self.model_info.get('name')}').\n"
                f"Detected candidate model directories under {self.models_dir}:\n"
            )
            for c in candidates:
                msg += f"  - {c}\n"
            msg += ("\nOptions:\n - Ensure model files (.safetensors/.pt/.bin) are present under the named directory,\n"
                    " - Or create a symlink matching the expected name, e.g.:\n"
                    f"     ln -s \"ActualModelFolder\" \"{self.models_dir / (self.model_info.get('name') or self.model_key)}\"\n")
            raise FileNotFoundError(msg)

        # Initialize MLX
        try:
            import mlx.core as mx  # noqa: F401
            from mlx_lm import load, generate
            print(f"Loading MLX model from: {self.model_path} ...")
            self.model, self.tokenizer = load(str(self.model_path))
            self.generate = generate
            self.available = True
            self.model_info = {**self.model_info, 'resolved_path': str(self.model_path)}
            print(f"✅ MLX Model loaded: {self.model_info.get('name')} (resolved: {self.model_path})")
            # Background warmup to reduce first-call latency (optional)
            if auto_warm:
                threading.Thread(target=lambda: self._safe_warmup(), daemon=True).start()
            
        except ImportError as e:
            self.available = False
            raise ImportError(
                "MLX not installed.\nInstall with: pip install mlx mlx-lm\n"
                f"Import error: {e}"
            )
        except Exception as e:
            self.available = False
            raise RuntimeError(f"Failed to load MLX model from {self.model_path}: {e}\n{traceback.format_exc()}")

    @classmethod
    def from_config(cls, models_dir: Path, config: dict, key: str):
        """
        Factory: create a LocalLLMEngine from CONFIG mapping.
        Accepts either a recommended key (like 'llama_3_2_3b_4bit_mlx') or a repo-id string.
        """
        model_key_or_repo = config.get(key) or key
        return cls(models_dir, model_key_or_repo)

    def _safe_warmup(self):
        """Run a single short generate to prime caches, ignoring errors."""
        try:
            # Use a very small call to warm model/tokenizer
            self.generate_response("Hi", max_tokens=1)
        except Exception:
            pass
    
    def _resolve_model_path(self) -> Path:
        """
        Resolve the best model directory path. Strategy:
         1. Use the last segment of model_info['name'] (handles 'mlx-community/ModelName').
         2. models_dir / that name (direct)
         3. Search inside for nested dirs with model files
         4. Otherwise scan models_dir for any directory that contains model files and pick best candidate
        """
        raw_name = str(self.model_info.get('name') or self.model_key)
        base_name = raw_name.split('/')[-1]  # handle 'org/ModelName' strings

        candidate_root = self.models_dir / base_name

        # 1) Direct path exists and has model files
        if candidate_root.exists() and candidate_root.is_dir():
            if self._dir_contains_model_files(candidate_root):
                return candidate_root
            # maybe model files are nested one-level down or deeper -> search inside
            nested = self._find_nested_model_dir(candidate_root)
            if nested:
                return nested

        # 2) Try variants and direct model_key folder
        variants = [
            candidate_root,
            self.models_dir / self.model_key,
            self.models_dir / raw_name,
            self.models_dir / base_name.lower(),
            self.models_dir / base_name.replace('-', '_')
        ]
        for v in variants:
            if v.exists() and v.is_dir() and self._dir_contains_model_files(v):
                return v
            if v.exists() and v.is_dir():
                nested = self._find_nested_model_dir(v)
                if nested:
                    return nested

        # 3) Global scan for any folder containing model files (prioritize by folder name similarity)
        candidates = self._list_candidate_model_dirs()
        if not candidates:
            return None

        wanted_tokens = set(base_name.lower().replace('-', ' ').split())
        best = None
        best_score = -1
        for c in candidates:
            name = c.name.lower()
            score = sum(1 for t in wanted_tokens if t and t in name)
            if score > best_score:
                best_score = score
                best = c

        return best or candidates[0]

    def _dir_contains_model_files(self, p: Path) -> bool:
        # Look for typical model files directly in directory
        for globpat in ("*.safetensors", "*.pt", "*.bin", "model.safetensors"):
            if any(p.glob(globpat)):
                return True
        return False

    def _find_nested_model_dir(self, top: Path) -> Path:
        # Depth-first search for a directory that contains model files
        for fpat in ("**/*.safetensors", "**/*.pt", "**/*.bin"):
            for m in top.glob(fpat):
                return m.parent
        return None

    def _list_candidate_model_dirs(self):
        # Return a list of paths under models_dir that look like model dirs (contain model files)
        candidates = []
        for sub in sorted(self.models_dir.iterdir()):
            if not sub.is_dir():
                continue
            if self._dir_contains_model_files(sub):
                candidates.append(sub)
            else:
                nested = self._find_nested_model_dir(sub)
                if nested:
                    candidates.append(nested)
        return candidates

    def generate_response(self, prompt: str, temperature: float = 0.0,
                         max_tokens: int = 4000) -> str:
        """Generate response using MLX"""
        if not self.available:
            return "LLM not available. Model not loaded."

        try:
            # Format prompt for instruction-tuned models
            formatted_prompt = f"<|begin_of_text|><|start_header_id|>user<|end_header_id|>\n\n{prompt}<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\n"

            # The installed version of mlx_lm.generate does not accept 'temp' or 'temperature'.
            # For temperature=0.0 (greedy decoding) we can omit sampler.
            response = self.generate(
                self.model,
                self.tokenizer,
                prompt=formatted_prompt,
                max_tokens=max_tokens,
                verbose=False
            )

            return response.strip()

        except TypeError as e:
            return f"MLX Argument Error: {str(e)}. Try updating mlx-lm or checking generate() arguments."
        except Exception as e:
            return f"Error generating response: {str(e)[:200]}"

print("✅ LocalLLMEngine with MLX ready (Apple Silicon optimized)")
print("\nSupported MLX models:")
for key, info in LocalLLMEngine.RECOMMENDED_MODELS.items():
    print(f"  • {key}: {info['params']} ({info['quant']}, ~{info['size_gb']}GB)")
print("\n💡 Install MLX: pip install mlx mlx-lm")

✅ LocalLLMEngine with MLX ready (Apple Silicon optimized)

Supported MLX models:
  • llama_8b_4bit_mlx: 8B (4bit mlx, ~4.3GB)
  • mistral_7b_4bit_mlx: 7B (4bit mlx, ~3.9GB)
  • llama_3_2_3b_4bit_mlx: 3B (4bit mlx, ~1.6GB)

💡 Install MLX: pip install mlx mlx-lm


#### 1.5.5 Instantiate Models

In [7]:
def create_local_llm_from_config(config_key: str, load_llm: bool = False):
    """
    If load_llm=True, create and return a LocalLLMEngine instance resolved from CONFIG.
    Otherwise returns None.
    """
    if not load_llm:
        print("LLM instantiation skipped (pass load_llm=True to create local LLM).")
        return None
    try:
        # Use from_config if available, else fall back to direct constructor
        if hasattr(LocalLLMEngine, "from_config"):
            engine = LocalLLMEngine.from_config(MODELS_DIR, CONFIG, config_key)
        else:
            engine = LocalLLMEngine(MODELS_DIR, config_key)
        print("Local LLM instantiated:", engine.model_info.get('resolved_path'))
        return engine
    except Exception as e:
        print("Failed to instantiate local LLM:", e)
        raise

# Usage:
llm_small = create_local_llm_from_config('llama_3_2_3b_4bit_mlx', load_llm=True)

Loading MLX model from: /Users/rodrigograndy/Desktop/coding_projects/posts_website_drafts/rag_boe/models/Llama-3.2-3B-Instruct-4bit ...
✅ MLX Model loaded: mlx-community/Llama-3.2-3B-Instruct-4bit (resolved: /Users/rodrigograndy/Desktop/coding_projects/posts_website_drafts/rag_boe/models/Llama-3.2-3B-Instruct-4bit)
Local LLM instantiated: /Users/rodrigograndy/Desktop/coding_projects/posts_website_drafts/rag_boe/models/Llama-3.2-3B-Instruct-4bit


#### 1.5.6 Test Small Model Loading

In [ ]:
# Test Local LLM Engine
# test_prompt = "Are you ready to work?"
# response = llm_small.generate_response(test_prompt, temperature=0.0, max_tokens=50)
# print("Test Prompt:")
# print(test_prompt)
# print("\nGenerated Response:")
# print(response)

#### 1.6.1 Metadata Extraction using LLM

In [ ]:
# Metadata extractor - optimized for speed (first 3 pages only)
def _parse_json_from_llm(text: str) -> Any:
    m = re.search(r"(\{.*\}|\[.*\])", text, re.S)
    if m:
        try:
            return json.loads(m.group(1))
        except Exception:
            pass
    try:
        return json.loads(text)
    except Exception as e:
        raise ValueError(f"Could not parse JSON from LLM response: {e}\nRaw (truncated): {text[:400]}")

def _collect_person_candidates(full_text: str, max_people: int = 40) -> List[Dict[str,Any]]:
    """Optimized: reduced from 60 to 40 for speed"""
    candidates = {}
    patterns = [
        r"\bBy\s+([A-Z][a-z]+(?:\s+[A-Z][a-z]+){0,3})\b",
        r"\bAuthor[s]?:\s*([A-Z][A-Za-z ,\.-]{2,80})",
        r"\bPrepared by[:\s]*([A-Z][A-Za-z ,\.-]{2,80})",
        r"\bReport author[s]?:\s*([A-Z][A-Za-z ,\.-]{2,80})"
    ]
    for pat in patterns:
        for m in re.findall(pat, full_text):
            name = m.strip().strip(",;. ")
            candidates[name] = candidates.get(name, 0) + 6
    for m in re.findall(r"\b([A-Z][a-z]+(?:\s+[A-Z][a-z]+){1,3})\b", full_text):
        name = m.strip()
        if len(name.split()) <= 4:
            candidates.setdefault(name, 0)
            candidates[name] += 1
    collected = []
    for name, score in sorted(candidates.items(), key=lambda kv: -kv[1])[:max_people]:
        occs = []
        for match in re.finditer(rf"\b{re.escape(name)}\b", full_text):
            start = max(0, match.start()-80)
            end = min(len(full_text), match.end()+80)
            ctx = re.sub(r"\s+", " ", full_text[start:end]).strip()
            occs.append(ctx)
            if len(occs) >= 2:  # Reduced from 3 to 2
                break
        collected.append({"name": name, "score": score, "count": len(list(re.finditer(rf"\b{re.escape(name)}\b", full_text))), "examples": occs})
    return collected

def _company_tokens_from_filename_and_headings(path: Path, full_text: str) -> List[str]:
    stem = path.stem.replace("_", " ").replace("-", " ")
    tokens = []
    for tok in re.split(r"[\s,()]+", stem):
        if tok and tok[0].isalpha() and len(tok) > 2:
            tokens.append(tok)
    # Reduced from 30 to 20 lines for speed
    for ln in (full_text.splitlines()[:20]):
        ln = ln.strip()
        if ln and len(ln) < 140 and any(c.isalpha() for c in ln):
            tokens.append(ln)
    # dedupe, prefer shorter tokens first
    seen = []
    for t in tokens:
        short = t.strip()
        if short and short not in seen:
            seen.append(short)
    return seen[:6]

def _matches_company_in_context(name: str, company_tokens: List[str], text: str) -> bool:
    for tok in company_tokens:
        pat1 = rf"{re.escape(name)}\s*,\s*{re.escape(tok)}"
        pat2 = rf"{re.escape(name)}\s*\(\s*{re.escape(tok)}\s*\)"
        pat3 = rf"{re.escape(tok)}[^\n]{{0,30}}{re.escape(name)}"
        if re.search(pat1, text, re.IGNORECASE) or re.search(pat2, text, re.IGNORECASE) or re.search(pat3, text, re.IGNORECASE):
            return True
    return False

def extract_file_metadata_strict_authors(filepath: Path, llm_engine=None, max_chars: int = 20000) -> Dict[str, Any]:
    """
    Fast metadata extraction from document header (first 3 pages).
    Optimized for speed - extracts only essential metadata for chunk tagging.
    
    Returns: dict with filename, file_extension, company_name, publication_year, 
    publication_quarter, title, doc_type, authors, verification, deterministic_evidence
    """
    if llm_engine is None:
        llm_engine = globals().get("llm_engine", None)
    if not llm_engine or not getattr(llm_engine, "available", False):
        raise ValueError("An available LLM engine is required (llm_engine.available == True).")

    p = Path(filepath)
    filename = p.name
    file_ext = p.suffix.lower()

    # Read ONLY first 3 pages for metadata (OPTIMIZATION)
    try:
        if file_ext == ".pdf":
            import fitz
            doc = fitz.open(str(p))
            # KEY CHANGE: Only read first 3 pages instead of all pages
            pages_to_read = min(3, doc.page_count)
            parts = [doc[i].get_text("text") for i in range(pages_to_read)]
            doc.close()
            text = "\n".join(parts)
        elif file_ext == ".docx":
            import docx
            doc = docx.Document(str(p))
            # Read paragraphs but limit total length
            parts = []
            char_count = 0
            for para in doc.paragraphs:
                if para.text.strip():
                    parts.append(para.text)
                    char_count += len(para.text)
                    if char_count > max_chars:
                        break
            text = "\n".join(parts)
        else:
            text = p.read_text(encoding="utf-8", errors="ignore")
    except Exception as e:
        raise RuntimeError(f"Failed to read document text: {e}")

    # Limit to max_chars
    text = text[:max_chars]

    # Deterministic evidence to guide LLM (optimized)
    person_candidates = _collect_person_candidates(text, max_people=40)  # Reduced from 80
    company_tokens = _company_tokens_from_filename_and_headings(p, text)
    year_candidates = re.findall(r"\b(19|20)\d{2}\b", text[:2000])  # Reduced from 4000
    year_candidates = sorted(set(year_candidates), reverse=True)

    # Reduced from 40 to 25 candidates
    compact_people = [{"i": i, "name": c["name"], "count": c["count"], "examples": c["examples"][:1]} 
                      for i, c in enumerate(person_candidates[:25])]

    # Optimized prompt - shorter and more focused
    prompt = (
        "Extract metadata from document header. Return JSON with keys: company_name (string|null), "
        "publication_year (int|null), publication_quarter (Q1..Q4|null), title (string|null), "
        "doc_type (report/transcript/rule/policy/other|null), "
        "authors: [{name:string, affiliation:string}] - include ONLY authors affiliated with the publishing company.\n\n"
        f"FILENAME: {filename}\n"
        f"COMPANY_TOKENS: {json.dumps(company_tokens[:4], ensure_ascii=False)}\n"
        f"YEAR_CANDIDATES: {json.dumps(year_candidates[:3])}\n"
        "PERSON_CANDIDATES:\n" + json.dumps(compact_people[:20], ensure_ascii=False) + "\n\n"
        "DOCUMENT HEADER (first 3 pages):\n" + text[:12000] + "\n\n"  # Reduced from 20000
        "Return JSON only."
    )
    
    try:
        resp = llm_engine.generate_response(prompt, temperature=0.0, max_tokens=500)  # Reduced from 700
    except Exception as e:
        raise RuntimeError(f"LLM call failed: {e}")

    parsed = _parse_json_from_llm(resp)
    if not isinstance(parsed, dict):
        raise ValueError("LLM did not return a JSON object.")

    # Normalize and strictly verify authors
    raw_auths = parsed.get("authors") or []
    verified_authors = []
    verification = []
    cmp_tokens = [t.lower() for t in company_tokens if isinstance(t, str)]
    
    for a in raw_auths:
        if not isinstance(a, dict):
            continue
        name = (a.get("name") or "").strip()
        aff = (a.get("affiliation") or "").strip()
        if not name or not aff:
            verification.append({"name": name or None, "affiliation": aff or None, "kept": False, "reason": "Missing name or affiliation"})
            continue
        
        # Deterministic check: affiliation contains any company token
        aff_l = aff.lower()
        matched_token = None
        for tok in cmp_tokens:
            if tok and tok in aff_l:
                matched_token = tok
                break
        
        # Context check if token not in affiliation
        context_match = False
        if not matched_token:
            if _matches_company_in_context(name, company_tokens, text):
                context_match = True
        
        keep = bool(matched_token or context_match)
        reason = "affiliation matched company token" if matched_token else ("context match" if context_match else "no company evidence")
        verification.append({"name": name, "affiliation": aff, "kept": bool(keep), "reason": reason})
        if keep:
            verified_authors.append({"name": name, "affiliation": aff})

    def _safe_int(v):
        try:
            return int(v)
        except Exception:
            return None

    # Final output - optimized for chunk metadata
    out = {
        "filename": filename,
        "file_extension": file_ext,
        "company_name": parsed.get("company_name") or (company_tokens[0] if company_tokens else None),
        "publication_year": _safe_int(parsed.get("publication_year")) or (int(year_candidates[0]) if year_candidates else None),
        "publication_quarter": parsed.get("publication_quarter") or None,
        "title": parsed.get("title") or None,
        "doc_type": parsed.get("doc_type") or None,
        "authors": verified_authors,
        "verification": verification,
        "deterministic_evidence": {
            "top_person_candidates": [c["name"] for c in person_candidates[:10]],
            "inferred_company_tokens": company_tokens,
            "year_candidates": year_candidates[:5]
        }
    }
    return out

#### 1.6.2 Test metadata extraction

In [ ]:
# Test
# From section 1.5.5 we use llm_small for metadata extraction
# meta = extract_file_metadata_strict_authors(Path("data/earnings_transcripts/barclays_1Q20_earnings_transcript.pdf"), llm_engine=llm_small)
# print(meta)

{'filename': 'barclays_1Q20_earnings_transcript.pdf', 'file_extension': '.pdf', 'company_name': 'Barclays PLC', 'publication_year': 2020, 'publication_quarter': 'Q1', 'title': 'Barclays PLC Q1 2020 Results', 'doc_type': 'report', 'authors': [{'name': 'Tushar Morzaria', 'affiliation': 'Barclays PLC'}], 'verification': [{'name': 'Alvaro Serrano', 'affiliation': 'Morgan Stanley', 'kept': False, 'reason': 'no company evidence'}, {'name': 'Tushar Morzaria', 'affiliation': 'Barclays PLC', 'kept': True, 'reason': 'affiliation matched company token'}, {'name': 'Jonathan Pierce', 'affiliation': 'Numis', 'kept': False, 'reason': 'no company evidence'}, {'name': 'Joseph Dickerson', 'affiliation': 'Jefferies', 'kept': False, 'reason': 'no company evidence'}], 'deterministic_evidence': {'top_person_candidates': ['Tushar Morzaria', 'Jonathan Pierce', 'Alvaro Serrano', 'Morgan Stanley', 'Group Finance Director \nOn', 'United States', 'Investor Relations', 'Numis \nTwo', 'Group Finance Director \nThe'

#### 1.7.1 Enhanced Document Processor

For chunking and downstream quality (accurate boundaries, keep tables together, attach correct section metadata): use use_layout=True.
Do not enable preserve_layout=True in regular/production runs (it’s for debugging or when you explicitly need raw pages). Instead, keep raw pages available in cache only when requested or during development.
Store only the structured metadata (headers, table regions, page_count, char positions) in your pipeline and DB. Avoid storing raw_layout_pages in the vector DB — it’s large and not needed for retrieval.

In [ ]:
# Enhanced Document Processor - Optimized for accurate structure extraction
EXTRACTION_CACHE: Dict[str, Dict[str, Any]] = {}
EXTRACTION_CACHE_DIR = Path(".extract_cache")
EXTRACTION_CACHE_DIR.mkdir(exist_ok=True)

class EnhancedDocumentProcessor:
    """
    Comprehensive document processor optimized for structure-aware chunking.
    Extracts: headers, sections, tables, lists, emphasis for intelligent chunking.
    """
    
    def __init__(self, cache_dir: Optional[Path] = None):
        self.supported_formats = ['.pdf', '.docx', '.txt', '.md', '.csv']
        self.cache_dir = cache_dir or EXTRACTION_CACHE_DIR
        self.cache_dir.mkdir(parents=True, exist_ok=True)

    def clean_text(self, text: str) -> str:
        """Minimal cleaning - preserve structure markers"""
        if not text:
            return ''
        if isinstance(text, (bytes, bytearray)):
            try:
                text = text.decode('utf8')
            except Exception:
                text = text.decode('utf8', errors='replace')
        text = text.replace('\x00', '')
        text = re.sub(r"\r\n|\r", "\n", text)
        # Preserve paragraph breaks
        text = re.sub(r"\n{4,}", "\n\n\n", text)
        return text.strip()

    def _read_text_file(self, p: Path) -> Tuple[str, Dict[str, Any]]:
        """Read plain text with markdown structure detection"""
        text = p.read_text(encoding="utf-8", errors="ignore")
        
        lines = text.split('\n')
        structure = []
        
        for i, line in enumerate(lines):
            stripped = line.strip()
            # Markdown headers
            if stripped.startswith('#'):
                level = len(stripped) - len(stripped.lstrip('#'))
                structure.append({
                    'type': 'header',
                    'level': min(level, 6),
                    'text': stripped.lstrip('#').strip(),
                    'line': i,
                    'char_position': sum(len(l) + 1 for l in lines[:i])
                })
            # Detect lists
            elif re.match(r'^\s*[\-\*\+]\s', line):
                structure.append({
                    'type': 'list_item',
                    'text': stripped.lstrip('-*+ '),
                    'line': i
                })
        
        meta = {
            'structure': structure if structure else None,
            'has_markdown': len(structure) > 0,
            'total_lines': len(lines)
        }
        return text, meta

    def _read_docx(self, p: Path) -> Tuple[str, Dict[str, Any]]:
        """
        DOCX extraction with comprehensive structure preservation.
        Preserves: headings, emphasis, tables, lists.
        """
        try:
            import docx
        except Exception as e:
            raise ImportError("python-docx required. Install: pip install python-docx") from e
        
        doc = docx.Document(str(p))
        parts = []
        structure = []
        tables_data = []
        char_position = 0
        
        for para in doc.paragraphs:
            text = para.text.strip()
            if not text:
                continue
            
            style_name = para.style.name.lower() if para.style else ''
            
            # Detect headers
            if 'heading' in style_name:
                level_match = re.search(r'heading[\s_]*(\d+)', style_name)
                level = int(level_match.group(1)) if level_match else 1
                structure.append({
                    'type': 'header',
                    'level': level,
                    'text': text,
                    'char_position': char_position
                })
                parts.append(f"{'#' * level} {text}")
            else:
                # Check for bold/emphasis
                if any(run.bold for run in para.runs):
                    structure.append({
                        'type': 'emphasis',
                        'style': 'bold',
                        'text': text[:50],
                        'char_position': char_position
                    })
                parts.append(text)
            
            char_position += len(text) + 1
        
        # Extract tables
        for tbl_idx, tbl in enumerate(doc.tables):
            rows = []
            for row in tbl.rows:
                cells = [cell.text.strip() for cell in row.cells]
                if any(cells):
                    rows.append(cells)
            
            if rows:
                # Create markdown table
                table_md = self._format_table(rows)
                parts.append(f"\n{table_md}\n")
                tables_data.append({
                    'index': tbl_idx,
                    'rows': len(rows),
                    'cols': len(rows[0]) if rows else 0,
                    'char_position': char_position
                })
                structure.append({
                    'type': 'table',
                    'index': tbl_idx,
                    'char_position': char_position
                })
                char_position += len(table_md) + 2
        
        combined = '\n\n'.join(parts)
        meta = {
            'structure': structure if structure else None,
            'tables': tables_data if tables_data else None,
            'structure_counts': {
                'headers': sum(1 for s in structure if s['type'] == 'header'),
                'tables': len(tables_data),
                'emphasis': sum(1 for s in structure if s['type'] == 'emphasis')
            }
        }
        return combined, meta

    def _format_table(self, rows: List[List[str]]) -> str:
        """Convert table rows to markdown format"""
        if not rows:
            return ""
        
        # Use first row as header
        header = rows[0]
        body = rows[1:] if len(rows) > 1 else []
        
        md = "| " + " | ".join(header) + " |\n"
        md += "| " + " | ".join(["---"] * len(header)) + " |\n"
        
        for row in body:
            # Pad row if needed
            row_padded = row + [''] * (len(header) - len(row))
            md += "| " + " | ".join(row_padded[:len(header)]) + " |\n"
        
        return md

    def _read_pdf(self, p: Path, use_layout: bool = True, preserve_layout: bool = False) -> Tuple[str, Dict[str, Any]]:
        """
        Enhanced PDF extraction using PyMuPDF with detailed structure tracking.
        
        Args:
            p: Path to PDF file
            use_layout: Extract tables and structure information with font details
            preserve_layout: Keep original layout with positioning
        """
        try:
            import fitz  # PyMuPDF
        except Exception as e:
            raise ImportError("PyMuPDF required. Install: pip install pymupdf") from e
        
        structure = []
        tables_data = []
        parts = []
        char_position = 0
        
        doc = fitz.open(str(p))
        
        try:
            for page_num, page in enumerate(doc, 1):
                if preserve_layout:
                    # Extract with layout preservation - maintains spacing
                    page_text = page.get_text("text", sort=True) or ""
                else:
                    # Standard extraction
                    page_text = page.get_text() or ""
                
                if use_layout:
                    # Extract text with font information for structure detection
                    blocks = page.get_text("dict")["blocks"]
                    
                    for block in blocks:
                        if block.get("type") == 0:  # Text block
                            for line in block.get("lines", []):
                                line_text = ""
                                avg_size = 0
                                bold_count = 0
                                
                                for span in line.get("spans", []):
                                    line_text += span.get("text", "")
                                    avg_size += span.get("size", 0)
                                    if "bold" in span.get("font", "").lower():
                                        bold_count += 1
                                
                                line_text = line_text.strip()
                                if not line_text:
                                    continue
                                
                                # Detect headers by font size or bold
                                spans = line.get("spans", [])
                                if spans:
                                    avg_size = avg_size / len(spans)
                                    # If font is large (>14pt) or bold, likely a header
                                    if avg_size > 14 or bold_count > 0:
                                        structure.append({
                                            'type': 'header',
                                            'text': line_text[:80],
                                            'page': page_num,
                                            'font_size': round(avg_size, 1),
                                            'is_bold': bold_count > 0,
                                            'char_position': char_position
                                        })
                    
                    # Extract tables using PyMuPDF's table detection
                    tabs = page.find_tables()
                    if tabs.tables:
                        for tbl_idx, table in enumerate(tabs.tables):
                            try:
                                rows = table.extract()
                                if rows and any(any(cell for cell in row) for row in rows):
                                    # Clean up rows
                                    clean_rows = []
                                    for row in rows:
                                        clean_row = [str(cell).strip() if cell else '' for cell in row]
                                        if any(clean_row):
                                            clean_rows.append(clean_row)
                                    
                                    if clean_rows:
                                        table_md = self._format_table(clean_rows)
                                        
                                        tables_data.append({
                                            'page': page_num,
                                            'index': tbl_idx,
                                            'rows': len(clean_rows),
                                            'cols': len(clean_rows[0]) if clean_rows else 0
                                        })
                                        structure.append({
                                            'type': 'table',
                                            'page': page_num,
                                            'char_position': char_position
                                        })
                                        parts.append(table_md)
                                        char_position += len(table_md)
                            except Exception:
                                pass  # Skip problematic tables
                
                # Additional header detection by pattern
                lines = page_text.split('\n')
                for line_idx, line in enumerate(lines):
                    stripped = line.strip()
                    # Heuristic: ALL CAPS or short lines (likely headers)
                    if stripped and stripped.isupper() and 3 <= len(stripped.split()) <= 8:
                        # Check if not already detected
                        if not any(s['type'] == 'header' and stripped in s['text'] for s in structure):
                            structure.append({
                                'type': 'header',
                                'text': stripped[:80],
                                'page': page_num,
                                'char_position': char_position + line_idx * 50
                            })
                
                parts.append(page_text)
                char_position += len(page_text) + 1
        
        finally:
            doc.close()
        
        combined = '\n\n'.join(parts)
        meta = {
            'structure': structure if structure else None,
            'tables': tables_data if tables_data else None,
            'structure_counts': {
                'headers': sum(1 for s in structure if s['type'] == 'header'),
                'tables': len(tables_data)
            },
            'preserve_layout': preserve_layout
        }
        return combined, meta

    def _read_csv(self, p: Path) -> Tuple[str, Dict[str, Any]]:
        """Read CSV and convert to markdown table"""
        import csv
        
        with open(p, 'r', encoding='utf-8', errors='ignore') as f:
            reader = csv.reader(f)
            rows = list(reader)
        
        if not rows:
            return "", {'structure': None}
        
        table_md = self._format_table(rows)
        meta = {
            'structure': [{
                'type': 'table',
                'rows': len(rows),
                'cols': len(rows[0]) if rows else 0,
                'char_position': 0
            }],
            'tables': [{'rows': len(rows), 'cols': len(rows[0]) if rows else 0}],
            'structure_counts': {'tables': 1}
        }
        return table_md, meta

    def _cache_key(self, p: Path) -> str:
        """Create unique cache key from file stats"""
        stat = p.stat()
        return f"{p.stem}_{stat.st_size}_{int(stat.st_mtime)}"

    def _cache_load(self, p: Path) -> Optional[Tuple[str, Dict[str, Any]]]:
        """Load from cache if available"""
        key = self._cache_key(p)
        cache_file = self.cache_dir / f"{key}.json"
        
        if not cache_file.exists():
            return None
        
        try:
            with open(cache_file, 'r', encoding='utf-8') as f:
                data = json.load(f)
            return data['text'], data['meta']
        except Exception:
            return None

    def _cache_save(self, p: Path, text: str, meta: Dict[str, Any]) -> None:
        """Save to cache"""
        key = self._cache_key(p)
        cache_file = self.cache_dir / f"{key}.json"
        
        try:
            with open(cache_file, 'w', encoding='utf-8') as f:
                json.dump({'text': text, 'meta': meta}, f)
        except Exception:
            pass

    def extract_text(
        self, 
        file_path: Path,
        use_layout: bool = True,
        preserve_layout: bool = False,
        max_chars: Optional[int] = None,
        use_cache: bool = True
    ) -> Tuple[str, Dict[str, Any]]:
        """
        Extract text with structure from single file.
        
        Args:
            file_path: Path to the file
            use_layout: Extract tables and structure information
            preserve_layout: Preserve original layout with spacing (PDFs only)
            max_chars: Maximum characters to return (None = no limit)
            use_cache: Use cached results if available
            
        Returns: (text, metadata)
        """
        p = Path(file_path)
        ext = p.suffix.lower()
        
        if ext not in self.supported_formats:
            raise ValueError(f"Unsupported format: {ext}")
        
        # Check cache
        if use_cache:
            cached = self._cache_load(p)
            if cached:
                text, meta = cached
                # Apply max_chars if specified
                if max_chars and len(text) > max_chars:
                    text = text[:max_chars]
                    meta['truncated'] = True
                    meta['original_length'] = len(cached[0])
                return text, meta
        
        key = self._cache_key(p)
        if use_cache and key in EXTRACTION_CACHE:
            text = EXTRACTION_CACHE[key]['text']
            meta = EXTRACTION_CACHE[key]['meta']
            # Apply max_chars if specified
            if max_chars and len(text) > max_chars:
                text = text[:max_chars]
                meta['truncated'] = True
                meta['original_length'] = len(EXTRACTION_CACHE[key]['text'])
            return text, meta
        
        # Extract
        if ext == '.pdf':
            text, meta = self._read_pdf(p, use_layout=use_layout, preserve_layout=preserve_layout)
        elif ext == '.docx':
            text, meta = self._read_docx(p)
        elif ext == '.csv':
            text, meta = self._read_csv(p)
        else:
            text, meta = self._read_text_file(p)
        
        text = self.clean_text(text)
        
        # Add universal metadata
        meta.setdefault('filename', p.name)
        meta.setdefault('file_extension', ext)
        meta.setdefault('file_size_bytes', p.stat().st_size)

        # Cache before truncation
        if use_cache:
            EXTRACTION_CACHE[key] = {"text": text, "meta": meta.copy()}
            self._cache_save(p, text, meta)

        # Apply max_chars if specified
        if max_chars and len(text) > max_chars:
            meta['truncated'] = True
            meta['original_length'] = len(text)
            text = text[:max_chars]

        return text, meta

    def process_files(
        self, 
        file_paths: List[Path], 
        use_layout: bool = True,
        show_progress: bool = True
    ) -> List[Tuple[Path, Optional[str], Optional[Dict[str, Any]]]]:
        """
        Batch process files with structure extraction.
        """
        files = [Path(fp) for fp in file_paths]
        results: List[Tuple[Path, Optional[str], Optional[Dict[str, Any]]]] = []
        success = 0

        with tqdm(total=len(files), desc="Extracting structure", disable=not show_progress) as pbar:
            for fp in files:
                try:
                    text, metadata = self.extract_text(fp, use_layout=use_layout, use_cache=True)
                    results.append((fp, text, metadata))
                    success += 1
                    
                    # Show structure info in progress
                    if metadata.get('structure_counts'):
                        counts = metadata['structure_counts']
                        pbar.set_postfix(
                            headers=counts.get('headers', 0),
                            tables=counts.get('tables', 0)
                        )
                    
                except Exception as e:
                    print(f"\n  ⚠️  {fp.name}: {str(e)[:80]}")
                    results.append((fp, None, {"error": str(e), "filename": fp.name}))
                finally:
                    pbar.update(1)

        if show_progress:
            total_structures = sum(
                len(m.get('structure', [])) 
                for _, _, m in results if m and 'structure' in m
            )
            print(f"\n✅ Extraction complete:")
            print(f"   Files: {success}/{len(files)} ({success/len(files)*100:.1f}%)")
            print(f"   Structures detected: {total_structures:,}")

        return results


print("✅ Enhanced Document Processor ready")

✅ Enhanced Document Processor ready


#### 1.7.2. Test Enhanced Document Processor

In [ ]:
# Test extraction with layout preservation
processor = EnhancedDocumentProcessor()
text, meta = processor.extract_text(
    Path("data/earnings_transcripts/barclays_1Q20_earnings_transcript.pdf"),
    use_layout=True,
    preserve_layout=True,
    max_chars=None,
    use_cache=False   # force fresh extraction for verification
)
print("META KEYS:", list(meta.keys()))
print("\n--- RECONSTRUCTED LAYOUT (first 4000 chars) ---\n")
print(text[:4000])

TypeError: EnhancedDocumentProcessor.extract_text() got an unexpected keyword argument 'preserve_layout'

#### 1.7.3 Chunking Function



In [68]:
def chunk_with_structure_and_metadata(
    text: str,
    meta: Dict,
    chunk_size: int = 600,
    overlap: int = 100,
    chunk_id_prefix: str = "chunk"
) -> List[Dict]:
    """
    Structure-aware chunking that preserves layout, tables, and headers.
    Uses meta['structure'] and meta['tables'] from EnhancedDocumentProcessor.
    
    Returns chunks with enriched metadata including section headers.
    """
    if not text:
        return []
    
    # Collect structure boundaries
    boundaries = set([0, len(text)])
    structure_map = {}  # char_pos -> structure info
    
    for s in (meta.get('structure') or []):
        if 'char_position' in s:
            pos = int(s['char_position'])
            boundaries.add(pos)
            structure_map[pos] = s
    
    for t in (meta.get('tables') or []):
        if 'char_position' in t:
            pos = int(t['char_position'])
            boundaries.add(pos)
            structure_map[pos] = t
    
    # Sort boundaries
    b = sorted(boundaries)
    
    chunks = []
    cid = 0
    current_header = None
    
    # Process each section between boundaries
    for i in range(len(b) - 1):
        start_section = b[i]
        end_section = b[i + 1]
        
        # Update current header if we're at a header boundary
        if start_section in structure_map:
            s = structure_map[start_section]
            if s.get('type') == 'header':
                current_header = {
                    'text': s['text'],
                    'level': s.get('level'),
                    'page': s.get('page')
                }
        
        section_text = text[start_section:end_section].strip()
        if not section_text:
            continue
        
        # Check if this section is a table - keep it intact
        is_table = (start_section in structure_map and 
                   structure_map[start_section].get('type') in ['table_region', 'visual_element'])
        
        if is_table or len(section_text) <= chunk_size:
            # Keep small sections or tables intact
            chunk = {
                "chunk_id": f"{chunk_id_prefix}_{cid}",
                "text": section_text,
                "offset": {"start": start_section, "end": end_section},
                "size": len(section_text),
                "section_header": current_header,
                "is_table": is_table,
                "boundary_type": "table" if is_table else "section"
            }
            chunks.append(chunk)
            cid += 1
        else:
            # Sliding window within section
            s_idx = 0
            while s_idx < len(section_text):
                end_idx = min(s_idx + chunk_size, len(section_text))
                
                # Find natural break
                window = section_text[s_idx:end_idx]
                cut = None
                for sep in ['\n\n', '. ', '; ', '\n', ' ']:
                    pos = window.rfind(sep)
                    if pos > chunk_size // 3:
                        cut = s_idx + pos + len(sep)
                        break
                
                actual_end = cut if (cut and cut > s_idx) else end_idx
                chunk_text = section_text[s_idx:actual_end].strip()
                
                if chunk_text:
                    chunk = {
                        "chunk_id": f"{chunk_id_prefix}_{cid}",
                        "text": chunk_text,
                        "offset": {"start": start_section + s_idx, "end": start_section + actual_end},
                        "size": len(chunk_text),
                        "section_header": current_header,
                        "is_table": False,
                        "boundary_type": "natural" if cut else "hard"
                    }
                    chunks.append(chunk)
                    cid += 1
                
                # Advance with overlap
                s_idx = max(actual_end - overlap, s_idx + 1)
    
    # Merge tiny chunks
    merged = []
    for chunk in chunks:
        if (merged and 
            not chunk['is_table'] and 
            not merged[-1]['is_table'] and
            len(chunk['text']) < 120 and 
            len(merged[-1]['text']) + len(chunk['text']) < chunk_size * 1.5):
            
            prev = merged[-1]
            prev['text'] += '\n\n' + chunk['text']
            prev['offset']['end'] = chunk['offset']['end']
            prev['size'] = len(prev['text'])
            prev['boundary_type'] = 'merged'
        else:
            merged.append(chunk)
    
    return merged

print("✅ Structure-aware chunking ready")
print("   Features: header preservation, table integrity, section awareness")

✅ Structure-aware chunking ready
   Features: header preservation, table integrity, section awareness


#### 1.7.4 Test Chunking Function

In [71]:
# Test Chunking with structure
processor = EnhancedDocumentProcessor()

pdf_path = Path("data/pra_rulebook/Algorithmic Trading_16-09-2025.pdf")
text, meta = processor.extract_text(pdf_path, use_layout=True, preserve_layout=False, use_cache=False)

chunks = chunk_with_structure_and_metadata(
    text,
    meta,
    chunk_size=600,
    overlap=100,
    chunk_id_prefix=pdf_path.stem
)

# Print summary
print(f"Document: {pdf_path.name}")
print(f"Page count (meta): {meta.get('page_count')}")
print(f"Detected structure counts: {meta.get('structure_counts')}")
print(f"Detected tables (count): {len(meta.get('tables') or [])}")
print(f"Generated chunks: {len(chunks)}")


Document: Algorithmic Trading_16-09-2025.pdf
Page count (meta): 4
Detected structure counts: {'headers': 29, 'lists': 0, 'emphasis': 13, 'tables': 6}
Detected tables (count): 6
Generated chunks: 26


In [60]:
# Show first 4 chunk summaries
for i, c in enumerate(chunks[:4]):
    hdr = c.get('section_header') and c['section_header'].get('text') or None
    print(f"\nChunk {i+1}: id={c['chunk_id']}, size={c['size']}, is_table={c.get('is_table')}, header={hdr}")
    print("Excerpt:", c['text'][:300].replace("\n", " ") + ("..." if len(c['text']) > 300 else ""))

# Basic assertions
assert len(chunks) > 0, "No chunks produced"
# Offsets are non-decreasing
for prev, cur in zip(chunks, chunks[1:]):
    assert cur['offset']['start'] >= prev['offset']['start'], "Chunk offsets not increasing"

# If tables were detected, assert at least one chunk is marked as table
if meta.get('tables'):
    assert any(c.get('is_table') for c in chunks), "Tables detected in meta but no chunk marked as table"

print("\n✅ chunk_with_structure_and_metadata test passed")


Chunk 1: id=barclays_1Q20_earnings_transcript_0, size=175, is_table=False, header=None
Excerpt: 1  ### Barclays PLC Q1 2020 Results  ### 29 April 2020  Results call Q&A transcript (amended in places to improve accuracy and readability)  ### Alvaro Serrano, Morgan Stanley

Chunk 2: id=barclays_1Q20_earnings_transcript_5, size=574, is_table=False, header=Alvaro Serrano, Morgan Stanley
Excerpt: I’ve got one on provisions and another one on capital. Obviously, the provision taken in the quarter is a very credible and sizeable one, but I’m just asking for some help [around] how we could think about –  in the next few quarters –  the total provisions we might see this year? [Could you] give u...

Chunk 3: id=barclays_1Q20_earnings_transcript_6, size=360, is_table=False, header=Alvaro Serrano, Morgan Stanley
Excerpt: unemployment? And also, on the oil price overlay in the provision, what oil price are you assuming? And the second question on capital. Obviously, you have said that you’re goin

In [54]:
# DIAGNOSTIC: Check what's actually being detected in meta
processor = EnhancedDocumentProcessor()
pdf_path = Path("data/earnings_transcripts/barclays_1Q20_earnings_transcript.pdf")
text, meta = processor.extract_text(pdf_path, use_layout=True, preserve_layout=False, use_cache=False)

print(f"Document: {pdf_path.name}")
print(f"Page count: {meta.get('page_count')}")
print(f"\nStructure counts: {meta.get('structure_counts')}")

# Show structure breakdown
structure = meta.get('structure', [])
print(f"\n=== STRUCTURE ELEMENTS ({len(structure)}) ===")
print(f"By type:")
from collections import Counter
type_counts = Counter(s['type'] for s in structure)
for typ, count in type_counts.items():
    print(f"  {typ}: {count}")

# Sample first 10 structure elements
print(f"\n=== FIRST 10 STRUCTURE ELEMENTS ===")
for i, s in enumerate(structure[:10]):
    print(f"{i+1}. Type: {s['type']}, Page: {s.get('page')}")
    if s['type'] == 'header':
        print(f"   Level: {s.get('level')}, Font size: {s.get('font_size')}, Bold: {s.get('is_bold')}")
    if s['type'] == 'emphasis':
        print(f"   Bold: {s.get('bold')}, Italic: {s.get('italic')}")
    print(f"   Text: {s['text'][:100]}")

# Show tables
tables = meta.get('tables', [])
print(f"\n=== TABLES ({len(tables)}) ===")
for i, t in enumerate(tables[:5]):
    print(f"{i+1}. Type: {t.get('type')}, Page: {t.get('page')}")
    if t.get('type') == 'table_region':
        print(f"   Cells: {len(t.get('cells', []))}")
    if t.get('type') == 'visual_element':
        print(f"   Bbox: {t.get('bbox')}")
        print(f"   Width x Height: {t.get('width'):.1f} x {t.get('height'):.1f}")

# Show text sample with markers
print(f"\n=== TEXT SAMPLE (first 2000 chars with structure markers) ===")
print(text[:2000])

Document: barclays_1Q20_earnings_transcript.pdf
Page count: 14

Structure counts: {'headers': 14, 'lists': 0, 'emphasis': 36, 'tables': 2}

=== STRUCTURE ELEMENTS (50) ===
By type:
  header: 14
  emphasis: 36

=== FIRST 10 STRUCTURE ELEMENTS ===
1. Type: header, Page: 1
   Level: 3, Font size: 10.0, Bold: True
   Text: Barclays PLC Q1 2020 Results
2. Type: header, Page: 1
   Level: 3, Font size: 10.0, Bold: True
   Text: 29 April 2020
3. Type: emphasis, Page: 1
   Bold: True, Italic: False
   Text: Results call Q&A transcript (amended in places to improve accuracy and readability)
4. Type: header, Page: 1
   Level: 3, Font size: 10.0, Bold: True
   Text: Alvaro Serrano, Morgan Stanley
5. Type: emphasis, Page: 1
   Bold: True, Italic: False
   Text: I’ve got one on provisions and another one on capital. Obviously, the provision taken in the quarter
6. Type: emphasis, Page: 2
   Bold: True, Italic: False
   Text: In terms of metrics to look at, I think unemployment is probably the most i

In [ ]:
# DIAGNOSTIC: Check PRA rulebook table detection
processor = EnhancedDocumentProcessor()
pdf_path = Path("data/pra_rulebook/Algorithmic Trading_16-09-2025.pdf")
text, meta = processor.extract_text(pdf_path, use_layout=True, preserve_layout=False, use_cache=False)

print(f"Document: {pdf_path.name}")
print(f"Page count: {meta.get('page_count')}")
print(f"\nDetected tables: {len(meta.get('tables', []))}")

# Show ALL detected tables with details
tables = meta.get('tables', [])
for i, t in enumerate(tables, 1):
    print(f"\n{'='*60}")
    print(f"TABLE {i}:")
    print(f"  Type: {t.get('type')}")
    print(f"  Page: {t.get('page')}")
    
    if t.get('type') == 'table_region':
        cells = t.get('cells', [])
        print(f"  Cells count: {len(cells)}")
        print(f"  Cell samples:")
        for j, cell in enumerate(cells[:5], 1):
            print(f"    {j}. {cell[:60]}")
    
    # Check for table_cell entries that got grouped
    if t.get('bbox'):
        bbox = t['bbox']
        width = bbox[2] - bbox[0] if len(bbox) >= 3 else 0
        height = bbox[3] - bbox[1] if len(bbox) >= 4 else 0
        print(f"  Bbox: {bbox}")
        print(f"  Width x Height: {width:.1f} x {height:.1f}")
        
        # Calculate aspect ratio
        if height > 0:
            aspect_ratio = width / height
            print(f"  Aspect ratio: {aspect_ratio:.2f}")

print(f"\n{'='*60}")
print("\nSample text to see what's between the 'tables':")
print(text[1000:2000])

#### Fix Summary

**Fixed Issues:**
1. **Header Detection**: Now properly detects bold text as headers (level 3) instead of marking them as "emphasis"
   - Added detection for bold + reasonable size (speaker names, section titles)
   - Added pattern matching for "FirstName LastName, Company" format
   - Headers < 80 chars with bold styling are now recognized

2. **Table Detection**: Improved to avoid false positives
   - Added page number exclusion (single digits are not table cells)
   - Added minimum size threshold for visual elements (50x20 pixels)
   - Only mark blocks as potential table cells if font size is below median

3. **Emphasis vs Headers**: Fixed distinction
   - Emphasis only applies to longer text (> 30 chars)
   - Short bold text is now correctly classified as headers

**Run the diagnostic cell below to verify the fix.**

#### Sentiment Analysis functions

Provide a small, robust wrapper around a Transformers sentiment pipeline (FinBERT) to produce per-chunk financial sentiment labels and confidence scores; plus a simple batch runner with progress reporting.

- analyze_fin_sentiment(text: str, model: pipeline) -> Dict[str, float]
  - Inputs: a text string and a Transformers pipeline instance (expected FinBERT-style pipeline returning label/score entries).
  - Preprocessing: truncates input to the first 512 characters to respect model/token limits.
  - Invocation: calls model(text) and expects results shaped as a list containing score dictionaries.
  - Selection: chooses the label with the highest score (max by 'score').
  - Output: returns {'sentiment': <label.lower()>, 'confidence': <rounded_score>} where confidence is rounded to 3 decimals.
  - Error handling: on any exception returns a safe fallback {'sentiment': 'neutral', 'confidence': 0.0}.

- batch_sentiment_analysis(chunks: List[str], model: pipeline) -> List[Dict[str, float]]
  - Inputs: list of chunk texts and a pipeline model.
  - Behavior: iterates over chunks, runs analyze_fin_sentiment for each, and collects results.
  - Progress: displays progress with tqdm ("Sentiment").
  - Output: list of per-chunk sentiment dicts in the same order as input.

- Design notes / assumptions
  - Built for FinBERT-style outputs where model(text) → [[{label, score}, ...]]; the wrapper extracts the best-scoring label accordingly.
  - Truncation to 512 chars prevents token-limit failures at the cost of possibly losing context on very long chunks.
  - Conservative fallback ensures downstream pipeline robustness when the sentiment model fails.

In [13]:
def analyze_fin_sentiment(text: str, model: pipeline) -> Dict[str, float]:
    """Get financial sentiment with confidence scores"""
    try:
        text = text[:512]  # Truncate to model limit
        results = model(text)
        best_result = max(results[0], key=lambda x: x['score'])
        return {
            'sentiment': best_result['label'].lower(),
            'confidence': round(best_result['score'], 3)
        }
    except Exception:
        return {'sentiment': 'neutral', 'confidence': 0.0}

def batch_sentiment_analysis(chunks: List[str], model: pipeline) -> List[Dict[str, float]]:
    """Analyze sentiment for multiple chunks"""
    results = []
    for chunk in tqdm(chunks, desc="Sentiment"):
        results.append(analyze_fin_sentiment(chunk, model))
    return results

#### Embedding Generation Functions

Utilities for producing vector embeddings from text chunks, handling batching, progress reporting, normalization and safe fallback to the embedding model loaded in Section 1.5.

- EMBEDDING_MODEL
  - List of candidate embedding model names used as configuration/choices; BGE v1.5 is marked as the selected model.

- generate_embeddings(chunks: List[str], model: SentenceTransformer) -> np.ndarray
  - Purpose: Encode a list of texts into embeddings using a SentenceTransformer instance.
  - Output: numpy.ndarray of embeddings (empty array when input empty).
  - Behavior: Calls model.encode(..., show_progress_bar=True, normalize_embeddings=True).
  - Assumption: Caller supplies a valid SentenceTransformer (no model loading/fallback inside this function).

- embed_texts(texts: List[str], model: SentenceTransformer = None, batch_size: int = 32, show_progress: bool = True, normalize: bool = True) -> List[List[float]]
  - Purpose: Batch-safe embedding runner with optional progress display and normalization.
  - Output: flat list of embedding vectors (each a Python list of floats).

- embed_chunks(chunks: List[Dict], **kwargs) -> List[Dict]
  - Purpose: High-level helper to embed a list of chunk dicts while preserving chunk metadata.
  - Behavior: Extracts texts, calls embed_texts, zips embeddings back into chunk dict copies.

In [ ]:
# Generate embeddings
def generate_embeddings(chunks: List[str], model: SentenceTransformer) -> np.ndarray:
    """Generate embeddings for text chunks"""
    if not chunks:
        return np.array([])
    return model.encode(chunks, show_progress_bar=True, normalize_embeddings=True)

# Embedding configuration
EMBEDDING_MODEL = [
    'all-MiniLM-L6-v2',
    'thenlper/gte-large',
    'sentence-transformers/all-mpnet-base-v2',
    'BAAI/bge-large-en-v1.5',  # Selected model (index 3)
]

# Embed texts with progress and normalization
def embed_texts(
    texts: List[str], 
    model: SentenceTransformer = None,
    batch_size: int = 32, 
    show_progress: bool = True, 
    normalize: bool = True
) -> List[List[float]]:
    """Embed texts with progress and normalization."""
    if not texts: 
        return []
    
    # Use provided model or fall back to global embedding_model
    if model is None:
        if 'embedding_model' not in globals():
            raise ValueError("No embedding model available. Please run Section 1.5 first.")
        model = globals()['embedding_model']
    
    vectors = []
    iter_range = range(0, len(texts), batch_size)
    
    if show_progress:
        iter_range = tqdm(iter_range, desc="Embedding", unit="batch")
    
    for i in iter_range:
        batch = texts[i:i+batch_size]
        emb = model.encode(batch, show_progress_bar=False, normalize_embeddings=normalize)
        vectors.extend(emb.tolist())
    
    return vectors

# Embed chunks with metadata preservation
def embed_chunks(chunks: List[Dict], **kwargs) -> List[Dict]:
    """Embed chunks while preserving metadata."""
    texts = [chunk['text'] for chunk in chunks]
    embeddings = embed_texts(texts, **kwargs)
    return [{**chunk, 'embedding': emb} for chunk, emb in zip(chunks, embeddings)]

print("✅ Embedding generation functions ready")
print(f"   Using model: {CONFIG['embedding_model']}")
print(f"   Model loaded in Section 1.5: {'embedding_model' in globals()}")

✅ Embedding generation functions ready
   Using model: BAAI/bge-large-en-v1.5
   Model loaded in Section 1.5: True


#### ChromaDB setup with auto-repair

Initialize a persistent ChromaDB client, ensure the store directory exists, handle common DB errors.

- Key functions
  - create_chroma_client(persist_dir: str) -> chromadb.Client
    - Ensures the persistence directory exists.
    - Attempts to instantiate a chromadb. PersistentClient and verify responsiveness via client.list_collections().
    - On failure, logs the error, removes the existing store directory (shutil.rmtree), recreates it, re-instantiates the PersistentClient, and reports repair success. Returns the client instance.

  - get_or_create_collection(client: chromadb.Client, name: str) -> chromadb.Collection
    - Tries to retrieve an existing collection by name.
    - If retrieval fails, creates a new collection with basic metadata (description) and returns it.
    - Encapsulates collection creation/retrieval with simple error handling.

  - display_database_status(client: chromadb.Client, store_dir: Path) -> None
    - Prints a structured status report: ChromaDB version, database path, total collections.
    - Iterates collections to query per-collection counts (collection.count()), aggregates total chunk/document counts, and prints per-collection diagnostics.
    - Catches and reports exceptions per collection and for the overall status check.

- Initialization flow (cell-level behavior)
  - Calls create_chroma_client with STORE_DIR; if it succeeds, sets a global client.
  - Optionally selects/creates a collection when CONFIG contains a collection_name, storing it in the global variable collection.
  - Calls display_database_status to output diagnostics and guide next steps (e.g., run document processing if no collections).
  - All major failure points print user-facing guidance (permissions, locks, readonly issues) and encourage re-running the repair path or restarting the kernel.

- Error-handling & repair strategy
  - Proactive: tests client responsiveness immediately after creation.
  - Recoverable-failure approach: on error tries a filesystem-level repair (delete+recreate store directory) and reinstantiates the client.
  - Informative: prints specific hints for common failure modes (readonly, permission, lock/busy).

- Side effects & outputs
  - Creates the persistence directory if missing.
  - May remove and recreate the chroma store directory during "repair".
  - Produces a human-readable database status report; returns client and collection objects for downstream cells.

In [ ]:
# ChromaDB setup with auto-repair - Clean Implementation
def create_chroma_client(persist_dir: str) -> chromadb.Client:
    """Create persistent ChromaDB client with auto-repair capability."""
    persist_path = Path(persist_dir)
    persist_path.mkdir(parents=True, exist_ok=True)

    try:
        client = chromadb.PersistentClient(path=str(persist_path))
        # Test client responsiveness
        client.list_collections()
        return client
    except Exception as e:
        print(f"ChromaDB issue detected: {e}")
        print(f"Attempting database repair...")
        
        # Simple repair: recreate directory
        if persist_path.exists():
            shutil.rmtree(persist_path, ignore_errors=True)
        persist_path.mkdir(parents=True, exist_ok=True)
        
        client = chromadb.PersistentClient(path=str(persist_path))
        print(f"Database repaired successfully")
        return client

def get_or_create_collection(client: chromadb.Client, name: str) -> chromadb.Collection:
    """Get existing collection or create new one."""
    try:
        return client.get_collection(name)
    except Exception:
        return client.create_collection(
            name=name,
            metadata={"description": "Financial documents collection"}
        )

def display_database_status(client: chromadb.Client, store_dir: Path) -> None:
    """Display comprehensive database status information."""
    print(f"ChromaDB Status Report")
    print(f"{'='*50}")
    
    try:
        # Get collections
        collections = client.list_collections()
        print(f"Database location: {store_dir}")
        print(f"ChromaDB version: {chromadb.__version__}")
        print(f"Total collections: {len(collections)}")
        
        if collections:
            print(f"\nCollection Details:")
            total_documents = 0
            
            for i, col in enumerate(collections, 1):
                try:
                    collection_obj = client.get_collection(col.name)
                    count = collection_obj.count()
                    total_documents += count
                    print(f"   {i}. {col.name}: {count:,} chunks")
                except Exception as e:
                    print(f"   {i}. {col.name}: Error ({str(e)[:30]}...)")
            
            print(f"\nTotal documents: {total_documents:,}")
        else:
            print(f"\nNo collections found")
            print(f"Run document processing (Sections 3-4) to populate database")
            
    except Exception as e:
        print(f"Status check failed: {e}")
        
    print(f"{'='*50}")

# Initialize ChromaDB client
print("Initializing ChromaDB...")
try:
    client = create_chroma_client(str(STORE_DIR))
    print(f"ChromaDB client ready")
    
    # Initialize collection variable for global use
    collection = None
    if CONFIG.get('collection_name'):
        try:
            collection = get_or_create_collection(client, CONFIG['collection_name'])
            print(f"Collection '{CONFIG['collection_name']}' ready")
        except Exception as e:
            print(f"Collection setup warning: {e}")
    
    # Display comprehensive status
    display_database_status(client, STORE_DIR)
    
except Exception as e:
    print(f"ChromaDB initialization failed: {e}")
    print(f"Check your environment and permissions")

Initializing ChromaDB...
ChromaDB client ready
ChromaDB Status Report
Database location: /Users/rodrigograndy/Desktop/coding_projects/posts_website_drafts/rag_boe/chroma_store
ChromaDB version: 1.0.21
Total collections: 2

Collection Details:
   1. transcrips_barclasys: 5,591 chunks
   2. pra_rules: 2,991 chunks

Total documents: 8,582


#### Collection management functions

Small utility layer that wraps common ChromaDB collection operations with defensive checks, friendly messages and simple error handling. Exposes four primary helpers for listing, creating, inspecting and selecting collections.

Key functions

- list_existing_collections(client) -> List[str]
  - Calls client.list_collections(), extracts collection.name; on exception prints an error and returns an empty list.

- create_new_collection(client, collection_name, description: str = None) -> chromadb.Collection | None
  - Purpose: Create (or retrieve) a named collection and attach simple metadata.
  - Behavior:
    - First checks existing collections (via list_existing_collections) and returns the existing collection if present.
    - Otherwise calls client.create_collection(name=..., metadata={...}).
    - Contains targeted error handling: inspects exception text and prints actionable messages for common states (read-only store, permission issues, DB locks/busy); returns None on failure.

- get_collection_info(collection: chromadb.Collection) -> Dict[str, Any]
  - Purpose: Fetch lightweight diagnostics for a collection.
  - Behavior: Calls collection.get() and returns a dict with name, metadata, document_count (len(ids) if present) and sample_ids (first 5). On error returns a dict with the collection name and an error string.

- select_or_create_collection(client, collection_name: str = None) -> chromadb.Collection
  - Purpose: Convenience chooser that uses a supplied name or CONFIG['collection_name'] to pick or create a collection.
  - Behavior: Lists existing collections, returns existing collection if present, otherwise calls create_new_collection to create and return a new collection (prints status messages).


In [15]:
# Collection management functions with enhanced error handling
def list_existing_collections(client: chromadb.Client) -> List[str]:
    """List all existing collections in the ChromaDB instance"""
    try:
        collections = client.list_collections()
        return [collection.name for collection in collections]
    except Exception as e:
        print(f"Error listing collections: {e}")
        return []
# Create new collection with error handling
def create_new_collection(client: chromadb.Client, collection_name: str, description: str = None) -> chromadb.Collection:
    """Create a new collection with the specified name, with enhanced error handling"""
    try:
        # Check if collection already exists
        existing_collections = list_existing_collections(client)
        if collection_name in existing_collections:
            print(f"Collection '{collection_name}' already exists. Retrieving existing collection.")
            return client.get_collection(collection_name)

        # Create new collection
        metadata = {"description": description or f"Financial documents collection: {collection_name}"}
        collection = client.create_collection(name=collection_name, metadata=metadata)
        print(f"Created new collection: '{collection_name}'")
        return collection

    except Exception as e:
        error_msg = str(e).lower()

        # Handle specific database errors
        if "readonly" in error_msg or "1032" in error_msg:
            print(f"Database is read-only. Attempting to repair...")
            print("Run the database repair cell above to fix this issue.")
            print("Alternative: Restart the notebook kernel and run the repair cell.")

        elif "permission" in error_msg or "access" in error_msg:
            print(f"Permission denied. Database directory may need permission fixes.")
            print("Run the database repair cell above to fix permissions.")

        elif "lock" in error_msg or "busy" in error_msg:
            print(f"Database is locked (another process may be using it).")
            print("Close other notebook instances and try again.")

        else:
            print(f"Unexpected error creating collection '{collection_name}': {e}")
            print("Try running the database repair cell above.")

        return None
# Get collection info with document count
def get_collection_info(collection: chromadb.Collection) -> Dict[str, Any]:
    """Get information about a collection including document count"""
    try:
        data = collection.get()
        return {
            'name': collection.name,
            'metadata': collection.metadata,
            'document_count': len(data['ids']) if data['ids'] else 0,
            'sample_ids': data['ids'][:5] if data['ids'] else []
        }
    except Exception as e:
        return {'name': collection.name, 'error': str(e)}

def select_or_create_collection(client: chromadb.Client, collection_name: str = None) -> chromadb.Collection:
    """Select existing collection or create new one based on name"""
    if not collection_name:
        collection_name = CONFIG['collection_name']

    existing_collections = list_existing_collections(client)

    if collection_name in existing_collections:
        print(f"Using existing collection: '{collection_name}'")
        return client.get_collection(collection_name)
    else:
        print(f"Collection '{collection_name}' not found. Creating new collection.")
        return create_new_collection(client, collection_name)



#### 1.8 Rag workflow variables.

Initialize variables that would normally be created in Section 3 (document processing)

- Small compatibility/bootstrapping cell that ensures later sections (5–7) can run even if document-processing (Sections 3–4) was skipped. It defines a set of global placeholders, preserves any existing values, and emits brief runtime hints.

- all_chunks -> dict  
  - Document-indexed structure mapping each file path to its extracted text, chunk list and merged metadata. Populated in Section 3 but initialized here as an empty dict to avoid NameError in downstream code.

- all_embeddings -> list  
  - Linear list/array of embeddings for all chunks. Downstream code expects embeddings to be present; initialized empty.

- all_sentiments -> list  
  - Per-chunk sentiment result list (FinBERT outputs). Initialized empty so sentiment-dependent steps can still run defensively.

- results -> list  
  - General-purpose list used earlier for (path, text, metadata) tuples from extraction. Initialized empty for compatibility.

- all_chunk_texts -> list  
  - Flat list of chunk texts (used by embedding/sentiment batch runners and Section 4.6). Added explicitly for Section 4.6 compatibility.

- bm25_index -> None or index object  
  - Placeholder for an in-memory BM25 index; set to None when not built.

Behavior and design notes
- Guarded initialization: each variable is only created if absent in globals() so re-running cells or preserving already-processed data is safe.

In [16]:
# Initialize empty variables for Section 5-7 compatibility
if 'all_chunks' not in globals():
    all_chunks = {}  # Empty dict - would be populated by Section 3
    print("Initialized all_chunks (empty - populate via Section 3 if needed)")

if 'all_embeddings' not in globals():
    all_embeddings = []  # Empty list - would be populated by Section 3
    print("Initialized all_embeddings (empty - populate via Section 3 if needed)")
if 'all_sentiments' not in globals():
    all_sentiments = []  # Empty list - would be populated by Section 3
    print("Initialized all_sentiments (empty - populate via Section 3 if needed)")

if 'results' not in globals():
    results = []  # Empty list - would be populated by Section 3
    print("Initialized results (empty - populate via Section 3 if needed)")
if 'all_chunk_texts' not in globals():
    all_chunk_texts = []  # Empty list - would be populated by Section 3.10.3
    print("Initialized all_chunk_texts (empty - populate via Section 3 if needed)")

if 'bm25_index' not in globals():
    bm25_index = None  # Would be built in Section 5
    print("Initialized bm25_index (None - will be built in Section 5)")
print("All core functions defined and available")
print("Document discovery and metadata extraction")
print("Enhanced document processing (PDF, DOCX, TXT, MD, CSV)")
print("Text chunking with boundary respect")
print("Financial sentiment analysis")
print("Embedding generation with BGE model")
print("ChromaDB storage with metadata sanitization")
print("Collection management utilities")
print("Ready for use across all workflow scenarios!")
print("Section 5-7 compatibility variables initialized")
print("Added all_chunk_texts for section 4.6 compatibility")

Initialized all_chunks (empty - populate via Section 3 if needed)
Initialized all_embeddings (empty - populate via Section 3 if needed)
Initialized all_sentiments (empty - populate via Section 3 if needed)
Initialized results (empty - populate via Section 3 if needed)
Initialized all_chunk_texts (empty - populate via Section 3 if needed)
Initialized bm25_index (None - will be built in Section 5)
All core functions defined and available
Document discovery and metadata extraction
Enhanced document processing (PDF, DOCX, TXT, MD, CSV)
Text chunking with boundary respect
Financial sentiment analysis
Embedding generation with BGE model
ChromaDB storage with metadata sanitization
Collection management utilities
Ready for use across all workflow scenarios!
Section 5-7 compatibility variables initialized
Added all_chunk_texts for section 4.6 compatibility


## 2. Document Processing

This section includes functions for loading files, extracting and parsing text, chunking, doing sentiment analysis on chunks, creating embeddings, and storing them in ChromaDB.

- Designed to handle text extraction from different file types (PDF, DOCX, TXT, .MD, .CSV)
- Currently, it has been tested with PDF files only.

#### 2.2 Directory Verification

Quick diagnostics to verify the presence and contents of the configured data directories before running document processing.

- Inputs: uses globals established in Section 1.4 — DATA_DIR (root data folder) and file_directories (list of subdirectory Path objects).

In [21]:
print("Data directories:")
for dir_path in file_directories:
    if dir_path.exists():
        files = [f for f in dir_path.glob('*.pdf')]  # Adjust extension as needed
        print(f"{dir_path.name}: {len(files)} PDF files")
    else:
        print(f"{dir_path.name}: Not found")

Data directories:
pra_rulebook: 41 PDF files


#### 2.3 Extract Metadata from file names and content

The class provides a single, consolidated extractor that pulls structured metadata from a file path (filename) and optionally from the document text, with domain-focused cues for financial documents and conflict resolution between filename and content-derived values.

- Key methods
  - __init__
    - Loads spaCy (with a fallback downloader) into self.nlp when available.
    - Initializes domain vocabularies: financial_terms and document_type_patterns used by content classifiers.

  - _load_nlp_model
    - Loads en_core_web_sm from spaCy package.

  - extract_comprehensive_metadata(filepath: Path, text: str | None) -> Dict
    - Top-level method: always extracts filename metadata, then if text is provided runs content extraction, merges content fields into the filename-derived metadata and applies conflict resolution.
    - Returns a unified metadata dict suitable for merging into pipeline records.

  - _extract_filename_metadata(filepath: Path) -> Dict
    - Parses filename.stem to infer company (first underscore part), year (4- or 2-digit normalization), quarter (Q1–Q4 patterns) and a doc_type heuristic based on extension and name tokens.
    - Returns filename, file_extension, company, year, quarter, doc_type and extraction_source='filename'.

  - _classify_doc_type_from_filename(filepath, name) -> str
    - Heuristic type classifier by extension and keywords in filename (e.g., 'transcript', 'meeting').

  - _extract_content_metadata(text: str) -> Dict
    - Extracts dates/years and quarter mentions from text.
    - Runs document-type scoring via _classify_doc_type_from_content.
    - Invokes spaCy entity extraction (_extract_entities_spacy) when available.
    - Detects financial terms via _extract_financial_terms.
    - Computes simple content stats (word_count, has_financial_data).
    - Returns content-derived fields prefixed (e.g., primary_year_content, years_content, quarters_content, doc_type_content, extracted_entities, financial_terms, word_count).

  - _classify_doc_type_from_content(text: str) -> Optional[str]
    - Scores document_type_patterns against text (counts pattern matches) and returns the highest-scoring doc_type when present.

  - _extract_entities_spacy(text: str) -> Dict
    - Uses spaCy to extract limited sets of ORG (companies), PERSON, MONEY and PERCENT entities from a text slice (performance-limited to ~5k chars), deduplicates and caps results, returns under key 'extracted_entities'.

  - _extract_financial_terms(text: str) -> Dict
    - Simple keyword presence detection using the configured financial_terms categories; returns categorized matches as 'financial_terms'.

  - _resolve_conflicts(metadata: Dict) -> Dict
    - Merges filename and content outputs with deterministic precedence: prefers content-derived doc_type/year/quarter when present, annotates sources (e.g., doc_type_source, year_source, quarter_source).
    - Produces a resolved metadata dict ready for downstream storage.

- Outputs / fields to expect
  - Combined metadata includes filename, file_extension, company, year, quarter, doc_type plus content-derived fields when text provided: primary_year_content, years_content, quarters_content, doc_type_content, extracted_entities, financial_terms, word_count, has_financial_data, and explicit *_source fields after conflict resolution.

- Integration note
  - The notebook initializes consolidated_metadata_extractor = ConsolidatedMetadataExtractor() and Section 2.7 calls extract_comprehensive_metadata(path, text) to produce the merged metadata used downstream (chunking, storage).



In [25]:
# Extract Metadata from file names AND content
class ConsolidatedMetadataExtractor:
    """Extract metadata from filenames and document content with financial domain focus"""
    
    def __init__(self):
        self.nlp = self._load_nlp_model()
        
        # Financial terms for content analysis
        self.financial_terms = {
            'metrics': ['revenue', 'profit', 'earnings', 'ebitda', 'cash flow', 'margin'],
            'instruments': ['bonds', 'stocks', 'derivatives', 'securities'],
            'regulations': ['compliance', 'regulatory', 'pra', 'basel']
        }
        
        # Document type patterns
        self.document_type_patterns = {
            'earnings_transcript': ['earnings call', 'quarterly results', 'q1', 'q2', 'q3', 'q4'],
            'annual_report': ['annual report', 'form 10-k', '10-k'],
            'regulatory_filing': ['form 8-k', '8-k', 'sec filing']
        }
    
    def _load_nlp_model(self):
        """Load spaCy model from system installation."""
        try:
            return spacy.load("en_core_web_sm")
        except OSError:
            raise FileNotFoundError(
                f"spaCy model 'en_core_web_sm' not found. "
                f"Please install it by running: python -m spacy download en_core_web_sm"
            )
    
    def extract_comprehensive_metadata(self, filepath: Path, text: str = None) -> Dict[str, Any]:
        """
        Extract metadata from both filename and content
        """
        # Start with filename metadata
        metadata = self._extract_filename_metadata(filepath)
        
        # Add content metadata if text provided
        if text:
            content_meta = self._extract_content_metadata(text)
            metadata.update(content_meta)
            metadata = self._resolve_conflicts(metadata)
        
        return metadata
    
    def _extract_filename_metadata(self, filepath: Path) -> Dict[str, Any]:
        """Extract metadata from filename"""
        name = filepath.stem.lower()
        parts = name.split('_')
        
        metadata = {
            'filename': filepath.name,
            'file_extension': filepath.suffix.lower(),
            'extraction_source': 'filename'
        }
        
        # Extract company (first part)
        metadata['company'] = parts[0] if parts else 'unknown'
        
        # Extract year (4-digit or 2-digit)
        year_match = re.search(r'(20\d{2})|(\d{2})', name)
        if year_match:
            year = year_match.group(0)
            if len(year) == 2:
                year_int = int(year)
                year = str(2000 + year_int if year_int <= 50 else 1900 + year_int)
            metadata['year'] = year
        else:
            metadata['year'] = 'unknown'
        
        # Extract quarter
        quarter_match = re.search(r'([1-4]q|q[1-4])', name, re.IGNORECASE)
        if quarter_match:
            q = quarter_match.group(1).upper()
            metadata['quarter'] = f"Q{q[0]}" if q[0].isdigit() else q
        else:
            metadata['quarter'] = 'unknown'
        
        # Classify document type
        metadata['doc_type'] = self._classify_doc_type_from_filename(filepath, name)
        
        return metadata
    
    def _classify_doc_type_from_filename(self, filepath: Path, name: str) -> str:
        """Classify document type from filename"""
        if filepath.suffix.lower() == '.pdf':
            if 'transcript' in name:
                return 'earnings_transcript'
            elif 'meeting' in name:
                return 'analyst_meeting'
            return 'financial_document'
        elif filepath.suffix.lower() in ['.docx', '.txt', '.md']:
            return 'text_document'
        return 'unknown'
    
    def _extract_content_metadata(self, text: str) -> Dict[str, Any]:
        """Extract metadata from document content"""
        content_meta = {'extraction_source': 'content'}
        
        # Extract dates from content
        years = re.findall(r'\b(20\d{2})\b', text)
        if years:
            content_meta['years_content'] = sorted(set(years), reverse=True)[:5]
            content_meta['primary_year_content'] = years[0]
        
        # Extract quarters
        quarters = re.findall(r'\b([Q1-4]|[1-4]Q)\b', text, re.IGNORECASE)
        if quarters:
            content_meta['quarters_content'] = list(set(q.upper() for q in quarters))
        
        # Document type from content
        doc_type = self._classify_doc_type_from_content(text)
        if doc_type:
            content_meta['doc_type_content'] = doc_type
        
        # Entity extraction with spaCy
        if self.nlp:
            entities = self._extract_entities_spacy(text)
            content_meta.update(entities)
        
        # Financial terms
        financial_info = self._extract_financial_terms(text)
        content_meta.update(financial_info)
        
        # Content statistics
        content_meta['word_count'] = len(text.split())
        content_meta['has_financial_data'] = bool(financial_info.get('financial_terms'))
        
        return content_meta
    
    def _classify_doc_type_from_content(self, text: str) -> Optional[str]:
        """Classify document type from content"""
        text_lower = text.lower()
        
        type_scores = {}
        for doc_type, patterns in self.document_type_patterns.items():
            score = sum(len(re.findall(re.escape(p.lower()), text_lower)) for p in patterns)
            if score > 0:
                type_scores[doc_type] = score
        
        return max(type_scores, key=type_scores.get) if type_scores else None
    
    def _extract_entities_spacy(self, text: str) -> Dict[str, Any]:
        """Extract entities using spaCy"""
        entities = {
            'companies': [],
            'people': [],
            'money': [],
            'percentages': []
        }
        
        try:
            doc = self.nlp(text[:5000])  # Limit for performance
            for ent in doc.ents:
                if ent.label_ == 'ORG':
                    entities['companies'].append(ent.text)
                elif ent.label_ == 'PERSON':
                    entities['people'].append(ent.text)
                elif ent.label_ == 'MONEY':
                    entities['money'].append(ent.text)
                elif ent.label_ == 'PERCENT':
                    entities['percentages'].append(ent.text)
            
            # Deduplicate and limit
            for key in entities:
                entities[key] = list(set(entities[key]))[:5]
            
            return {'extracted_entities': entities}
        except Exception as e:
            return {'extracted_entities': entities}
    
    def _extract_financial_terms(self, text: str) -> Dict[str, Any]:
        """Extract financial terms from content"""
        text_lower = text.lower()
        
        found_terms = {}
        for category, terms in self.financial_terms.items():
            found = [term for term in terms if term in text_lower]
            if found:
                found_terms[category] = found
        
        return {'financial_terms': found_terms} if found_terms else {}
    
    def _resolve_conflicts(self, metadata: Dict[str, Any]) -> Dict[str, Any]:
        """Resolve conflicts between filename and content metadata"""
        resolved = metadata.copy()
        
        # Prefer content for doc_type if confidence is high
        doc_type_content = metadata.get('doc_type_content')
        doc_type_filename = metadata.get('doc_type')
        if doc_type_content:
            resolved['doc_type'] = doc_type_content
            resolved['doc_type_source'] = 'content'
        else:
            resolved['doc_type_source'] = 'filename'
        
        # Prefer content for year if available
        year_content = metadata.get('primary_year_content')
        if year_content:
            resolved['year'] = year_content
            resolved['year_source'] = 'content'
        else:
            resolved['year_source'] = 'filename'
        
        # Prefer content for quarter if available
        quarters_content = metadata.get('quarters_content')
        if quarters_content:
            resolved['quarter'] = quarters_content[0]
            resolved['quarter_source'] = 'content'
        else:
            resolved['quarter_source'] = 'filename'
        
        return resolved

# Initialize
try:
    consolidated_metadata_extractor = ConsolidatedMetadataExtractor()
    print("Consolidated Metadata Extractor initialized")
    print("• Filename parsing (company, year, quarter, doc_type)")
    print("• Content analysis (dates, entities, financial terms)")
    print("• spaCy NLP entity extraction")
    print("• Conflict resolution (content takes precedence)")
    print("• Financial domain optimization")
except Exception as e:
    print(f"Initialization failed: {e}")

Consolidated Metadata Extractor initialized
• Filename parsing (company, year, quarter, doc_type)
• Content analysis (dates, entities, financial terms)
• spaCy NLP entity extraction
• Conflict resolution (content takes precedence)
• Financial domain optimization


#### 2.4 Document Text Extraction and Parsing Verification

In [40]:
# Verify the processor is available
if 'EnhancedDocumentProcessor' not in dir():
    print("❌ EnhancedDocumentProcessor not found")
    print("   Please run Section 1.7.4 first")
else:
    print("✅ Document processor ready")
    print(f"   Supported formats: {EnhancedDocumentProcessor().supported_formats}")
    print("   • PDF extraction (simple and layout modes)")
    print("   • DOCX, TXT, MD, CSV support")
    print("   • Automatic text cleaning")

✅ Document processor ready
   Supported formats: ['.pdf', '.docx', '.txt', '.md', '.csv']
   • PDF extraction (simple and layout modes)
   • DOCX, TXT, MD, CSV support
   • Automatic text cleaning


#### 2.5 Run Data Processor

Orchestrates document discovery and batch extraction, producing the notebook-wide results list used by downstream steps (chunking, metadata merging, embeddings, storage). Class is defined in section 1.7.4.
- pdf_mode behavior: accepts "auto" | "layout" | "simple". In "auto" a heuristic (e.g., page_count > 3) picks layout extraction (detailed spans) vs simple text extraction; exceptions fall back to simpler mode and extraction metadata records extraction_mode and optional layout spans.
- extract_text outputs: cleaned text (raises on missing/empty), and metadata dict describing extraction_mode and, for layout PDF, a layout list of span dicts.
- process_files outputs: for each file a tuple (Path, text or None, metadata or {"error": "..."}); it captures per-file exceptions rather than aborting the full run and prints a simple progress summary.
- Integration note: this cell sets the global results variable expected by sections 2.6–2.10. Downstream logic assumes results contains (path, text, extraction_meta) tuples for metadata merging, chunking, sentiment and embedding.

In [41]:
# Default: auto-detect PDF complexity. Options: "auto", "layout", "simple". Auto should detect complexity of files (needs to be tested more)
processor = EnhancedDocumentProcessor()
files = discover_documents(file_directories)
results = processor.process_files(files, pdf_mode="auto")

Supported file extensions: ['.pdf', '.docx', '.txt', '.md', '.csv']
Data directories provided: [PosixPath('/Users/rodrigograndy/Desktop/coding_projects/posts_website_drafts/rag_boe/data/pra_rulebook')]
Searching directory: /Users/rodrigograndy/Desktop/coding_projects/posts_website_drafts/rag_boe/data/pra_rulebook
  Contents of /Users/rodrigograndy/Desktop/coding_projects/posts_website_drafts/rag_boe/data/pra_rulebook: ['Market Risk_16-09-2025.pdf', 'Own Funds and Eligible Liabilities (CRR)_16-09-2025.pdf', 'Benchmarking of Internal Approaches_16-09-2025.pdf', 'Public Disclosure_16-09-2025.pdf', 'Credit Valuation Adjustment Risk (CRR)_16-09-2025.pdf', 'Trading Book (CRR)_16-09-2025.pdf', 'Supervised Run-Off_16-09-2025.pdf', 'Reporting Pillar 2_16-09-2025.pdf', 'Large Exposures_16-09-2025.pdf', 'Resolution Assessment_16-09-2025.pdf', 'General Organisational Requirements_16-09-2025.pdf', 'Liquidity Coverage Requirement - UK Designated Investment Firms_16-09-2025.pdf', 'Designation_16-09-2

Processing files:   0%|          | 0/41 [00:00<?, ?it/s]

Processed 41 files (41 succeeded, 0 failed)


#### 2.6 Check results

Quick verification of the extraction output produced by the data processor (results list produced in 2.5).
- Input: expects the global results variable which is a list of (Path, text, metadata) tuples.
- Behavior:
  - Inspects the first result (results[:1]) to avoid iterating the whole corpus.
  - If the file's metadata contains a 'layout' key (PDF layout extraction), it prints the document name, the number of layout spans and a 500-char sample of the extracted text.
  - Otherwise, if plain text is present, it prints the document name and a 500-char sample of the text.
- Role in pipeline: lightweight, read-only sanity check to confirm extraction mode (layout vs simple) and that non-empty text was returned; no state mutation or downstream changes.

In [42]:
# Check results with layout data
for path, text, metadata in results[:1]:
    if metadata and 'layout' in metadata:
        print(f"\nDocument: {path.name}")
        print(f"Extracted {len(metadata['layout'])} text spans")
        print(f"Sample text: {text[:500]}...")
    elif text:
        print(f"\nDocument: {path.name}")
        print(f"Text: {text[:500]}...")


Document: Market Risk_16-09-2025.pdf
Extracted 204 text spans
Sample text: Prudential Regulation Authority Rulebook
Part
Market Risk
Printed on: 16/09/2025
Rulebook at: 16/09/2025
Related links
PS7/13 - Strengthening capital standar
ds: implementing CRD IV, feedback an
d final rules
https://www.bankofengland.co.uk/prudential-regula
tion/publication/2013/strengthening-capital-standar
ds-implementing-crd-4
PS20/21 - Financial holding companie
s: Further implementation
https://www.bankofengland.co.uk/prudential-regula
tion/publication/2021/june/financial-holding-compa
nie...


#### 2.7 Merge metadata


In [ ]:
failed_extractions = []

for i, (path, text, extraction_meta) in enumerate(results):
    try:
        comprehensive_metadata = consolidated_metadata_extractor.extract_comprehensive_metadata(path, text)
        
        if extraction_meta:
            comprehensive_metadata.update(extraction_meta)
        
        results[i] = (path, text, comprehensive_metadata)
        
    except Exception as e:
        # TRACK FAILURES but continue processing
        print(f"Metadata extraction failed for {path.name}: {e}")
        
        # Use basic metadata temporarily BUT flag it
        file_meta = parse_filename_metadata(path)
        merged_meta = merge_metadata(file_meta, extraction_meta)
        merged_meta['metadata_extraction_failed'] = True  # FLAG IT
        merged_meta['extraction_error'] = str(e)
        results[i] = (path, text, merged_meta)
        
        failed_extractions.append({
            'file': path.name,
            'error': str(e)
        })

# REPORT ALL FAILURES at the end
if failed_extractions:
    print(f"\nWARNING: {len(failed_extractions)} document(s) had metadata extraction failures:")
    for failure in failed_extractions:
        print(f"   • {failure['file']}: {failure['error']}")
    print(f"\n   These documents have incomplete metadata - review and fix before production!")

else:
    print(f"✅ Metadata merged for {len(results)} document(s)")

✅ Metadata merged for 41 document(s)


#### 2.8 Chunking Function

Creates chunks from extracted text.

- Inputs relied on  
  - results: list of (Path, text, extraction_meta) tuples created earlier.  
  - chunk_text_enhanced(text, ...): boundary-aware chunker defined in Section 1.7.5.  
  - extract_file_metadata(path) and merge_metadata(file_meta, extraction_meta).

  - Invokes chunk_text_enhanced on the cleaned text with chunk_id_prefix=path.stem so chunk IDs are stable and document-scoped.
  - Stores per-document record into all_chunks[str(path)] = { "text": text, "chunks": chunks, "metadata": merged_meta }.
  - Tracks and reports total files and total chunks using a tqdm progress bar.

- Outputs / side effects
  - all_chunks populated with one entry per document; each entry contains original text, an ordered list of chunk dicts (chunk_id, text, offset, size, boundary_type), and merged metadata suitable for sentiment/embedding/storage steps.
  
- Design notes / important behaviors
  - Ensures chunk IDs include filename stem for stable provenance and deduplication.
  - Uses merge_metadata so extraction-derived fields (e.g., layout, extraction_mode, content-derived year/quarter) are preserved and attached to stored chunks.
  - It reassigns all_chunks (overwrites existing mapping) — re-running will regenerate chunks.

In [ ]:
processor = EnhancedDocumentProcessor()
text, meta = processor.extract_text(
    Path("data/earnings_transcripts/barclays_1Q20_earnings_transcript.pdf"),
    use_layout=True,         # structured extraction for chunking
    preserve_layout=False,   # do not keep raw pages in meta/cache output
    use_cache=True
)
# meta now contains structure/tables/char positions useful for chunking

##### 2.8.1 Display sample chunk with metadata

Quick, read-only sanity check that prints a small sample of chunk records stored in all_chunks for human inspection.

- Behavior:
  - Iterates the first document in all_chunks (list(all_chunks.items())[:1]).
  - For up to two chunks from that document, prints a concise summary:
    - chunk index and chunk_id
    - character length of chunk text
    - boundary_type and offset mapping
    - the chunk text itself (trimmed by the print)
  - Uses basic dictionary access; does not modify state.

- Output: human-readable console prints for manual verification (no return value).

In [45]:
# show a small sample for quick inspection
for name, doc in list(all_chunks.items())[:1]:
    if doc.get('chunks'):
        print(f"\nSample chunks from '{name}' (first 2):\n---\n")
        for i, chunk_dict in enumerate(doc['chunks'][:2]):
            chunk_text = chunk_dict['text']  # Extract text from the dictionary
            print(f"Chunk {i+1} (id={chunk_dict['chunk_id']}, len={len(chunk_text)}):")
            print(f"Boundary: {chunk_dict['boundary_type']}, Offset: {chunk_dict['offset']}")
            print(f"Text:\n{chunk_text}\n---\n")


Sample chunks from '/Users/rodrigograndy/Desktop/coding_projects/posts_website_drafts/rag_boe/data/pra_rulebook/Market Risk_16-09-2025.pdf' (first 2):
---

Chunk 1 (id=Market Risk_16-09-2025_0, len=582):
Boundary: natural, Offset: {'start': 0, 'end': 583}
Text:
Prudential Regulation Authority Rulebook
Part
Market Risk
Printed on: 16/09/2025
Rulebook at: 16/09/2025
Related links
PS7/13 - Strengthening capital standar
ds: implementing CRD IV, feedback an
d final rules
https://www.bankofengland.co.uk/prudential-regula
tion/publication/2013/strengthening-capital-standar
ds-implementing-crd-4
PS20/21 - Financial holding companie
s: Further implementation
https://www.bankofengland.co.uk/prudential-regula
tion/publication/2021/june/financial-holding-compa
nies-further-implementation
Legislation.gov.uk
http://www.legislation.gov.uk/
Eur-Lex
---

Chunk 2 (id=Market Risk_16-09-2025_1, len=553):
Boundary: natural, Offset: {'start': 483, 'end': 1037}
Text:
holding-compa
nies-further-implementatio

In [46]:
# Financial sentiment analysis
def analyze_fin_sentiment(text: str, model: pipeline) -> Dict[str, float]:
    """Get financial sentiment with confidence scores"""
    try:
        text = text[:512]  # Truncate to model limit
        results = model(text)
        best_result = max(results[0], key=lambda x: x['score'])
        return {
            'sentiment': best_result['label'].lower(),
            'confidence': round(best_result['score'], 3)
        }
    except Exception:
        return {'sentiment': 'undetermined', 'confidence': 0.0}

def batch_sentiment_analysis(chunks: List[str], model: pipeline) -> List[Dict[str, float]]:
    """Analyze sentiment for multiple chunks"""
    results = []
    for chunk in tqdm(chunks, desc="Sentiment"):
        results.append(analyze_fin_sentiment(chunk, model))
    return results

print(f"Sentiment analysis functions ready")

Sentiment analysis functions ready


##### 2.9.1 Apply Sentiment Analysis to all chunks

Batch-runs the FinBERT sentiment wrapper across every text chunk in all_chunks and writes per-chunk sentiment metadata back into each chunk dict.

Preconditions
- all_chunks is populated (each document entry contains a 'chunks' list).
- finbert pipeline and analyze_fin_sentiment() are available.
- analyze_fin_sentiment truncates input to 512 chars, returns {'sentiment': str, 'confidence': float} and falls back to {'sentiment':'undetermined','confidence':0.0} on errors.

Key variables & flow
- all_chunk_texts: flattened list of every chunk['text'] across all documents — used for a single progress pass.
- chunk_file_mapping: parallel list recording which document each chunk came from (provenance).
- all_sentiments: list collecting analyze_fin_sentiment results in the same order as all_chunk_texts.
- sentiment assignment loop: iterates documents and their chunks in all_chunks and assigns chunk['sentiment'] = all_sentiments[sentiment_idx], preserving original chunk ordering.

Behavioral notes
- Uses tqdm to show progress during sentiment inference.
- Prints summary messages before and after processing.
- Downstream code expects chunk['sentiment'] to contain a label and numeric confidence for storage, filtering, and UI display.

In [47]:
# Apply sentiment analysis to all chunks
print("Applying sentiment analysis to all chunks...")

# Collect all chunks from all documents for batch processing
all_chunk_texts = []
chunk_file_mapping = []  # Keep track of which file each chunk belongs to

for name, doc in all_chunks.items():
    if doc.get('chunks'):
        for chunk in doc['chunks']:
            all_chunk_texts.append(chunk['text'])
            chunk_file_mapping.append(name)

# Process all chunks with a single progress bar
print(f"Processing {len(all_chunk_texts)} chunks across {len(all_chunks)} files...")
all_sentiments = []
for chunk_text in tqdm(all_chunk_texts, desc="Sentiment analysis"):
    sentiment = analyze_fin_sentiment(chunk_text, finbert)
    all_sentiments.append(sentiment)

# Assign sentiment results back to chunks
sentiment_idx = 0
for name, doc in all_chunks.items():
    if doc.get('chunks'):
        for chunk in doc['chunks']:
            chunk['sentiment'] = all_sentiments[sentiment_idx]
            sentiment_idx += 1

print(f"Sentiment analysis complete for all {len(all_chunk_texts)} chunks")

Applying sentiment analysis to all chunks...
Processing 2991 chunks across 41 files...


Sentiment analysis:   0%|          | 0/2991 [00:00<?, ?it/s]

Sentiment analysis complete for all 2991 chunks


##### 2.9.2 Display sample chunk with sentiment

Quick sanity check to confirm sentiment labels and confidences were attached to chunks and that chunk metadata and document metadata (including layout summaries) look correct before downstream storage or analysis.

In [ ]:
# Print a sample chunk with sentiment for verification
for name, doc in list(all_chunks.items())[:1]:
    if doc.get('chunks'):
        print(f"\nSample chunk with sentiment from '{Path(name).name}':\n" + "="*60 + "\n")
        sample_chunk = doc['chunks'][0]
        print(f"Chunk ID: {sample_chunk['chunk_id']}")
        print(f"Text Length: {len(sample_chunk['text'])} characters")

        # Check if sentiment data exists
        sentiment_data = sample_chunk.get('sentiment', {})
        if sentiment_data and 'sentiment' in sentiment_data:
            sentiment_label = sentiment_data['sentiment'].title()
            confidence = sentiment_data.get('confidence', 0.0)
            print(f"Sentiment: {sentiment_label} (confidence: {confidence:.1%})")
        else:
            print(f"Sentiment: Not available (run sentiment analysis first)")

        print(f"Boundary Type: {sample_chunk.get('boundary_type', 'unknown')}")
        print(f"Offset: {sample_chunk.get('offset', {})}")
        print(f"\nText Sample:\n{sample_chunk['text'][:300]}{'...' if len(sample_chunk['text']) > 300 else ''}\n")
        print(f"Document Metadata:")
        if doc['metadata']:
            for key, value in doc['metadata'].items():
                if key == 'layout':
                    print(f"   {key}: {len(value)} layout spans" if isinstance(value, list) else f"   {key}: {value}")
                else:
                    print(f"   {key}: {value}")
        else:
            print("   No metadata available")
        print("="*60 + "\n")


Sample chunk with sentiment from 'Market Risk_16-09-2025.pdf':

Chunk ID: Market Risk_16-09-2025_0
Text Length: 582 characters
Sentiment: Neutral (confidence: 100.0%)
Boundary Type: natural
Offset: {'start': 0, 'end': 583}

Text Sample:
Prudential Regulation Authority Rulebook
Part
Market Risk
Printed on: 16/09/2025
Rulebook at: 16/09/2025
Related links
PS7/13 - Strengthening capital standar
ds: implementing CRD IV, feedback an
d final rules
https://www.bankofengland.co.uk/prudential-regula
tion/publication/2013/strengthening-capit...

Document Metadata:
   filename: Market Risk_16-09-2025.pdf
   company: market risk
   quarter: 16-09-2025
   doc_type: financial_document
   file_extension: .pdf
   filename_parts: ['market risk', '16-09-2025']
   extraction_source: content
   company_filename: market risk
   year_filename: 2025
   years_filename: ['2025']
   doc_type_filename: financial_document
   content_dates: {'full_date_content': ['16/09/2025', '16/09/2025', '17/09/2021', '17/09/2

##### 2.10.1 Generate Embeddings for all chunks

Orchestrates flattening of all chunk texts, generates embeddings with the previously-loaded SentenceTransformer, and writes embeddings back into each chunk dict.


- all_chunks: mapping of document path → { "text", "chunks": [...], "metadata": ... } must be populated.
- embedding_model: global SentenceTransformer instance loaded in Section 1.5 must be available.
- Embedding generation
  - Calls generate_embeddings(all_chunk_texts, embedding_model) in one shot.
  - generate_embeddings performs model.encode(..., show_progress_bar=True, normalize_embeddings=True) and returns a numpy.ndarray (empty array on no input).
  - Note: this is a single-call approach (no internal batching), so memory use scales with corpus size (This something to improve in the future).

- Assignment back to chunks
  - Iterates over all_chunks in original order and assigns chunk['embedding'] = all_embeddings[embedding_idx] while guarding index bounds.
  - Tracks total_chunks_embedded and increments embedding_idx for each assigned embedding.
  - Mutates all_chunks in-place so downstream steps can use chunk['embedding'].

- Important properties / caveats
  - Embeddings are normalized by the encoder (normalize_embeddings=True).
  - If all_chunk_texts is empty, all_embeddings stays empty and no assignments occur.
  - Single-call encoding may cause high memory use for large corpora; downstream code elsewhere provides batched alternatives.
  - The parallel chunk_file_mapping is collected but not used in the assignment loop (provenance is preserved via chunk IDs and metadata).

Effect
- After the cell runs, each chunk dict in all_chunks that had a corresponding embedding receives an 'embedding' field (numpy array), enabling storage, similarity search and downstream ingestion.

In [54]:
# Apply embedding generation to all chunks
print("Applying embedding generation to all chunks...")

# Collect all chunk texts from all documents for batch processing
all_chunk_texts = []
chunk_file_mapping = []  # Keep track of which file each chunk belongs to

for name, doc in all_chunks.items():
    if doc.get('chunks'):
        for chunk in doc['chunks']:
            all_chunk_texts.append(chunk['text'])
            chunk_file_mapping.append(name)

# Process all chunks with a single progress bar
print(f"Processing {len(all_chunk_texts)} chunks across {len(all_chunks)} files...")
all_embeddings = []


# Generate embeddings for all chunks at once
if all_chunk_texts:
    all_embeddings = generate_embeddings(all_chunk_texts, embedding_model) #  From section 1.5 

# Assign embedding results back to chunks
embedding_idx = 0
total_chunks_embedded = 0

for name, doc in all_chunks.items():
    if doc.get('chunks'):
        for chunk in doc['chunks']:
            if embedding_idx < len(all_embeddings):
                chunk['embedding'] = all_embeddings[embedding_idx]
                embedding_idx += 1
                total_chunks_embedded += 1

print(f"Embedding generation complete for all {total_chunks_embedded} chunks")

Applying embedding generation to all chunks...
Processing 2991 chunks across 41 files...


Batches:   0%|          | 0/94 [00:00<?, ?it/s]

Embedding generation complete for all 2991 chunks


##### 2.10.2 Display sample embedding

Sanity check to verify that embeddings were attached to chunks and that embedding/sentiment fields have expected types and shapes.

In [56]:
# Print a sample chunk with embeddings for verification
for name, doc in list(all_chunks.items())[:1]:
    if doc.get('chunks'):
        print(f"\nSample chunk with embeddings from '{Path(name).name}':\n---\n")
        sample_chunk = doc['chunks'][0]
        print(f"Chunk ID: {sample_chunk['chunk_id']}")
        print(f"Text Length: {len(sample_chunk['text'])} characters")
        print(f"Text Sample: {sample_chunk['text'][:200]}...")

        embedding = sample_chunk.get('embedding')
        if embedding is not None:
            if isinstance(embedding, np.ndarray):
                print(f"Embedding: numpy array of shape {embedding.shape}")
                print(f"   First 5 values: {embedding[:5]}")
            elif isinstance(embedding, list):
                print(f"Embedding: list of length {len(embedding)}")
                print(f"   First 5 values: {embedding[:5]}")
            else:
                print(f"Embedding: {type(embedding)}")
        else:
            print("No embedding found! (run embedding generation first)")

        # Check sentiment data
        sentiment = sample_chunk.get('sentiment', {})
        if sentiment and 'sentiment' in sentiment:
            print(f"Sentiment: {sentiment['sentiment'].title()} (confidence: {sentiment.get('confidence', 0.0):.1%})")
        else:
            print("Sentiment: Not available (run sentiment analysis first)")

        print("---\n")


Sample chunk with embeddings from 'Market Risk_16-09-2025.pdf':
---

Chunk ID: Market Risk_16-09-2025_0
Text Length: 582 characters
Text Sample: Prudential Regulation Authority Rulebook
Part
Market Risk
Printed on: 16/09/2025
Rulebook at: 16/09/2025
Related links
PS7/13 - Strengthening capital standar
ds: implementing CRD IV, feedback an
d fin...
Embedding: numpy array of shape (1024,)
   First 5 values: [-0.00994054 -0.00976403 -0.0113218   0.00842191  0.01860651]
Sentiment: Neutral (confidence: 100.0%)
---



## 3. Database Storage

This section includes functions for creating and managing the vector database.

#### 3.1 Collection Management with Delete Functionality

Provide an interactive UI (ipywidgets) to list/create/select/delete ChromaDB collections and to store processed document chunks into a selected collection. For new collection add new collection name, create and save the collection. For existing collection, select from dropdown menu, and stored processed document chunks into the selected collection.

- create_enhanced_collection_manager_ui(client, all_chunks)
  - Builds and returns the full widget layout; requires a chromadb client and the notebook-level all_chunks mapping.

Key UI components
- existing_dropdown: collection chooser populated from client via list_existing_collections.
- new_name_input: text input for creating a new collection.
- Buttons: Select, Create New, Delete, and Store Documents.
- storage_progress: FloatProgress used during ingestion.
- output: Output widget used for all console messages.

- refresh_collections(): fetches current collection names from the client.
- update_dropdown(): updates dropdown state and enables/disables buttons (delete, store) based on availability of collections and all_chunks.
- on_select(btn): selects the dropdown value, assigns the global collection object (client.get_collection), prints basic collection info (via get_collection_info) and toggles store button.
- on_create(btn): uses create_new_collection(client, name) to create or fetch a collection, refreshes UI, and enables storage when all_chunks is present.
- on_delete(btn): shows a confirmation UI (Confirm Delete / Cancel). on_confirm calls client.delete_collection(name), clears global collection if it was selected, refreshes the dropdown and prints success/error. on_cancel simply dismisses the confirmation.
- on_store_documents(btn): guarded ingestion routine:
  - Validates a collection is selected and that all_chunks contains processed documents.
  - Shows storage_progress, iterates all_chunks, and for each document calls store_enriched_chunks(collection, chunks, metadata).
  - Tracks total_stored and failed_files, updates progress bar, prints a concise end summary (total stored and brief failure list), hides the progress bar, and attempts to print final collection.count().


In [113]:
# Enhanced Collection Management UI with Document Storage Integration
def create_enhanced_collection_manager_ui(client, all_chunks):
    """Complete collection management UI with create, select, delete, and storage operations"""
    
    def refresh_collections():
        """Helper to get current collection list"""
        return list_existing_collections(client)
    
    # UI Components
    existing_dropdown = widgets.Dropdown(
        options=['No collections found'],
        description='Collection:',
        disabled=True,
        layout=widgets.Layout(width='400px')
    )
    
    new_name_input = widgets.Text(
        description='New Name:',
        placeholder='Enter collection name',
        layout=widgets.Layout(width='400px')
    )
    
    # Buttons
    select_btn = widgets.Button(
        description='Select',
        button_style='primary',
        layout=widgets.Layout(width='120px')
    )
    
    create_btn = widgets.Button(
        description='Create New',
        button_style='success',
        layout=widgets.Layout(width='120px')
    )
    
    delete_btn = widgets.Button(
        description='Delete',
        button_style='danger',
        layout=widgets.Layout(width='120px'),
        disabled=True
    )
    
    # NEW: Store Documents Button
    store_btn = widgets.Button(
        description='📥 Store Documents',
        button_style='info',
        layout=widgets.Layout(width='150px'),
        disabled=True,
        tooltip='Store processed documents in selected collection'
    )
    
    # NEW: Progress indicators for storage
    storage_progress = widgets.FloatProgress(
        value=0,
        min=0,
        max=100,
        description='Storing:',
        bar_style='info',
        layout=widgets.Layout(width='400px', visibility='hidden')
    )
    
    output = widgets.Output()
    
    def update_dropdown():
        """Refresh dropdown with current collections"""
        collections = refresh_collections()
        if collections:
            existing_dropdown.options = collections
            existing_dropdown.disabled = False
            existing_dropdown.value = collections[0]
            delete_btn.disabled = False
            # Enable store button if collection is selected and documents are available
            store_btn.disabled = not (all_chunks and len(all_chunks) > 0)
        else:
            existing_dropdown.options = ['No collections found']
            existing_dropdown.disabled = True
            delete_btn.disabled = True
            store_btn.disabled = True
    
    def on_select(btn):
        with output:
            clear_output()
            if existing_dropdown.disabled:
                print("No collections available")
                return
            
            name = existing_dropdown.value
            try:
                global collection
                collection = client.get_collection(name)
                info = get_collection_info(collection)
                print(f"Selected: '{name}'")
                print(f"   Documents: {info['document_count']:,}")
                
                # Enable store button if documents are available
                store_btn.disabled = not (all_chunks and len(all_chunks) > 0)
                
            except Exception as e:
                print(f"Error selecting '{name}': {e}")
    
    def on_create(btn):
        with output:
            clear_output()
            name = new_name_input.value.strip()
            
            if not name:
                print("Enter collection name")
                return
            
            try:
                global collection
                collection = create_new_collection(client, name)
                if collection:
                    print(f"Created: '{name}'")
                    new_name_input.value = ''
                    update_dropdown()
                    
                    # Enable store button if documents are available
                    store_btn.disabled = not (all_chunks and len(all_chunks) > 0)
                    
                    # Show helpful message about storing documents
                    if all_chunks and len(all_chunks) > 0:
                        print(f"\nReady to store documents!")
                        print(f"   Click 'Store Documents' to save {len(all_chunks)} document(s)")
                    else:
                        print(f"\nProcess documents first (Section 2) to enable storage")
                else:
                    print(f"Failed to create '{name}'")
            except Exception as e:
                print(f"Error: {e}")
    
    def on_delete(btn):
        with output:
            clear_output()
            if existing_dropdown.disabled:
                print("No collections to delete")
                return
            
            name = existing_dropdown.value
            
            try:
                temp_collection = client.get_collection(name)
                doc_count = temp_collection.count()
                info_msg = f"Delete collection '{name}'?\n   This will permanently remove {doc_count:,} document chunks."
            except:
                info_msg = f"Delete collection '{name}'?\n   This will permanently remove all documents."
            
            print(info_msg)
            
            confirm_btn = widgets.Button(
                description='Confirm Delete',
                button_style='danger',
                layout=widgets.Layout(width='140px')
            )
            
            cancel_btn = widgets.Button(
                description='Cancel',
                button_style='',
                layout=widgets.Layout(width='100px')
            )
            
            def on_confirm(b):
                with output:
                    clear_output()
                    try:
                        client.delete_collection(name)
                        
                        global collection
                        if collection and collection.name == name:
                            collection = None
                        
                        print(f"Deleted collection: '{name}'")
                        update_dropdown()
                    except Exception as e:
                        print(f"Error deleting '{name}': {e}")
                        update_dropdown()
            
            def on_cancel(b):
                with output:
                    clear_output()
                    print("Deletion cancelled")
            
            confirm_btn.on_click(on_confirm)
            cancel_btn.on_click(on_cancel)
            
            display(widgets.HBox([confirm_btn, cancel_btn], layout=widgets.Layout(margin='10px 0')))
    
    def on_store_documents(btn):
        """Store processed documents in the selected collection"""
        with output:
            clear_output()
            
            # Check if collection is selected
            if 'collection' not in globals() or collection is None:
                print("No collection selected")
                print("   → Select a collection first")
                return
            
            # Check if documents are available
            if not all_chunks or len(all_chunks) == 0:
                print("No processed documents found")
                print("   → Run Section 2 to process documents first")
                return
            
            # Show progress bar
            storage_progress.layout.visibility = 'visible'
            storage_progress.value = 0
            
            total_stored = 0
            failed_files = []
            total_files = len(all_chunks)
            
            for idx, (filepath, doc_data) in enumerate(all_chunks.items(), 1):
                chunks = doc_data.get('chunks', [])
                metadata = doc_data.get('metadata', {})
                filename = Path(filepath).name
                
                # Update progress
                storage_progress.value = (idx / total_files) * 100
                
                if not chunks:
                    print(f"  {filename}: No chunks (skipped)")
                    continue
                
                try:
                    num_stored = store_enriched_chunks(collection, chunks, metadata)
                    total_stored += num_stored
                    
                except Exception as e:
                    error_msg = str(e)[:100]
                    print(f"  {filename}: {error_msg}")
                    failed_files.append((filename, error_msg))
            
            print("=" * 60)
            print(f"Storage complete: {total_stored:,} chunks stored")
            
            if failed_files:
                print(f"\n{len(failed_files)} file(s) failed:")
                for filename, error in failed_files[:5]:
                    print(f"   • {filename}: {error}")
            
            # Hide progress bar
            storage_progress.layout.visibility = 'hidden'
            storage_progress.value = 0
            
            # Show collection info
            try:
                final_count = collection.count()
                print(f"\nCollection '{collection.name}' now contains: {final_count:,} chunks")
            except:
                pass
    
    # Bind events
    select_btn.on_click(on_select)
    create_btn.on_click(on_create)
    delete_btn.on_click(on_delete)
    store_btn.on_click(on_store_documents)
    
    # Initial population
    update_dropdown()
    
    # Build UI
    ui = widgets.VBox([
        widgets.HTML("<h4>📂 Collection Management</h4>"),
        widgets.HTML("<hr style='margin: 10px 0;'>"),
        
        # Select existing
        widgets.HTML("<b>Select Existing Collection:</b>"),
        existing_dropdown,
        widgets.HBox([select_btn, delete_btn], layout=widgets.Layout(margin='5px 0')),
        
        widgets.HTML("<div style='height: 20px;'></div>"),
        
        # Create new
        widgets.HTML("<b>Create New Collection:</b>"),
        new_name_input,
        widgets.HBox([create_btn], layout=widgets.Layout(margin='5px 0')),
        
        widgets.HTML("<div style='height: 20px;'></div>"),
        
        # Store documents section
        widgets.HTML("<b>Store Documents in Collection:</b>"),
        widgets.HBox([store_btn], layout=widgets.Layout(margin='5px 0')),
        storage_progress,
        
        widgets.HTML("<div style='height: 15px;'></div>"),
        output
    ], layout=widgets.Layout(
        padding='15px',
        border='1px solid #e2e8f0',
        border_radius='8px',
        background_color='#f8fafc',
        width='550px'
    ))
    
    return ui

print("Enhanced Collection Management UI with Document Storage ready")

Enhanced Collection Management UI with Document Storage ready


#### 3.2 Choose collection to Store Documents in ChromaDB

Display the collection-management UI built in 3.1 so the user can select/create/delete collections and store processed documents.

In [ ]:
# Enhanced Collection Manager with Storage
if 'client' in globals():
    # Check if documents have been processed
    if 'all_chunks' not in globals():
        all_chunks = {}
        print("No processed documents found")
        print("Documents from Section 2 will be available for storage once processed")
    
    enhanced_collection_ui = create_enhanced_collection_manager_ui(client, all_chunks)
    display(enhanced_collection_ui)
    
    if all_chunks and len(all_chunks) > 0:
        print(f"\n{len(all_chunks)} document(s) ready for storage")
    else:
        print("\nProcess documents in Section 2 to enable storage")
else:
    print("ChromaDB client not available. Run Section 1.6 first.")


41 document(s) ready for storage


#### 3.3 Verify collection readiness

- verify_collection_ready(): confirm a selected ChromaDB collection is set and accessible.  
- display_all_collections(): enumerate all collections in the ChromaDB client and show per-collection diagnostics.
- Cell-level orchestration after definitions  
  - Calls display_all_collections() to show database state.  
  - Then checks if a global collection exists: if so, prints "Selected Collection Status:" and runs verify_collection_ready(); otherwise prints guidance to run Section 3.2.

In [ ]:
# Verify Collection Readiness
def verify_collection_ready() -> bool:
    """Verify a collection is selected and accessible"""
    global collection
    
    if collection is None:
        print("No collection selected")
        print("Run Section 3.2 to select/create a collection")
        return False
    
    try:
        count = collection.count()
        print(f"Collection ready: '{collection.name}'")
        print(f"   Documents: {count:,} chunks")
        return True
    except Exception as e:
        print(f"Collection error: {e}")
        print("Re-run Section 3.2 to select a valid collection")
        return False

def display_all_collections() -> None:
    """Display all available collections with document counts"""
    print("\nAvailable Collections:")
    print("=" * 60)
    
    try:
        collections = client.list_collections()
        
        if not collections:
            print("   No collections found")
            print("   Process documents (Section 2) to create collections")
            return
        
        for col in collections:
            try:
                col_obj = client.get_collection(col.name)
                count = col_obj.count()
                print(f"   • {col.name}: {count:,} chunks")
            except Exception as e:
                print(f"   • {col.name}: Error accessing ({str(e)[:50]})")
        
        print("=" * 60)
        
    except Exception as e:
        print(f"Failed to list collections: {e}")

# Display all collections
display_all_collections()

# Verify selected collection if one exists
if 'collection' in globals() and collection is not None:
    print("\nSelected Collection Status:")
    verify_collection_ready()
else:
    print("\nNo collection selected yet")
    print("Run Section 3.2 to select a collection")


Available Collections:
   • transcrips_barclasys: 5,591 chunks
   • pra_rules: 2,991 chunks

Selected Collection Status:
Collection ready: 'pra_rules'
   Documents: 2,991 chunks


## 4. RAG System Implementation

#### 4.1.1 Enhanced Collections Manager with BM25 Indexing

Management layer around ChromaDB collections that adds an on-disk, cached BM25 index per collection and unified retrieval helpers (BM25 and semantic). It centralizes collection access, index creation/loading, tokenization logic and result formatting.

Key components and behavior
- Initialization
  - Accepts a chromadb client and store directory.
  - Prepares a bm25 storage directory and in-memory caches:
    - self.bm25_indices: in-memory BM25 objects keyed by collection name
    - self.collections_cache: cached chromadb.Collection objects

- get_collection(name)
  - Returns cached collection if present; otherwise attempts client.get_collection and caches the result.
  - Defensive: returns None on failure.

- get_available_collections()
  - Returns collection names from client.list_collections(), safe-fails to [].

- _get_index_path(collection_name)
  - Deterministic path for storing the pickled BM25 index per collection.

- _tokenize_documents(documents)
  - Normalizes and tokenizes texts for BM25 (lowercase, removes non-alphanumeric/dots, filters short tokens).
  - Encapsulates tokenization policy so BM25 behavior is consistent.

- build_bm25_index(collection_name)
  - Loads documents from the collection (via collection.get include=['documents']), tokenizes them, constructs BM25Okapi, caches it and persists it to disk with pickle.
  - Returns success boolean and prints/logs failures.

- load_bm25_index(collection_name)
  - Loads a previously saved BM25 object from disk into the in-memory cache; returns boolean success.

- search_bm25(query, collection_name, top_k)
  - Ensures BM25 is available (load or build on demand).
  - Tokenizes query with the same tokenizer, gets BM25 scores, selects top indices and fetches corresponding documents/metadatas from the collection.
  - Returns a list of result dicts containing: text, metadata, bm25_score, collection and search_method.
  - Skips items with non-positive score; defensive around missing collection/data.

- semantic_search(query, collection_name, top_k)
  - Uses the global embedding model if available to encode the query, otherwise queries ChromaDB with query_texts.
  - Calls collection.query(include=['documents','metadatas','distances']) and converts distances to a semantic score (1 - distance).
  - Returns formatted results with semantic_score, distance, metadata and collection.

Design notes (concise)
- Persistent BM25 indices: pickled to disk for reuse and loaded into memory on demand.
- Consistent tokenization: ensures BM25 and query tokens share the same pre-processing rules.
- Caching: reduces repeated ChromaDB calls and re-build overhead.

In [19]:
# Enhanced Collections Manager
class EnhancedCollectionsManager:
    """Manages ChromaDB collections with BM25 indexing"""
    
    def __init__(self, client: chromadb.Client, store_dir: Path):
        self.client = client
        self.store_dir = Path(store_dir)
        self.bm25_dir = self.store_dir / "bm25_indices"
        self.bm25_dir.mkdir(parents=True, exist_ok=True)
        
        # Caches
        self.bm25_indices: Dict[str, BM25Okapi] = {}
        self.collections_cache: Dict[str, chromadb.Collection] = {}
    
    def get_collection(self, name: str) -> Optional[chromadb.Collection]:
        """Get collection from cache or client"""
        if not name:
            return None
        if name not in self.collections_cache:
            try:
                self.collections_cache[name] = self.client.get_collection(name)
            except:
                return None
        return self.collections_cache[name]
    
    def get_available_collections(self) -> List[str]:
        """List all collection names"""
        try:
            return [col.name for col in self.client.list_collections()]
        except:
            return []
    
    def _get_index_path(self, collection_name: str) -> Path:
        """Get BM25 index file path"""
        return self.bm25_dir / f"{collection_name}_bm25.pkl"
    
    def _tokenize_documents(self, documents: List[str]) -> List[List[str]]:
        """Tokenize documents for BM25"""
        tokenized = []
        for doc in documents:
            # Keep alphanumeric and dots, remove other punctuation
            clean = re.sub(r"[^0-9a-zA-Z\.\s]", " ", doc.lower())
            tokens = [t for t in clean.split() if len(t) > 1]
            tokenized.append(tokens)
        return tokenized
    
    def build_bm25_index(self, collection_name: str) -> bool:
        """Build and save BM25 index for collection"""
        collection = self.get_collection(collection_name)
        if not collection:
            return False
        
        try:
            # Get documents
            data = collection.get(include=['documents'])
            documents = data.get('documents', [])
            if not documents:
                return False
            
            # Tokenize and build index
            tokenized = self._tokenize_documents(documents)
            bm25 = BM25Okapi(tokenized, k1=1.2, b=0.75)
            
            # Cache and save
            self.bm25_indices[collection_name] = bm25
            with open(self._get_index_path(collection_name), 'wb') as f:
                pickle.dump(bm25, f)
            
            return True
        except Exception as e:
            print(f"Failed to build BM25 index for {collection_name}: {e}")
            return False
    
    def load_bm25_index(self, collection_name: str) -> bool:
        """Load pre-built BM25 index from disk"""
        if collection_name in self.bm25_indices:
            return True
        
        index_path = self._get_index_path(collection_name)
        if not index_path.exists():
            return False
        
        try:
            with open(index_path, 'rb') as f:
                self.bm25_indices[collection_name] = pickle.load(f)
            return True
        except:
            return False
    
    def search_bm25(self, query: str, collection_name: str, top_k: int = 10) -> List[Dict]:
        """Search using BM25 index"""
        # Ensure index is loaded
        if collection_name not in self.bm25_indices:
            if not self.load_bm25_index(collection_name):
                if not self.build_bm25_index(collection_name):
                    return []
        
        bm25 = self.bm25_indices[collection_name]
        
        # Tokenize query
        query_tokens = [t for t in re.sub(r'[^\w\s\.]', ' ', query.lower()).split() if len(t) > 1]
        if not query_tokens:
            return []
        
        # Get scores
        scores = bm25.get_scores(query_tokens)
        top_indices = sorted(range(len(scores)), key=lambda i: scores[i], reverse=True)[:top_k]
        
        # Get documents
        collection = self.get_collection(collection_name)
        if not collection:
            return []
        
        data = collection.get(include=['documents', 'metadatas'])
        documents = data.get('documents', [])
        metadatas = data.get('metadatas', [])
        
        # Build results
        results = []
        for idx in top_indices:
            if idx < len(documents) and scores[idx] > 0:
                results.append({
                    'text': documents[idx],
                    'metadata': metadatas[idx] if idx < len(metadatas) else {},
                    'bm25_score': float(scores[idx]),
                    'collection': collection_name,
                    'search_method': 'bm25'
                })
        
        return results
    
    def semantic_search(self, query: str, collection_name: str, top_k: int = 10) -> List[Dict]:
        """Search using semantic similarity"""
        collection = self.get_collection(collection_name)
        if not collection:
            return []
        
        try:
            # Use global embedding model if available
            if 'embedding_model' in globals():
                query_emb = embedding_model.encode([query])
                results = collection.query(
                    query_embeddings=query_emb.tolist(),
                    n_results=top_k,
                    include=['documents', 'metadatas', 'distances']
                )
            else:
                results = collection.query(
                    query_texts=[query],
                    n_results=top_k,
                    include=['documents', 'metadatas', 'distances']
                )
            
            # Format results
            formatted = []
            docs = results.get('documents', [[]])[0]
            metas = results.get('metadatas', [[]])[0]
            dists = results.get('distances', [[]])[0]
            
            for i, doc in enumerate(docs):
                dist = dists[i] if i < len(dists) else 1.0
                formatted.append({
                    'text': doc,
                    'metadata': metas[i] if i < len(metas) else {},
                    'semantic_score': 1.0 - dist,
                    'distance': dist,
                    'collection': collection_name,
                    'search_method': 'semantic'
                })
            
            return formatted
        except Exception as e:
            print(f"Semantic search error: {e}")
            return []

print("✅ EnhancedCollectionsManager ready")

✅ EnhancedCollectionsManager ready


#### 4.1.2 Initialize BM25 Index

Instantiates the EnhancedCollectionsManager and verify BM25 index storage readiness for the running environment.

- Guard: checks that a running Chroma client ("client") and STORE_DIR are available in globals(); otherwise prints a short error and does not proceed.
- Initialization: creates collections_manager = EnhancedCollectionsManager(client, STORE_DIR). That constructor creates the bm25_indices directory (bm25_dir), initializes in-memory caches (bm25_indices, collections_cache) and prepares tokenization/indexing helpers defined earlier.
- Status output: prints confirmation plus the path to the bm25 directory and the list of currently available collections (via collections_manager.get_available_collections()).
- Side effects: ensures bm25 storage directory exists on disk and exposes the manager instance for downstream operations (build/load/search BM25 indices).

Notes
- This cell does not build indexes itself, it only prepares the manager and directory; index building/loading is handled by subsequent prebuilder code (BM25IndexPreBuilder) or manager methods.

In [20]:
# Initialize Collections Manager

if 'client' in globals() and 'STORE_DIR' in globals():
    collections_manager = EnhancedCollectionsManager(client, STORE_DIR)
    print("✅ Collections Manager initialized")
    print(f"   BM25 directory: {collections_manager.bm25_dir}")
    print(f"   Collections: {collections_manager.get_available_collections()}")
else:
    print("❌ Missing client or STORE_DIR - run Section 1 first")

✅ Collections Manager initialized
   BM25 directory: /Users/rodrigograndy/Desktop/coding_projects/posts_website_drafts/rag_boe/chroma_store/bm25_indices
   Collections: ['transcrips_barclasys', 'pra_rules']


#### 4.2.1 BM25 Index Building Function

Pre-build or load BM25 indices for all ChromaDB collections via the EnhancedCollectionsManager.

Key class: BM25IndexPreBuilder

- check_index_status(collection_name) -> Dict
  - Verifies the collection exists (via manager.get_collection).
  - Computes the expected index file path (manager._get_index_path) and collection.count().
  - Returns a status dict containing: collection name, document_count, index_path, and status ('missing' or 'exists').
  - If the index file exists it also reports file size (MB) and last_modified timestamp.
  - Used to decide whether to load, skip, or rebuild an index.

- prebuild_all_indices(force_rebuild: bool = False) -> Dict
  - Orchestrates index preparation for every collection returned by manager.get_available_collections().
  - Short-circuits if no collections found (returns {'status': 'no_collections', 'results': {}}).
  - For each collection:
    - Calls check_index_status to inspect current state.
    - If an index exists and force_rebuild is False: attempts to load it via manager.load_bm25_index and records action='loaded_existing'.
    - Otherwise: builds index via manager.build_bm25_index and records action='built_new'.
  - Uses tqdm for progress, tracks timing and counts successful builds/loads.
  - Returns a summary dict with overall status, total_collections, successful count, total_time and per-collection results (boolean success + action).

In [21]:
# BM25 Index Pre-Builder
class BM25IndexPreBuilder:
    """Pre-build BM25 indices for all collections"""
    
    def __init__(self, manager: EnhancedCollectionsManager):
        self.manager = manager
    
    def check_index_status(self, collection_name: str) -> Dict[str, Any]:
        """Check if index exists and get stats"""
        collection = self.manager.get_collection(collection_name)
        if not collection:
            return {'status': 'collection_not_found'}
        
        index_path = self.manager._get_index_path(collection_name)
        document_count = collection.count()
        
        result = {
            'collection': collection_name,
            'document_count': document_count,
            'index_path': str(index_path),
            'status': 'missing'
        }
        
        if index_path.exists():
            stat = index_path.stat()
            result.update({
                'status': 'exists',
                'index_size_mb': stat.st_size / (1024 * 1024),
                'last_modified': time.strftime('%Y-%m-%d %H:%M:%S', 
                                              time.localtime(stat.st_mtime))
            })
        
        return result
    
    def prebuild_all_indices(self, force_rebuild: bool = False) -> Dict[str, Any]:
        """Build indices for all collections"""
        collections = self.manager.get_available_collections()
        
        if not collections:
            return {'status': 'no_collections', 'results': {}}
        
        print(f"Building BM25 indices for {len(collections)} collections...")
        print("=" * 60)
        
        results = {}
        start_time = time.time()
        
        for collection_name in tqdm(collections, desc="Building indices"):
            # Check if rebuild needed
            status = self.check_index_status(collection_name)
            
            if status['status'] == 'exists' and not force_rebuild:
                results[collection_name] = {
                    'action': 'loaded_existing',
                    'success': self.manager.load_bm25_index(collection_name)
                }
            else:
                results[collection_name] = {
                    'action': 'built_new',
                    'success': self.manager.build_bm25_index(collection_name)
                }
        
        total_time = time.time() - start_time
        successful = sum(1 for r in results.values() if r['success'])
        
        print("\n" + "=" * 60)
        print(f"✅ Complete: {successful}/{len(collections)} indices ready")
        print(f"⏱️  Total time: {total_time:.1f}s")
        print("=" * 60)
        
        return {
            'status': 'complete',
            'total_collections': len(collections),
            'successful': successful,
            'total_time': total_time,
            'results': results
        }

print("✅ BM25IndexPreBuilder ready")

✅ BM25IndexPreBuilder ready


#### 4.2.2 Initialize BM25 Index

Prepares the BM25 pre-builder utility so index building/loading can be triggered later.

- Checks that collections_manager exists in globals() (requires Chroma client + STORE_DIR initialized).
- Instantiates bm25_prebuilder = BM25IndexPreBuilder(collections_manager).

- Side effects / observable output
  - Ensures the pre-builder is available as a global object for downstream use.
  - Prints confirmation, the bm25 storage directory path (collections_manager.bm25_dir), and short usage hints:
    - bm25_prebuilder.prebuild_all_indices()
    - bm25_prebuilder.prebuild_all_indices(force_rebuild=True)

In [22]:
# Initialize Pre-Builder

if 'collections_manager' in globals():
    bm25_prebuilder = BM25IndexPreBuilder(collections_manager)
    print("✅ BM25 Pre-Builder initialized")
    print(f"   Storage: {collections_manager.bm25_dir}")
    print("\nUsage:")
    print("  • bm25_prebuilder.prebuild_all_indices()")
    print("  • bm25_prebuilder.prebuild_all_indices(force_rebuild=True)")
else:
    print("❌ Collections manager not available")

✅ BM25 Pre-Builder initialized
   Storage: /Users/rodrigograndy/Desktop/coding_projects/posts_website_drafts/rag_boe/chroma_store/bm25_indices

Usage:
  • bm25_prebuilder.prebuild_all_indices()
  • bm25_prebuilder.prebuild_all_indices(force_rebuild=True)


#### 4.2.3 BM25 Index Version Check

Provides a lightweight health/version/status checker for BM25 indices managed by BM25IndexPreBuilder.

Key class: BM25IndexChecker

- check_all_indices() -> Dict[str, Any]
  - Iterates all collection names from prebuilder.manager.get_available_collections().
  - Calls prebuilder.check_index_status(collection_name) for each collection and aggregates results.
  - Produces and returns a summary dict with:
    - total (number of collections),
    - ready (count of indices with status 'exists'),
    - missing (count of indices not present),
    - details (per-collection status dicts returned by check_index_status).
  - Does not modify on-disk indices; purely diagnostic.

Initialization behavior
- If bm25_prebuilder is present in globals(), the notebook creates index_checker = BM25IndexChecker(bm25_prebuilder) and prints readiness.
- If prebuilder is missing, it prints an error and does not create the checker.

In [23]:
# Section 4.2.3: Index Status Checker
class BM25IndexChecker:
    """Check BM25 index health and version"""
    
    def __init__(self, prebuilder: BM25IndexPreBuilder):
        self.prebuilder = prebuilder
    
    def check_all_indices(self) -> Dict[str, Any]:
        """Check status of all indices"""
        collections = self.prebuilder.manager.get_available_collections()
        
        summary = {
            'total': len(collections),
            'ready': 0,
            'missing': 0,
            'details': {}
        }
        
        for collection_name in collections:
            status = self.prebuilder.check_index_status(collection_name)
            summary['details'][collection_name] = status
            
            if status['status'] == 'exists':
                summary['ready'] += 1
            else:
                summary['missing'] += 1
        
        return summary
    
    def print_status(self):
        """Print formatted status report"""
        summary = self.check_all_indices()
        
        print("\nBM25 Index Status")
        print("=" * 60)
        print(f"Total collections: {summary['total']}")
        print(f"Indices ready: {summary['ready']}")
        print(f"Indices missing: {summary['missing']}")
        
        if summary['details']:
            print("\nDetails:")
            for name, status in summary['details'].items():
                if status['status'] == 'exists':
                    size = status.get('index_size_mb', 0)
                    print(f"  {name}: {size:.2f} MB")
                else:
                    print(f"  {name}: missing")

# Initialize checker
if 'bm25_prebuilder' in globals():
    index_checker = BM25IndexChecker(bm25_prebuilder)
    print("Index Checker ready")
else:
    print("Pre-builder not available")

Index Checker ready


#### 4.2.4 Execute BM25 Index building

Orchestrates the on-demand pre-building/loading of BM25 indices for all collections using the BM25IndexPreBuilder and reports high-level progress/results.

- Trigger build:
  - Calls bm25_prebuilder.prebuild_all_indices(force_rebuild=False).
    - This method iterates available collections, calls BM25IndexPreBuilder.check_index_status for each, and either loads an existing on-disk index (manager.load_bm25_index) or builds a new BM25 index (manager.build_bm25_index). It returns a summary dict with per-collection outcomes and overall counts/timing.

- Post-run reporting:
  - Inspects the returned summary (results) and prints a final success message when all indices succeeded, otherwise prints a warning showing how many indices were built vs total.


In [24]:
# Execute Index Building
if 'bm25_prebuilder' in globals():
    print("🚀 Starting BM25 index pre-building...")
    
    # Check current status
    if 'index_checker' in globals():
        index_checker.print_status()
    
    # Build indices
    results = bm25_prebuilder.prebuild_all_indices(
        force_rebuild=False  # Change to True to force rebuild
    )
    
    # Show final status
    if results['successful'] == results['total_collections']:
        print("\n✅ All indices ready - system optimized!")
    else:
        print(f"\n⚠️ {results['successful']}/{results['total_collections']} indices built")
else:
    print("❌ Pre-builder not available")

🚀 Starting BM25 index pre-building...

BM25 Index Status
Total collections: 2
Indices ready: 2
Indices missing: 0

Details:
  transcrips_barclasys: 3.70 MB
  pra_rules: 1.63 MB
Building BM25 indices for 2 collections...


Building indices:   0%|          | 0/2 [00:00<?, ?it/s]


✅ Complete: 2/2 indices ready
⏱️  Total time: 0.1s

✅ All indices ready - system optimized!


#### 4.3.1 Score Normalization for Hybrid Search Engine with RRF

Provides deterministic, reusable normalization routines so BM25 and semantic scores are on comparable scales before fusion (used by RRFFusionEngine._add_normalized_scores).

- normalize_bm25(scores: List[float]) -> List[float]: Compress extreme BM25 values and scale into [0,1] for robust fusion.
    - Early-return if input empty or all zeros.
    - Compute 95th percentile (clip_val) and clip scores to [0, clip_val].
    - If clip_val > 0, divide clipped values by clip_val to produce normalized scores in [0,1]; otherwise return zeros.
  - Notes: percentile-based clipping limits the influence of outliers while preserving relative ordering below the clip threshold.

- normalize_semantic(scores: List[float]) -> List[float]: Convert semantic similarity/distance values into a smooth [0,1] score distribution suitable for combination with BM25.
    - Early-return if input empty or all zeros.
    - Compute mean and standard deviation (with small epsilon to avoid div-by-zero).
    - Convert to z-scores, scale z by factor 2, then map through a sigmoid 1/(1+exp(-z*2)) to push values into (0,1).
  - Notes: Z-score + sigmoid centers and compresses the semantic distribution, handling both similarity scores and distance-derived inputs; small std epsilon stabilizes near-constant inputs.

In [25]:
# Score Normalization (Standalone utility)
class ScoreNormalizer:
    """Normalize BM25 and semantic scores for fair fusion"""
    
    @staticmethod
    def normalize_bm25(scores: List[float]) -> List[float]:
        """Clip and normalize BM25 scores (percentile-based)"""
        if not scores or all(s == 0 for s in scores):
            return scores
        
        arr = np.array(scores)
        clip_val = np.percentile(arr, 95)
        clipped = np.clip(arr, 0, clip_val)
        
        if clip_val > 0:
            return (clipped / clip_val).tolist()
        return [0.0] * len(scores)
    
    @staticmethod
    def normalize_semantic(scores: List[float]) -> List[float]:
        """Normalize semantic scores using sigmoid (handles distance-based scores)"""
        if not scores or all(s == 0 for s in scores):
            return scores
        
        arr = np.array(scores)
        mean = np.mean(arr)
        std = np.std(arr) + 1e-8
        
        z_scores = (arr - mean) / std
        normalized = 1 / (1 + np.exp(-z_scores * 2))
        return normalized.tolist()

print("✅ ScoreNormalizer ready")

✅ ScoreNormalizer ready


#### 4.3.2 Hybrid Search (BM25 + Embeddings)

Orchestrates retrieval from two modalities (BM25 and semantic/embedding search) and fuses them with an enhanced Reciprocal Rank Fusion (RRF) implementation.

- RRFFusionEngine: combines two ranked result lists (semantic, BM25) into a single, scored ranking using RRF plus simple confidence blending and cross-method boosting.
  - Important state: params dict (k, semantic_weight, bm25_weight, cross_boost, decay_factor) controls RRF formula, relative weighting, boosting for docs found in both sets, and exponential decay per rank.
  - Methods:
    - fuse(semantic_results, bm25_results) -> List[Dict]: Filters out entries with empty text.
      - Calls _add_normalized_scores on both lists (uses ScoreNormalizer).
      - Iterates semantic results, assigns initial RRF contributions by rank; then iterates BM25 results, adds RRF contributions and applies cross_boost when doc already present.
      - Builds final result dicts that include rrf_score, a computed confidence (mean of normalized factors when present), and final_score = rrf_score + (confidence * 0.05). Sorts by final_score descending.
    - _add_normalized_scores(results, score_type, normalizer) -> results
      - Extracts raw scores, uses ScoreNormalizer.normalize_bm25 or .normalize_semantic to add e.g. 'semantic_score_normalized' or 'bm25_score_normalized' fields to each result.
    - _calculate_rrf(rank, weight) -> float
      - Implements RRF contribution with decay: (weight / (rank + 1 + k)) * decay_factor**rank.
    - _get_doc_id(result) -> str
      - Stable doc id extraction: prefers result['id'] then metadata.chunk_id, otherwise fallback hash(filename+text snippet).


- HybridSearchEngine: fetches candidate lists from BM25 and semantic backends, call fusion engine, provide single-collection and multi-collection search entry points.
  - Methods:
    - search(query, collection_name, top_k=20) -> List[Dict]
      - Ensures BM25 index is loaded (loads/builds via collections_manager as needed).
      - Expands retrieval depth (expanded_k = int(top_k * 2.5)) to collect richer candidate pools.
      - Calls collections_manager.search_bm25 and .semantic_search with expanded_k.
      - Calls RRFFusionEngine.fuse(semantic_results, bm25_results) and returns top_k of fused results.
      - Defensive: returns [] if both retrievals are empty.
    - search_multi(query, collection_names, top_k=10) -> List[Dict]
      - Runs search over each collection (calling search), tags results with source_collection, aggregates.
      - Applies _promote_diversity when searching multiple collections, re-sorts and truncates to requested volume.
    - _promote_diversity(results, collections) -> List[Dict]
      - Simple per-collection boosting to favor under-represented collections (adds/subtracts small constants to final_score based on current counts).
  - Interaction: relies on collections_manager for tokenization, BM25 index management, and Chroma semantic queries; relies on ScoreNormalizer via RRFFusionEngine for comparable scores.

Operational/edge behaviors
- Normalization: semantic and BM25 scores are normalized before fusion (ScoreNormalizer percentile clipping + sigmoid z-score).
- Candidate expansion: uses a larger retrieval window to let fusion and reranking identify cross-method matches.
- Robustness: filters empty texts, loads or builds BM25 indices on demand, and uses simple fallbacks for stable IDs.

Result fields produced (typical)
- final_score, rrf_score, confidence, semantic_rank, bm25_rank, in_both_sets, fusion_method, plus original result fields and normalized score fields.

In [26]:
# RRF Fusion Engine
class RRFFusionEngine:
    """Reciprocal Rank Fusion with enhanced scoring"""
    
    def __init__(self):
        self.params = {
            'k': 60,
            'semantic_weight': 0.6,
            'bm25_weight': 0.4,
            'cross_boost': 1.15,
            'decay_factor': 0.95
        }
    
    def fuse(self, semantic_results: List[Dict], bm25_results: List[Dict]) -> List[Dict]:
        """Fuse semantic and BM25 results using RRF"""
        # Validate inputs
        semantic_results = [r for r in semantic_results if r.get('text')]
        bm25_results = [r for r in bm25_results if r.get('text')]
        
        if not semantic_results and not bm25_results:
            return []
        
        # Normalize scores
        normalizer = ScoreNormalizer()
        semantic_results = self._add_normalized_scores(semantic_results, 'semantic', normalizer)
        bm25_results = self._add_normalized_scores(bm25_results, 'bm25', normalizer)
        
        # Build combined scores
        combined = {}
        
        # Process semantic results
        for rank, result in enumerate(semantic_results):
            doc_id = self._get_doc_id(result)
            rrf_score = self._calculate_rrf(rank, self.params['semantic_weight'])
            
            combined[doc_id] = {
                'result': result,
                'semantic_rank': rank + 1,
                'bm25_rank': None,
                'rrf_score': rrf_score,
                'in_both': False
            }
        
        # Process BM25 results
        for rank, result in enumerate(bm25_results):
            doc_id = self._get_doc_id(result)
            rrf_score = self._calculate_rrf(rank, self.params['bm25_weight'])
            
            if doc_id in combined:
                # Document in both sets - boost score
                combined[doc_id]['rrf_score'] += rrf_score
                combined[doc_id]['rrf_score'] *= self.params['cross_boost']
                combined[doc_id]['bm25_rank'] = rank + 1
                combined[doc_id]['in_both'] = True
            else:
                combined[doc_id] = {
                    'result': result,
                    'semantic_rank': None,
                    'bm25_rank': rank + 1,
                    'rrf_score': rrf_score,
                    'in_both': False
                }
        
        # Build final results
        final_results = []
        for doc_id, data in combined.items():
            result = data['result'].copy()
            
            # Calculate confidence
            factors = []
            if result.get('semantic_score_normalized'):
                factors.append(result['semantic_score_normalized'])
            if result.get('bm25_score_normalized'):
                factors.append(result['bm25_score_normalized'])
            
            confidence = np.mean(factors) if factors else 0.5
            
            # Final score
            final_score = data['rrf_score'] + (confidence * 0.05)
            
            result.update({
                'final_score': final_score,
                'rrf_score': data['rrf_score'],
                'confidence': confidence,
                'semantic_rank': data['semantic_rank'],
                'bm25_rank': data['bm25_rank'],
                'in_both_sets': data['in_both'],
                'fusion_method': 'Enhanced_RRF'
            })
            
            final_results.append(result)
        
        # Sort by final score
        final_results.sort(key=lambda x: x['final_score'], reverse=True)
        return final_results
    
    def _add_normalized_scores(self, results: List[Dict], score_type: str, normalizer: ScoreNormalizer) -> List[Dict]:
        """Add normalized scores to results"""
        scores = [r.get(f'{score_type}_score', 0) for r in results]
        
        if score_type == 'bm25':
            normalized = normalizer.normalize_bm25(scores)
        else:
            normalized = normalizer.normalize_semantic(scores)
        
        for i, result in enumerate(results):
            result[f'{score_type}_score_normalized'] = normalized[i]
        
        return results
    
    def _calculate_rrf(self, rank: int, weight: float) -> float:
        """Calculate RRF score with decay"""
        k = self.params['k']
        decay = self.params['decay_factor'] ** rank
        return (weight / (rank + 1 + k)) * decay
    
    def _get_doc_id(self, result: Dict) -> str:
        """Get consistent document ID"""
        if result.get('id'):
            return result['id']
        
        chunk_id = result.get('metadata', {}).get('chunk_id')
        if chunk_id:
            return chunk_id
        
        # Fallback: hash of text + filename
        text = result.get('text', '')[:100]
        filename = result.get('metadata', {}).get('filename', 'unknown')
        return f"doc_{hash(f'{filename}_{text}')}"

print("✅ RRFFusionEngine ready")

✅ RRFFusionEngine ready


#### 4.3.3 Hybrid Search Engine (Orchestrator)

Orchestrates hybrid retrieval by calling BM25 and semantic backends (via the collections manager), expanding candidate pools, and fusing results with the RRFFusionEngine to produce a single ranked result list.

- search(query, collection_name, top_k=20) -> List[Dict]: Ensures a BM25 index is present for the collection: attempts to load from cache, otherwise triggers build.
  - Expands retrieval depth (expanded_k = int(top_k * 2.5)) to collect larger candidate sets for both modalities.
  - Calls collections_manager.search_bm25(...) and collections_manager.semantic_search(...) with expanded_k.
  - If both retrievals are empty returns [].
  - Passes results to fusion_engine.fuse(semantic_results, bm25_results) and returns the top_k fused entries.
  - Defensive: handles index loading/building on demand and avoids failing when one modality yields no results.

- search_multi(query, collection_names, top_k=10) -> List[Dict]
  - Iterates over collection_names, calling search(...) for each collection.
  - Tags each returned result with source_collection.
  - Aggregates per-collection results, applies diversity promotion when multiple collections are involved, re-sorts by final_score and truncates to a sensible output size (top_k * len(collection_names)).
  - Catches and logs per-collection exceptions to allow partial results.

- _promote_diversity(results, collections) -> List[Dict]
  - Simple post-hoc balancing mechanism: computes per-collection counts and target_per_collection, then nudges final_score with small positive boosts for under-represented collections and small penalties for over-represented ones (constants used: +0.05 / -0.02).
  - Mutates final_score directly to favor distributional diversity across collections.


In [27]:
# Hybrid Search Engine (Orchestrator)
class HybridSearchEngine:
    """Main hybrid search orchestrator - uses collections manager for BM25/semantic search"""
    
    def __init__(self, collections_manager):
        self.manager = collections_manager
        self.fusion_engine = RRFFusionEngine()
    
    def search(self, query: str, collection_name: str, top_k: int = 20) -> List[Dict]:
        """Perform hybrid search on single collection"""
        # Ensure BM25 index loaded
        if collection_name not in self.manager.bm25_indices:
            if not self.manager.load_bm25_index(collection_name):
                self.manager.build_bm25_index(collection_name)
        
        # Expand retrieval for better fusion
        expanded_k = int(top_k * 2.5)
        
        # Get results from both methods
        bm25_results = self.manager.search_bm25(query, collection_name, expanded_k)
        semantic_results = self.manager.semantic_search(query, collection_name, expanded_k)
        
        if not bm25_results and not semantic_results:
            return []
        
        # Fuse results
        fused = self.fusion_engine.fuse(semantic_results, bm25_results)
        return fused[:top_k]
    
    def search_multi(self, query: str, collection_names: List[str], top_k: int = 10) -> List[Dict]:
        """Search across multiple collections"""
        all_results = []
        
        for collection_name in collection_names:
            try:
                results = self.search(query, collection_name, top_k)
                
                # Tag with source collection
                for result in results:
                    result['source_collection'] = collection_name
                
                all_results.extend(results)
            except Exception as e:
                print(f"Error searching {collection_name}: {e}")
                continue
        
        # Apply diversity if multi-collection
        if len(collection_names) > 1:
            all_results = self._promote_diversity(all_results, collection_names)
        
        # Re-sort and limit
        all_results.sort(key=lambda x: x.get('final_score', 0), reverse=True)
        return all_results[:top_k * len(collection_names)]
    
    def _promote_diversity(self, results: List[Dict], collections: List[str]) -> List[Dict]:
        """Promote collection diversity in results"""
        collection_counts = {name: 0 for name in collections}
        target_per_collection = len(results) / len(collections)
        
        for result in results:
            collection = result.get('source_collection', 'unknown')
            if collection not in collection_counts:
                continue
            
            count = collection_counts[collection]
            
            # Boost under-represented collections
            if count < target_per_collection:
                result['final_score'] = result.get('final_score', 0) + 0.05
            else:
                result['final_score'] = result.get('final_score', 0) - 0.02
            
            collection_counts[collection] += 1
        
        return results

print("✅ HybridSearchEngine ready")



✅ HybridSearchEngine ready


#### 4.3.4 Integration and Verification 

Wires the HybridSearchEngine into the running system and verify integration points so downstream orchestration uses the fused BM25+semantic search.
- Instantiates hybrid_search = HybridSearchEngine(collections_manager).
- Updates existing orchestrator objects by attaching the new hybrid search instance:
  - If rag_orchestrator exists, sets rag_orchestrator.hybrid_search = hybrid_search and prints confirmation.
  - If spec_rag exists, sets spec_rag.hybrid_search = hybrid_search and prints confirmation.
- Emits a summary printout confirming initialization and high-level features (BM25+semantic fusion, normalization, cross-boost, multi-collection diversity).

In [28]:
# Initialize Hybrid Search Engine
if 'collections_manager' in globals():
    hybrid_search = HybridSearchEngine(collections_manager)
    
    # Update global references
    if 'rag_orchestrator' in globals():
        rag_orchestrator.hybrid_search = hybrid_search
        print("✅ Updated rag_orchestrator")
    
    if 'spec_rag' in globals():
        spec_rag.hybrid_search = hybrid_search
        print("✅ Updated spec_rag")
    
    print("\n✅ Hybrid Search Engine initialized!")
    print("   • BM25 + Semantic fusion with RRF")
    print("   • Score normalization (percentile + sigmoid)")
    print("   • Cross-method boost for documents in both sets")
    print("   • Multi-collection search with diversity")
else:
    print("❌ collections_manager not available - run Section 4.1 first")


✅ Hybrid Search Engine initialized!
   • BM25 + Semantic fusion with RRF
   • Score normalization (percentile + sigmoid)
   • Cross-method boost for documents in both sets
   • Multi-collection search with diversity


#### 4.4 Cross-Encoder Reranking 

Provides a required cross‑encoder component that scores query/document pairs and reorders fused search results to improve final relevance.

- rerank(query: str, results: List[Dict], top_k: int = 10) -> List[Dict]
  - Inputs: original query string and a list of candidate result dicts (each expected to contain 'text' and optional existing scores).
  - Prepares (query, document_text) pairs, truncating document text to ~512 chars for efficiency.
  - Calls the cross-encoder model to predict a scalar relevance score per pair.
  - Side effects on each result dict:
    - Adds 'rerank_score' (float).
    - Updates 'final_score' by blending existing final_score (if present) and rerank_score with weights 60% original / 40% rerank; if no existing final_score, sets final_score = rerank_score.
  - Returns results sorted by the blended final_score, truncated to top_k.
  - Operates in-place (mutates provided result objects).

Operational notes
- Default cross-encoder model: cross-encoder/ms-marco-MiniLM-L-6-v2.
- Designed for efficiency: truncation of document text and batch prediction via CrossEncoder.predict.
- Integration behavior: the cell treats the cross-encoder as REQUIRED — initialization failures are escalated so downstream components expect reranking to be available.

In [29]:
# Cross-Encoder Reranking
class CrossEncoderReranker:
    """Cross-encoder reranking for improved relevance - REQUIRED component"""
    
    def __init__(self, model_name: str = "cross-encoder/ms-marco-MiniLM-L-6-v2"):
        try:
            from sentence_transformers import CrossEncoder
            self.model = CrossEncoder(model_name)
            self.available = True
            print(f"✅ Cross-encoder loaded: {model_name}")
        except ImportError as e:
            raise ImportError(
                "Cross-encoder unavailable: sentence-transformers not installed.\n"
                "Install with: pip install sentence-transformers\n"
                f"   Error: {e}"
            )
        except Exception as e:
            raise RuntimeError(
                f"Failed to load cross-encoder model: {model_name}\n"
                f"Error: {e}\n"
                "Ensure model is available or check network connection."
            )
    
    def rerank(self, query: str, results: List[Dict], top_k: int = 10) -> List[Dict]:
        """Rerank results using cross-encoder scores"""
        if not results:
            return []
        
        # Prepare query-document pairs (limit text for efficiency)
        pairs = [(query, result.get('text', '')[:512]) for result in results]
        
        # Get cross-encoder scores
        scores = self.model.predict(pairs)
        
        # Add scores to results
        for i, result in enumerate(results):
            result['rerank_score'] = float(scores[i])
            
            # Blend with existing score if present
            if 'final_score' in result:
                result['final_score'] = (
                    result['final_score'] * 0.6 +
                    result['rerank_score'] * 0.4
                )
            else:
                result['final_score'] = result['rerank_score']
        
        # Sort by blended score
        reranked = sorted(results, key=lambda x: x.get('final_score', 0), reverse=True)
        return reranked[:top_k]

# Initialize cross-encoder (MANDATORY - will raise exception if unavailable)
try:
    cross_encoder = CrossEncoderReranker()
    
    print("\n✅ Cross-Encoder Reranking initialized!")
    print("   • Model: cross-encoder/ms-marco-MiniLM-L-6-v2")
    print("   • Blends scores: 60% original + 40% rerank")
    print("   • Status: REQUIRED component (system will not work without it)")
    
except (ImportError, RuntimeError) as e:
    print(f"\n{str(e)}")
    print("\nCRITICAL: Cross-encoder is a REQUIRED component.")
    print("The RAG system cannot function without reranking capability.")
    print("Please install dependencies and re-run this section.")
    cross_encoder = None
    raise

✅ Cross-encoder loaded: cross-encoder/ms-marco-MiniLM-L-6-v2

✅ Cross-Encoder Reranking initialized!
   • Model: cross-encoder/ms-marco-MiniLM-L-6-v2
   • Blends scores: 60% original + 40% rerank
   • Status: REQUIRED component (system will not work without it)


## 5. Intelligent Query Processing and LLM Engine

#### 5.2 Query Classification and Routing 

LLM-first classifier that converts free-form queries into a small, validated routing object. Its main responsibilities are prompt construction, robust JSON extraction/parsing, basic schema validation, and returning normalized fields (type, confidence, subtype, params) for the RAG pipeline to route to metadata queries, summarization, comparisons, or search.

- classify_query(query, collections): Public entrypoint that returns a structured routing object for downstream handlers. It delegates to the LLM-based classifier (`_llm_classify`) and returns the normalized result; if the LLM-based call fails or returns no usable output it surfaces an error (no silent fallback in the implementation).

- _llm_classify(query, collections): Core LLM-driven classifier:
  - Builds a concise instruction prompt that requests a single JSON object describing query_type, confidence, reasoning, and an extracted_params block (e.g., target_document, word_count).
  - Calls llm.generate_response(...) to get a textual response.
  - Extracts JSON from the LLM output using a regex, parses it with json.loads, validates query_type against allowed types, and converts results into the internal normalized dictionary form.
  - Returns None on parsing/validation errors so callers can detect failure.

- _determine_subtype(query_type, query): Post-processing helper that derives a finer-grained subtype from the high-level query_type and the raw query text (e.g., map database queries to metadata_count vs metadata_list, or classify summarization as document_summary vs general_summary).

In [32]:
# Enhanced Query Classification with LLM
class EnhancedQueryClassifier:
    """Robust LLM-based query classifier with fallback patterns"""
    
    def __init__(self, llm_engine: LocalLLMEngine):
        if not llm_engine:
            raise ValueError("LLM engine is required for EnhancedQueryClassifier")
        if not getattr(llm_engine, "available", False):
            raise ValueError("LLM engine must be marked as available")
        
        self.llm = llm_engine
        print(f"✅ Enhanced Query Classifier initialized with LLM engine: {type(llm_engine).__name__}")
    
    def classify_query(self, query: str, collections: List[str]) -> Dict[str, Any]:
        """Classify query using LLM with structured output"""
        
        if not self.llm:
            raise RuntimeError("LLM engine not available")
        
        if not getattr(self.llm, "available", False):
            raise RuntimeError("LLM engine is not marked as available")
        
        result = self._llm_classify(query, collections)
        
        if not result:
            raise RuntimeError("LLM classification failed - no valid response")
        
        return result
    
    def _llm_classify(self, query: str, collections: List[str]) -> Optional[Dict]:
        """LLM-based classification with explicit examples"""
        
        prompt = f"""Classify this RAG system query into ONE type. Respond with JSON only.

Available collections: {', '.join(collections)}

QUERY TYPES:
1. "database_query" - Asking about files/documents in database (NOT content)
2. "summarization" - Requesting summary of document content
3. "comparison" - Comparing documents or checking compliance
4. "search" - General content search/questions
5. "question_answering" - Direct Q&A from documents.

CRITICAL RULES:
- "list files", "count documents", "show available files" → database_query
- "what does X say about Y" → search (asking about CONTENT)
- "summarize document.pdf" → summarization
- "compare X against Y" → comparison
- "Identify compliance" → comparison
- "Identify misalignment" → comparison

Query: "{query}"

Respond with JSON:
{{"query_type": "database_query|summarization|comparison|search",
  "confidence": 0.0-1.0,
  "reasoning": "brief explanation",
  "extracted_params": {{
    "target_document": "filename or null",
    "reference_collection": "collection name or null",
    "word_count": 300
  }}
}}

JSON:"""

        try:
            response = self.llm.generate_response(prompt, temperature=0, max_tokens=200)
            print(f"\n🔍 LLM Response for '{query}':")
            print(f"Raw response: {response[:200]}...")
            
            # Extract JSON
            json_match = re.search(r'\{[^{}]*(?:\{[^{}]*\}[^{}]*)*\}', response, re.DOTALL)
            if not json_match:
                print(f"❌ No JSON found in LLM response for query: {query}")
                return None
            
            data = json.loads(json_match.group(0))
            
            # Validate
            valid_types = {'database_query', 'summarization', 'comparison', 'search', 'question_answering'}
            if data.get('query_type') not in valid_types:
                return None
            
            # Build result
            result = {
                'type': data['query_type'],
                'confidence': float(data.get('confidence', 0.7)),
                'reasoning': data.get('reasoning', ''),
                'subtype': self._determine_subtype(data['query_type'], query),
                **data.get('extracted_params', {})
            }
            return result
        
        except (json.JSONDecodeError, ValueError) as e:
            print(f"LLM classification failed: {e}")
            return None
    
    def _determine_subtype(self, query_type: str, query: str) -> str:
        """Determine subtype based on query type"""
        if query_type == 'database_query':
            if 'count' in query.lower():
                return 'metadata_count'
            elif 'list' in query.lower() or 'show' in query.lower():
                return 'metadata_list'
            return 'metadata_query'
        
        elif query_type == 'summarization':
            return 'document_summary' if re.search(r'\w+\.pdf', query) else 'general_summary'
        
        elif query_type == 'comparison':
            return 'cross_reference_comparison' if 'compliance' in query.lower() else 'document_comparison'
        
        elif query_type == 'question_answering':
            return 'direct_qa'
        
        return 'general_search'
    
    

print("✅ Enhanced Query Classifier with LLM ready")

✅ Enhanced Query Classifier with LLM ready


#### 5.3 Enhanced LLM Query Expansion with Financial Domain Optimization

Augments user queries for the financial RAG pipeline by combining a small deterministic domain lexicon with a short, LLM-driven contextual expansion step to improve recall for hybrid search.

- Constructor (__init__(llm_engine)) :  
  - Requires an LLM engine (`llm_engine.available == True`) and raises immediately if missing.  
  - Initializes a compact, curated `financial_synonyms` map and an `abbreviations` map used as deterministic seed expansions.
    - Purpose of the maps:
      - Seed, not ceiling: They give deterministic hints so expansions include high-value domain terms. The LLM still generates extra terms based on the full prompt/context.
      - Control noise: Small curated lists reduce irrelevant expansions and help keep the downstream search precise.
      - When to expand the maps:
        - Add terms if you find repeated misses for domain-specific language (industry jargon, ticker conventions, local regulation names).
        - Don’t bloat if the LLM already supplies good contextual terms — very large lexicons add maintenance cost and can increase noisy expansions.

- Main operation (expand(query, max_terms=8)) :  
  - Validates `query` is non-empty.  
  - Seeds expansions deterministically by matching tokens from `financial_synonyms` and `abbreviations` found in the query (collects short synonym lists).  
  - Calls the LLM (via `llm.generate_response`) to request a concise, comma-separated list of additional financial terms; parsing is robust to newlines and commas.  
  - Normalizes, deduplicates, and truncates combined terms to `max_terms`.  
  - Returns a structured dict with keys: `original`, `expanded` (original query + appended terms), `matches` (seeded synonyms/abbreviations), `llm_terms` (terms returned by the LLM), and `expansion_count`.

In [33]:
# Enhanced LLM Query Expander 
class FinancialQueryExpander:
    """
    Expand financial queries using domain synonyms + LLM contextual terms.
    Requires a ready `llm_engine` with `generate_response(prompt, temperature, max_tokens)`.
    If LLM is not available this will raise ValueError so the caller can fix it.
    """

    def __init__(self, llm_engine):
        if not llm_engine or not getattr(llm_engine, "available", False):
            raise ValueError("FinancialQueryExpander requires an available LLM engine (llm_engine.available=True).")
        self.llm = llm_engine

        # Small curated domain lexicon to seed expansions (keeps output focused)
        self.financial_synonyms = {
            "revenue": ["income", "sales", "turnover", "earnings"],
            "profit": ["earnings", "net income", "margin"],
            "loss": ["deficit", "decline"],
            "compliance": ["regulatory", "requirements", "governance"],
            "risk": ["exposure", "liability", "uncertainty", "misalignment"],
            "cash": ["liquidity", "working capital", "cash flow"],
            "debt": ["liability", "borrowings", "loan"],
            "margin": ["profitability", "spread"],
            "growth": ["expansion", "increase"]
        }

        self.abbreviations = {
            "EBITDA": "earnings before interest tax depreciation amortization",
            "ROI": "return on investment",
            "CAPEX": "capital expenditure",
            "OPEX": "operating expenditure",
            "YoY": "year over year",
            "QoQ": "quarter over quarter",
            "PRA": "prudential regulatory authority"
        }

    def expand(self, query: str, max_terms: int = 8) -> Dict:
        """
        Expand `query` with domain synonyms and LLM-provided terms.
        Returns dict: { original, expanded, matches, llm_terms, expansion_count }
        """
        if not isinstance(query, str) or not query.strip():
            raise ValueError("Query must be a non-empty string.")

        original = query.strip()
        query_lower = original.lower()

        # 1) Seed expansions from curated lexicon (deterministic)
        matches = {}
        seed_terms = []
        for word, syns in self.financial_synonyms.items():
            if word in query_lower:
                matches[word] = syns[:3]
                seed_terms.extend(syns[:3])

        for abbr, full in self.abbreviations.items():
            if abbr.lower() in query_lower or abbr.lower() in original.lower():
                matches[abbr] = full
                seed_terms.extend(full.split())

        # 2) Always call LLM for contextual expansion (per requirements)
        # Request a short, comma-separated list of related financial terms (3-6)
        needed = max(0, max_terms - len(seed_terms))
        llm_terms = []
        try:
            prompt = (
                f"Provide {max(3, needed)} concise financial query expansion terms for:\n\n"
                f"\"{original}\"\n\n"
                "Respond with a single line containing comma-separated terms only (no explanations)."
            )
            resp = self.llm.generate_response(prompt, temperature=0, max_tokens=60)
            if not resp or not isinstance(resp, str):
                raise RuntimeError("LLM returned empty or invalid response for expansion.")
            # Parse comma-separated terms robustly
            parts = [p.strip().lower() for p in resp.replace("\n", ",").split(",")]
            # Filter out empties and duplicates, keep short tokens/phrases
            llm_terms = []
            for p in parts:
                if not p:
                    continue
                if p in llm_terms:
                    continue
                if len(p) < 2:
                    continue
                llm_terms.append(p)
                if len(llm_terms) >= max(3, needed):
                    break
        except Exception as e:
            # Per requirements: do NOT fallback silently; surface a clear error
            raise RuntimeError(f"LLM expansion failed: {e}")

        # 3) Combine seeds + llm_terms while deduping and limiting
        combined = []
        for t in seed_terms + llm_terms:
            norm = t.strip()
            if not norm:
                continue
            if norm in combined:
                continue
            combined.append(norm)
            if len(combined) >= max_terms:
                break

        expanded_query = original + (" " + " ".join(combined) if combined else "")
        return {
            "original": original,
            "expanded": expanded_query,
            "matches": matches,
            "llm_terms": llm_terms,
            "expansion_count": len(combined)
        }

#### 5.4 Document Finder and Reference Extractor

In [34]:
# Document Finder & Reference Extractor — uses hybrid_search and LLM
class DocumentFinderAndRefExtractor:
    """
    LLM-driven finder that uses the notebook's `hybrid_search` for retrieval and the provided
    `llm_engine` for claim extraction and alignment analysis.
    """

    def __init__(self, llm_engine, hybrid_search):
        if not llm_engine or not getattr(llm_engine, "available", False):
            raise ValueError("LLM engine required and must be available (llm_engine.available=True).")
        if not hybrid_search or not hasattr(hybrid_search, "search"):
            raise ValueError("hybrid_search required and must implement .search(query, collection, top_k).")
        self.llm = llm_engine
        self.hybrid = hybrid_search

    def _extract_json_from_text(self, text: str) -> Any:
        m = re.search(r"(\{.*\}|\[.*\])", text, re.S)
        if m:
            try:
                return json.loads(m.group(1))
            except Exception:
                pass
        try:
            return json.loads(text)
        except Exception:
            raise ValueError("Could not parse JSON from LLM response.")

    def find_documents(self, query: str, collections: List[str], top_k: int = 10) -> List[Dict]:
        """Search across collections using hybrid_search and return aggregated ranked results."""
        results = []
        for col in collections:
            results.extend(self.hybrid.search(query, col, top_k))
        results.sort(key=lambda r: r.get("final_score", r.get("score", 0)), reverse=True)
        return results[: top_k * len(collections)]

    def extract_claims(self, document_text: str, max_claims: int = 8) -> List[Dict]:
        """
        Ask the LLM to extract concise factual claims. Returns list of {'id','text'}.
        Raises on parsing or LLM errors.
        """
        prompt = (
            f"Extract up to {max_claims} concise factual claims from the document below.\n"
            "Return a JSON array of objects with keys: id, text. No extra commentary.\n\n"
            f"DOCUMENT:\n{document_text[:8000]}"
        )
        try:
            resp = self.llm.generate_response(prompt, temperature=0.0, max_tokens=600)
        except Exception as e:
            raise RuntimeError(f"LLM claim extraction failed: {e}")
        parsed = self._extract_json_from_text(resp)
        if not isinstance(parsed, list):
            raise ValueError("LLM did not return a JSON array of claims.")
        claims = []
        for i, item in enumerate(parsed[:max_claims]):
            text = item.get("text") if isinstance(item, dict) else str(item)
            claims.append({"id": f"claim_{i+1}", "text": text.strip()})
        return claims

    def retrieve_references(self, claim_text: str, ref_collection: str, top_k: int = 5) -> List[Dict]:
        """
        Retrieve reference passages using existing hybrid_search (BM25+semantic+fusion).
        Returns list of dicts with 'text' and 'metadata'.
        """
        return self.hybrid.search(claim_text, ref_collection, top_k)

    def analyze_claim_against_refs(self, claim: Dict, refs: List[Dict], target_doc: str, ref_collection: str) -> Dict:
        """
        Use LLM to determine alignment. Returns:
         { claim_id, claim_text, has_misalignment, severity, explanation, references }
        """
        refs_text = "\n\n".join(
            f"Ref {i+1} ({r.get('metadata', {}).get('filename','unknown')}): {r.get('text','')[:800]}"
            for i, r in enumerate(refs[:3])
        ) or "No references found."
        prompt = (
            "Compare the TARGET statement to the REFERENCES. Return only JSON with keys:\n"
            "status: 'ALIGNED' or 'MISALIGNED', severity: 'high'|'medium'|'low'|null, explanation: string.\n\n"
            f"TARGET DOCUMENT: {target_doc}\n\nSTATEMENT: {claim['text'][:1200]}\n\n"
            f"REFERENCES ({ref_collection}):\n{refs_text}"
        )
        try:
            resp = self.llm.generate_response(prompt, temperature=0.0, max_tokens=500)
        except Exception as e:
            raise RuntimeError(f"LLM alignment analysis failed: {e}")
        parsed = self._extract_json_from_text(resp)
        if not isinstance(parsed, dict):
            raise ValueError("LLM alignment response not a JSON object.")
        status = parsed.get("status", "").upper()
        has_misalignment = status != "ALIGNED"
        severity = parsed.get("severity") or (("medium") if has_misalignment else None)
        explanation = parsed.get("explanation", "").strip()
        return {
            "claim_id": claim.get("id"),
            "claim_text": claim.get("text"),
            "has_misalignment": bool(has_misalignment),
            "severity": severity,
            "explanation": explanation,
            "references": [
                {"filename": r.get("metadata", {}).get("filename", "unknown"),
                 "snippet": r.get("text", "")[:400]}
                for r in refs[:3]
            ]
        }

    def compare_document_to_collection(self, document_text: str, target_doc_name: str,
                                       ref_collection: str, max_claims: int = 8, top_k_refs: int = 5) -> Dict[str, Any]:
        """
        Full pipeline:
         - extract claims (LLM)
         - for each claim retrieve refs (hybrid_search)
         - analyze each claim vs refs (LLM)
        Returns summary dict with analyses and confidence.
        """
        claims = self.extract_claims(document_text, max_claims=max_claims)
        analyses = []
        for claim in claims:
            refs = self.retrieve_references(claim["text"], ref_collection, top_k_refs)
            analysis = self.analyze_claim_against_refs(claim, refs, target_doc_name, ref_collection)
            analyses.append(analysis)
        misalignments = [a for a in analyses if a.get("has_misalignment")]
        confidence = round(max(0.0, 1.0 - (len(misalignments) / max(1, len(claims)))), 3)
        return {
            "target_document": target_doc_name,
            "reference_collection": ref_collection,
            "total_claims": len(claims),
            "misalignments": misalignments,
            "analyses": analyses,
            "confidence": confidence
        }

In [48]:
# LLM-driven, verified document filter (uses hybrid_search + verification)
import json, re
from typing import List, Dict, Any

def _extract_json_from_text(text: str) -> Any:
    m = re.search(r"(\{.*\}|\[.*\])", text, re.S)
    if m:
        try:
            return json.loads(m.group(1))
        except Exception:
            pass
    try:
        return json.loads(text)
    except Exception as e:
        raise ValueError(f"Could not parse JSON from LLM response: {e}\nRaw: {text[:400]}")

def _metadata_year_matches(meta: dict, year: int) -> bool:
    if not meta:
        return False
    # look for explicit numeric year fields and date-like strings
    for k, v in meta.items():
        s = str(v or "").lower()
        if re.search(rf"\b{year}\b", s):
            return True
        # common date patterns: YYYY-MM-DD etc
        if re.search(rf"{year}[-/]", s) or re.search(rf"[-/]{year}", s):
            return True
    return False

def find_documents_with_verified_filters(llm_engine, hybrid_search, user_query: str,
                                         collections: List[str], year_hint: int = None,
                                         candidate_k: int = 50) -> Dict[str, Any]:
    """
    Hybrid retrieval -> LLM filter -> deterministic verification -> (optional) LLM recheck.
    Returns structured dict with filters, matches and verification details.
    """
    if not llm_engine or not getattr(llm_engine, "available", False):
        raise ValueError("LLM engine required and not available (llm_engine.available=True).")
    if not hybrid_search or not hasattr(hybrid_search, "search"):
        raise ValueError("hybrid_search required and must implement .search(query, collection, top_k).")

    # 1) get candidates
    candidates = []
    for col in collections:
        candidates.extend(hybrid_search.search(user_query, col, candidate_k))
    if not candidates:
        return {"filters": {}, "candidates": [], "matched": [], "llm_assessment": {"found": False}, "verification": [], "message": "No candidates returned by hybrid_search."}

    # 2) prepare compact candidate info including relevant metadata fields
    items = []
    for i, c in enumerate(candidates[:candidate_k]):
        meta = c.get("metadata") or {}
        filename = meta.get("filename") or meta.get("source") or meta.get("doc_id") or "<unknown>"
        # try to pull an explicit year-like field if present
        year_field = None
        for k in ("year", "file_year", "date", "file_date"):
            if k in meta:
                year_field = meta[k]
                break
        snippet = (c.get("text") or "")[:600].replace("\n", " ").strip()
        items.append({"i": i, "filename": str(filename), "year_meta": str(year_field) if year_field is not None else None, "snippet": snippet})

    items_json = json.dumps(items, ensure_ascii=False, indent=2)

    # 3) prompt LLM to extract filters and pick matching indexes, and to state evidence per index
    prompt = (
        "You are given a user query and a list of candidate documents (index, filename, year_meta, snippet).\n"
        f"USER QUERY: {user_query}\n\n"
        "Task (strict JSON only):\n"
        "  1) Extract filters implied by the query (year, exact_title, title_keywords, author, company, other).\n"
        "  2) From the candidates, identify which documents match those filters.\n"
        "  3) For each selected index return the evidence used (one of: 'metadata_year','filename','snippet','other') and the metadata value used.\n\n"
        "Return JSON: { filters: {...}, found: bool, indexes: [ints], evidence: {index: {evidence: 'metadata_year'|'filename'|'snippet'|'other', value: '...'}}, explanation: '...'}\n\n"
        "Candidates:\n" + items_json + "\n\nRespond with JSON only."
    )

    try:
        resp = llm_engine.generate_response(prompt, temperature=0.0, max_tokens=700)
    except Exception as e:
        raise RuntimeError(f"LLM call failed: {e}")

    assessment = _extract_json_from_text(resp)
    if not isinstance(assessment, dict):
        raise ValueError("LLM response is not a JSON object.")

    # normalization
    idxs = []
    for v in assessment.get("indexes", []):
        try:
            idxs.append(int(v))
        except Exception:
            continue

    # 4) deterministic verification: check metadata/filename/snippet for year if year filter present
    verification = []
    target_year = assessment.get("filters", {}).get("year") or year_hint
    for idx in idxs:
        entry = {"index": idx, "llm_evidence": assessment.get("evidence", {}).get(str(idx)) or assessment.get("evidence", {}).get(idx)}
        meta = candidates[idx].get("metadata") or {}
        filename = str(meta.get("filename") or meta.get("source") or "")
        md_match = _metadata_year_matches(meta, target_year) if target_year else False
        filename_match = bool(re.search(rf"\b{target_year}\b", filename)) if target_year else False
        snippet_match = bool(re.search(rf"\b{target_year}\b", (candidates[idx].get("text") or ""))) if target_year else False
        verification.append({
            "index": idx,
            "metadata_year_present": bool(md_match),
            "filename_contains_year": bool(filename_match),
            "snippet_contains_year": bool(snippet_match),
            "llm_evidence": entry.get("llm_evidence")
        })

    # 5) If LLM said found=True but verification shows none of the selected indexes have year evidence,
    #    re-prompt LLM with the verification table and ask it to reconsider.
    verified_matches = [v for v in verification if v["metadata_year_present"] or v["filename_contains_year"] or v["snippet_contains_year"]]
    if assessment.get("found") and not verified_matches and target_year:
        followup_prompt = (
            "You previously identified candidate indexes as matching the user's filters. However, deterministic checks show none of the selected candidates contain the requested year in metadata, filename, or snippet.\n"
            "Please re-evaluate the same candidate list and return corrected JSON with the same schema (filters, found, indexes, evidence, explanation). Use metadata year preferentially; if no reliable matches exist, set found:false and explain.\n\n"
            "Original USER QUERY: " + user_query + "\n\n"
            "Candidates (index, filename, year_meta, snippet):\n" + items_json + "\n\n"
            "Deterministic verification results:\n" + json.dumps(verification, ensure_ascii=False, indent=2) + "\n\nRespond with JSON only."
        )
        try:
            resp2 = llm_engine.generate_response(followup_prompt, temperature=0.0, max_tokens=700)
        except Exception as e:
            raise RuntimeError(f"LLM follow-up call failed: {e}")
        assessment2 = _extract_json_from_text(resp2)
        if not isinstance(assessment2, dict):
            raise ValueError("LLM follow-up response is not a JSON object.")
        assessment = assessment2
        # recompute idxs and verification based on new assessment
        idxs = []
        for v in assessment.get("indexes", []):
            try:
                idxs.append(int(v))
            except Exception:
                continue
        verification = []
        for idx in idxs:
            meta = candidates[idx].get("metadata") or {}
            filename = str(meta.get("filename") or meta.get("source") or "")
            md_match = _metadata_year_matches(meta, target_year) if target_year else False
            filename_match = bool(re.search(rf"\b{target_year}\b", filename)) if target_year else False
            snippet_match = bool(re.search(rf"\b{target_year}\b", (candidates[idx].get("text") or ""))) if target_year else False
            verification.append({
                "index": idx,
                "metadata_year_present": bool(md_match),
                "filename_contains_year": bool(filename_match),
                "snippet_contains_year": bool(snippet_match),
                "llm_evidence": assessment.get("evidence", {}).get(str(idx)) or assessment.get("evidence", {}).get(idx)
            })

    # 6) finalize matched list (only indexes that exist in candidates)
    matched = []
    for idx in idxs:
        if 0 <= idx < len(candidates):
            matched.append(candidates[idx])

    message = assessment.get("explanation", "")
    if assessment.get("found") and not matched:
        message = (message or "") + " (LLM indicated matches but no valid indexes or verification failed.)"

    return {
        "filters": assessment.get("filters", {}),
        "candidates": candidates[:candidate_k],
        "matched": matched,
        "llm_assessment": assessment,
        "verification": verification,
        "message": message or ("Found matches" if matched else "No reliable matches found")
    }

In [49]:
models_dir = Path(MODELS_DIR) if 'MODELS_DIR' in globals() and MODELS_DIR else Path.cwd() / "models"
print("Using models_dir:", models_dir)
chosen_model_dirname = "Llama-3.1-8B-Instruct-4bit"

llm_engine = LocalLLMEngine(models_dir, chosen_model_dirname)
print("LLM loaded. Resolved path:", llm_engine.model_info.get('resolved_path', llm_engine.model_info.get('name')))


Using models_dir: /Users/rodrigograndy/Desktop/coding_projects/posts_website_drafts/rag_boe/models
Loading MLX model from: /Users/rodrigograndy/Desktop/coding_projects/posts_website_drafts/rag_boe/models/Llama-3.1-8B-Instruct-4bit ...
✅ MLX Model loaded: Llama-3.1-8B-Instruct-4bit (resolved: /Users/rodrigograndy/Desktop/coding_projects/posts_website_drafts/rag_boe/models/Llama-3.1-8B-Instruct-4bit)
LLM loaded. Resolved path: /Users/rodrigograndy/Desktop/coding_projects/posts_website_drafts/rag_boe/models/Llama-3.1-8B-Instruct-4bit


In [50]:
finder = DocumentFinderAndRefExtractor(llm_engine, globals()['hybrid_search'])
print("DocumentFinderAndRefExtractor created successfully.")

query = "list files for 2022 in collections"

res = find_documents_with_verified_filters(
    llm_engine=llm_engine,
    hybrid_search=hybrid_search,
    user_query=query,
    collections=["pra_rules"],  # choose relevant collections
    candidate_k=20
)

print("Parsed filters:", res["filters"])
print("LLM assessment:", res["llm_assessment"])
print("Matches:", len(res["matched"]))
for i, r in enumerate(res["matched"], 1):
    meta = r.get("metadata", {}) or {}
    print(f"{i}. {meta.get('filename','<unknown>')}  score:{r.get('final_score', r.get('score',0))}")

DocumentFinderAndRefExtractor created successfully.
Parsed filters: {'year': '2022', 'exact_title': False, 'title_keywords': False, 'author': False, 'company': False, 'other': False}
LLM assessment: {'filters': {'year': '2022', 'exact_title': False, 'title_keywords': False, 'author': False, 'company': False, 'other': False}, 'found': True, 'indexes': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19], 'evidence': {'0': {'evidence': 'metadata_year', 'value': '2025'}, '1': {'evidence': 'metadata_year', 'value': '2025'}, '2': {'evidence': 'metadata_year', 'value': '2025'}, '3': {'evidence': 'metadata_year', 'value': '2025'}, '4': {'evidence': 'metadata_year', 'value': '2025'}, '5': {'evidence': 'metadata_year', 'value': '2025'}, '6': {'evidence': 'metadata_year', 'value': '2025'}, '7': {'evidence': 'metadata_year', 'value': '2025'}, '8': {'evidence': 'metadata_year', 'value': '2025'}, '9': {'evidence': 'metadata_year', 'value': '2020'}, '10': {'evidence': 'metadata_yea

#### 5.5 Results Processor with Confidence Scoring

In [33]:
# Results Processor with Confidence Scoring

class ResultsProcessor:
    """Process and score search results with confidence metrics"""
    
    def __init__(self, cross_encoder=None):
        self.cross_encoder = cross_encoder
        self.confidence_thresholds = {'high': 0.8, 'medium': 0.5, 'low': 0.3}
    
    def process_results(self, results: List[Dict], query: str,
                       apply_reranking: bool = True,
                       min_confidence: float = 0.3) -> Dict[str, Any]:
        """
        Process and enhance search results with confidence scoring.
        
        Args:
            results: Raw search results with text, scores, metadata
            query: Original query string
            apply_reranking: Whether to apply cross-encoder reranking
            min_confidence: Minimum confidence threshold for filtering
        
        Returns:
            Dict with processed results, statistics, and average confidence
        """
        if not results:
            return self._empty_results(query)
        
        processed_results = []
        
        for result in results:
            confidence = self._calculate_confidence(result)
            
            # Filter low-confidence results
            if confidence < min_confidence:
                continue
            
            # Enhance with metadata
            enhanced_result = {
                **result,
                'confidence': confidence,
                'confidence_level': self._get_confidence_level(confidence),
                'processed_text': self._process_text_snippet(result.get('text', '')),
                'relevance_indicators': self._extract_relevance_indicators(result, query)
            }
            
            processed_results.append(enhanced_result)
        
        # Apply cross-encoder reranking if available
        if apply_reranking and self.cross_encoder and processed_results:
            processed_results = self._apply_reranking(query, processed_results)
        
        # Sort by combined score
        processed_results.sort(
            key=lambda x: (x.get('final_score', 0) + x.get('confidence', 0)) / 2,
            reverse=True
        )
        
        avg_confidence = (
            sum(r['confidence'] for r in processed_results) / len(processed_results)
            if processed_results else 0.0
        )
        
        return {
            'results': processed_results,
            'total_found': len(results),
            'total_processed': len(processed_results),
            'average_confidence': avg_confidence,
            'query': query
        }
    
    def _calculate_confidence(self, result: Dict) -> float:
        """Calculate multi-factor confidence score"""
        factors = []
        
        # Score-based factors
        if 'final_score' in result:
            factors.append(min(result['final_score'] * 2, 1.0))
        if 'semantic_score' in result:
            factors.append(result['semantic_score'])
        if 'bm25_score' in result:
            factors.append(min(result['bm25_score'] / 10, 1.0))
        
        # Text length factor (optimal: 200-1000 chars)
        text_len = len(result.get('text', ''))
        if 200 <= text_len <= 1000:
            factors.append(0.8)
        elif 100 <= text_len <= 1500:
            factors.append(0.6)
        else:
            factors.append(0.4)
        
        # Metadata completeness
        metadata = result.get('metadata', {})
        if metadata.get('filename') and metadata.get('chunk_id'):
            factors.append(0.7)
        
        return np.mean(factors) if factors else 0.5
    
    def _get_confidence_level(self, confidence: float) -> str:
        """Map confidence score to level label"""
        if confidence >= self.confidence_thresholds['high']:
            return 'high'
        elif confidence >= self.confidence_thresholds['medium']:
            return 'medium'
        return 'low'
    
    def _process_text_snippet(self, text: str, max_length: int = 300) -> str:
        """Clean and truncate text for display"""
        if not text:
            return ""
        
        # Normalize whitespace
        clean_text = re.sub(r'\s+', ' ', text.strip())
        
        if len(clean_text) <= max_length:
            return clean_text
        
        # Break at sentence boundary if possible
        truncated = clean_text[:max_length]
        last_period = truncated.rfind('.')
        last_space = truncated.rfind(' ')
        
        if last_period > max_length - 50:
            return truncated[:last_period + 1]
        elif last_space > max_length - 20:
            return truncated[:last_space] + "..."
        return truncated + "..."
    
    def _extract_relevance_indicators(self, result: Dict, query: str) -> List[str]:
        """Extract why this result is relevant"""
        indicators = []
        text_lower = result.get('text', '').lower()
        query_terms = query.lower().split()
        
        # Direct term matches
        matches = [term for term in query_terms if term in text_lower]
        if matches:
            indicators.append(f"Contains: {', '.join(matches[:3])}")
        
        # Score indicators
        if result.get('semantic_score', 0) > 0.7:
            indicators.append("High semantic similarity")
        if result.get('bm25_score', 0) > 5:
            indicators.append("Strong keyword match")
        
        # Metadata indicators
        metadata = result.get('metadata', {})
        if metadata.get('doc_type') == 'financial_document':
            indicators.append("Financial document")
        
        return indicators
    
    def _apply_reranking(self, query: str, results: List[Dict]) -> List[Dict]:
        """Apply cross-encoder reranking if available"""
        if not self.cross_encoder or not hasattr(self.cross_encoder, 'predict'):
            return results
        
        try:
            # Prepare pairs (truncate for efficiency)
            pairs = [(query, result.get('text', '')[:500]) for result in results]
            
            # Get reranking scores
            rerank_scores = self.cross_encoder.predict(pairs)
            
            # Update results with blended scores
            for i, result in enumerate(results):
                result['rerank_score'] = float(rerank_scores[i])
                result['final_score'] = (
                    result.get('final_score', 0) * 0.7 +
                    result['rerank_score'] * 0.3
                )
            
            return results
        
        except Exception as e:
            print(f"Reranking failed: {e}")
            return results
    
    def _empty_results(self, query: str) -> Dict[str, Any]:
        """Return empty results structure"""
        return {
            'results': [],
            'total_found': 0,
            'total_processed': 0,
            'average_confidence': 0.0,
            'query': query
        }

print("✅ ResultsProcessor ready")
print("   • Multi-factor confidence scoring")
print("   • Cross-encoder reranking integration")
print("   • Text processing and relevance indicators")
print("   • Confidence-based filtering")

✅ ResultsProcessor ready
   • Multi-factor confidence scoring
   • Cross-encoder reranking integration
   • Text processing and relevance indicators
   • Confidence-based filtering


#### 5.6 Initialize LLM Components

In [34]:
# Initialize LLM engine with model detection
if 'MODELS_DIR' not in globals():
    print("⚠️ MODELS_DIR not defined - run Section 1.4 first")
    llm_engine = None
else:
    models_path = Path(MODELS_DIR)
    
    # Check for model DIRECTORIES (not just files)
    model_dirs = {}
    for key, info in LocalLLMEngine.RECOMMENDED_MODELS.items():
        # Try multiple name variants for directories
        candidates = [
            models_path / info['name'],                                    # Exact name
            models_path / info['name'].replace('.4bit', '-4bit'),          # Dash variant
            models_path / info['name'].replace('.4bit', '-4bit').replace('.', '-')  # Full dash
        ]
        
        for candidate in candidates:
            if candidate.exists() and candidate.is_dir():  # ✅ Check if it's a directory
                model_dirs[key] = {
                    'key': key,
                    'name': info['name'],
                    'params': info['params'],
                    'quant': info['quant'],
                    'size_gb': info['size_gb'],
                    'path': str(candidate)
                }
                break
    
    if not model_dirs:
        print(f"⚠️ No models found in {models_path}")
        print("\nSearched for:")
        for key, info in LocalLLMEngine.RECOMMENDED_MODELS.items():
            print(f"   • {info['name']} ({key})")
        
        # Show what IS in the directory
        if models_path.exists():
            existing = list(models_path.iterdir())
            if existing:
                print(f"\nFound in models directory:")
                for item in existing[:5]:
                    item_type = "dir" if item.is_dir() else "file"
                    print(f"   • {item.name} ({item_type})")
        
        llm_engine = None
    else:
        # Display available models
        print(f"📦 Found {len(model_dirs)} model(s):")
        for model_info in model_dirs.values():
            print(f"   • {model_info['key']}: {model_info['params']} ({model_info['quant']}, ~{model_info['size_gb']}GB)")
            print(f"     Path: {model_info['path']}")
        
        # Select model (prefer llama3.1-8b-4bit_mlx, fallback to first available)
        model_keys = list(model_dirs.keys())
        selected_key = 'llama3.1-8b-4bit_mlx' if 'llama3.1-8b-4bit_mlx' in model_keys else model_keys[0]
        
        print(f"\n🎯 Selected: {selected_key}")
        
        # Initialize LLM engine with detected directory path
        try:
            selected_info = model_dirs[selected_key]
            llm_engine = LocalLLMEngine(models_path, selected_key)
            
            if llm_engine.available:
                print(f"✅ Model loaded: {llm_engine.model_info['name']}")
                print(f"   Parameters: {llm_engine.model_info['params']}, Quantization: {llm_engine.model_info['quant']}")
                print(f"   Location: {llm_engine.model_path}")
            
        except Exception as e:
            print(f"❌ Failed to initialize: {e}")
            llm_engine = None

# Initialize other Section 5 components if LLM engine available
if llm_engine and llm_engine.available:
    query_classifier = EnhancedQueryClassifier(llm_engine)
    doc_finder = DocumentFinder(collections_manager) if 'collections_manager' in globals() else None
    query_expander = FinancialQueryExpander(llm_engine)
    results_processor = ResultsProcessor(cross_encoder if 'cross_encoder' in globals() else None)
    
    print("\n✅ Section 5 components initialized:")
    print("   • LocalLLMEngine")
    print("   • EnhancedQueryClassifier")
    print("   • DocumentFinder")
    print("   • FinancialQueryExpander")
    print("   • ResultsProcessor")
    
    # Quick LLM test
    print("\n🧪 Testing LLM...")
    test_response = llm_engine.generate_response("Say 'OK' in one word.", temperature=0.0, max_tokens=5)
    
    if test_response and not test_response.startswith("Error"):
        print(f"   ✅ LLM responding: {test_response[:20]}")
    else:
        print(f"   ⚠️ Test response: {test_response}")
else:
    print("\n⚠️ LLM engine not available - Section 5 components not initialized")
    print("   Some RAG features will be limited without LLM")

📦 Found 1 model(s):
   • mistral-7b-4bit_mlx: 7B (4bit mlx, ~3.9GB)
     Path: /Users/rodrigograndy/Desktop/coding_projects/posts_website_drafts/rag_boe/models/mistral-7b-instruct-v0.3-4bit

🎯 Selected: mistral-7b-4bit_mlx
Loading MLX model: mistral-7b-instruct-v0.3-4bit...
✅ MLX Model loaded: mistral-7b-instruct-v0.3-4bit
   Parameters: 7B, Quantization: 4bit mlx
   Location: /Users/rodrigograndy/Desktop/coding_projects/posts_website_drafts/rag_boe/models/mistral-7b-instruct-v0.3-4bit
   🚀 Optimized for Apple Silicon
✅ Model loaded: mistral-7b-instruct-v0.3-4bit
   Parameters: 7B, Quantization: 4bit mlx
   Location: /Users/rodrigograndy/Desktop/coding_projects/posts_website_drafts/rag_boe/models/mistral-7b-instruct-v0.3-4bit

✅ Section 5 components initialized:
   • LocalLLMEngine
   • EnhancedQueryClassifier
   • DocumentFinder
   • FinancialQueryExpander
   • ResultsProcessor

🧪 Testing LLM...
   ✅ LLM responding: OK. How can I


## 6. Enhanced User Interface

#### 6.0 Enhanced Citation and Cross-Document Analysis Systems

In [49]:
# Enhanced Citation and Cross-Document Analysis Systems
class EnhancedCitationSystem:
    """Generate precise citations with confidence scores from search results"""
    
    def __init__(self, collections_manager):
        self.manager = collections_manager
    
    def generate_precise_citations(self, references: List[Dict]) -> List[Dict]:
        """Create structured citations from search results"""
        citations = []
        
        for i, ref in enumerate(references):
            if not ref.get('text'):
                continue
            
            metadata = ref.get('metadata', {})
            confidence = ref.get('confidence', 0.0)
            
            citation = {
                'id': f"cite_{i+1}",
                'filename': metadata.get('filename', 'Unknown'),
                'collection': ref.get('collection', metadata.get('collection', 'Unknown')),
                'chunk_id': metadata.get('chunk_id'),
                'page_num': metadata.get('page_num'),
                'confidence_score': confidence,
                'relevance_score': ref.get('final_score', 0.0),
                'text_snippet': ref['text'][:150] + '...' if len(ref['text']) > 150 else ref['text'],
                'source_type': 'document_chunk',
                'verification_status': 'verified' if confidence > 0.7 else 'low_confidence'
            }
            
            citations.append(citation)
        
        return citations
    
    def format_citation_text(self, citation: Dict) -> str:
        """Format single citation for inline display"""
        return (f"[{citation['id']}: {citation['filename']} "
                f"from {citation['collection']}, "
                f"confidence: {citation['confidence_score']:.2f}]")


class CrossDocumentAnalyzer:
    """Analyze patterns and consistency across multiple document sources"""
    
    def __init__(self, llm_engine, collections_manager):
        self.llm_engine = llm_engine
        self.manager = collections_manager
    
    def analyze_cross_document_patterns(self, references: List[Dict], query: str) -> Dict[str, Any]:
        """Analyze document coverage and consistency across sources"""
        
        # Group references by document
        docs = {}
        for ref in references:
            filename = ref.get('metadata', {}).get('filename', 'Unknown')
            if filename not in docs:
                docs[filename] = {
                    'chunks': [],
                    'confidence_sum': 0,
                    'collection': ref.get('collection', 'Unknown')
                }
            
            docs[filename]['chunks'].append(ref)
            docs[filename]['confidence_sum'] += ref.get('confidence', 0)
        
        # Calculate per-document insights
        insights = []
        for filename, data in docs.items():
            chunk_count = len(data['chunks'])
            avg_confidence = data['confidence_sum'] / chunk_count if chunk_count else 0
            
            # Simple coverage assessment
            if chunk_count > 2:
                coverage = 'high'
            elif chunk_count > 1:
                coverage = 'medium'
            else:
                coverage = 'low'
            
            insights.append({
                'filename': filename,
                'collection': data['collection'],
                'chunk_count': chunk_count,
                'average_confidence': avg_confidence,
                'coverage_assessment': coverage
            })
        
        # Assess overall consistency
        consistency = self._assess_consistency(references)
        
        # Determine information completeness
        total_refs = len(references)
        if total_refs > 5:
            completeness = 'high'
        elif total_refs > 2:
            completeness = 'medium'
        else:
            completeness = 'low'
        
        return {
            'total_documents': len(docs),
            'document_insights': insights,
            'cross_document_consistency': consistency,
            'information_completeness': completeness
        }
    
    def _assess_consistency(self, references: List[Dict]) -> str:
        """Simple consistency check based on confidence variance"""
        if len(references) < 2:
            return 'insufficient_data'
        
        confidences = [ref.get('confidence', 0) for ref in references]
        avg_conf = np.mean(confidences)
        var_conf = np.var(confidences)
        
        # High consistency: high average, low variance
        if avg_conf > 0.7 and var_conf < 0.1:
            return 'high'
        elif avg_conf > 0.5:
            return 'medium'
        else:
            return 'low'


print("✅ Citation and Cross-Document Analysis Systems initialized")
print("   • EnhancedCitationSystem: Generate structured citations")
print("   • CrossDocumentAnalyzer: Assess coverage and consistency")

✅ Citation and Cross-Document Analysis Systems initialized
   • EnhancedCitationSystem: Generate structured citations
   • CrossDocumentAnalyzer: Assess coverage and consistency


#### 6.0.1 Cross-Reference Comparison System

In [50]:
# Cross-Reference Comparison System

class CrossReferenceComparator:
    """
    Compare target documents against reference collections (e.g., regulations).
    Identifies misalignments and compliance issues.
    """
    
    def __init__(self, llm_engine, collections_manager, embedding_model, hybrid_search=None):
        self.llm_engine = llm_engine
        self.manager = collections_manager
        self.embedding_model = embedding_model
        self.hybrid_search = hybrid_search
    
    def compare_documents(self, target_doc: str, reference_collection: str,
                         target_collection: str = None, top_k: int = 20) -> Dict[str, Any]:
        """
        Main comparison entry point.
        Returns comprehensive report with misalignments and confidence score.
        """
        # Step 1: Get target document content
        target_content = self._get_document_content(target_doc, target_collection)
        if not target_content:
            return {
                'status': 'error',
                'message': f"Document not found: {target_doc}",
                'target_doc': target_doc,
                'reference_collection': reference_collection
            }
        
        # Step 2: Generate document overview
        overview = self._generate_overview(target_doc, target_content)
        
        # Step 3: Extract and analyze key claims
        claims = self._extract_claims(target_content, max_claims=15)
        misalignments = []
        
        for claim in claims:
            # Get relevant reference passages
            refs = self._retrieve_references(claim['text'], reference_collection, top_k)
            
            # Check alignment
            analysis = self._check_alignment(claim, refs, target_doc, reference_collection)
            if analysis['has_misalignment']:
                misalignments.append(analysis)
        
        # Step 4: Build final report
        confidence = self._calculate_confidence(misalignments, len(claims))
        return self._build_report(target_doc, reference_collection, overview, 
                                  misalignments, confidence, len(claims))
    
    def detect_query_intent(self, query: str) -> Dict[str, Any]:
        """Detect if query requests cross-reference comparison."""
        query_lower = query.lower()
        
        # Check for comparison patterns
        comparison_verbs = ['compare', 'check', 'analyze', 'review', 'verify', 'assess']
        comparison_indicators = ['against', 'with', 'compliance', 'conflict', 'misalign']
        
        has_verb = any(verb in query_lower for verb in comparison_verbs)
        has_indicator = any(ind in query_lower for ind in comparison_indicators)
        
        if not (has_verb or has_indicator):
            return {'is_comparison_query': False}
        
        # Extract target document and reference collection
        intent = {
            'is_comparison_query': True,
            'target_document': None,
            'reference_collection': None,
            'original_query': query
        }
        
        # Pattern matching for "compare X against Y"
        patterns = [
            r'(?:compare|check|analyze)\s+([^\s]+(?:\s+[^\s]+){0,5}?)\s+(?:against|with)\s+(?:the\s+)?([^\s]+)',
            r'(?:does|is)\s+([^\s]+(?:\s+[^\s]+){0,5}?)\s+(?:align|comply)\s+(?:with|to)\s+([^\s]+)'
        ]
        
        for pattern in patterns:
            match = re.search(pattern, query_lower)
            if match:
                intent['target_document'] = match.group(1).strip()
                intent['reference_collection'] = self._map_collection(match.group(2).strip())
                break
        
        return intent
    
    def _get_document_content(self, doc_name: str, collection_name: str = None) -> List[Dict]:
        """Retrieve all chunks for specified document."""
        collections = [collection_name] if collection_name else self.manager.get_available_collections()
        
        all_chunks = []
        for coll_name in collections:
            collection = self.manager.get_collection(coll_name)
            if not collection:
                continue
            
            try:
                results = collection.get()
                for i, metadata in enumerate(results.get('metadatas', [])):
                    filename = metadata.get('filename', '')
                    if doc_name.lower() in filename.lower():
                        all_chunks.append({
                            'text': results['documents'][i],
                            'metadata': metadata,
                            'collection': coll_name
                        })
            except Exception as e:
                print(f"Error retrieving from {coll_name}: {e}")
        
        # Sort by chunk offset
        all_chunks.sort(key=lambda x: x['metadata'].get('start_offset', 0))
        return all_chunks
    
    def _generate_overview(self, doc_name: str, chunks: List[Dict]) -> str:
        """Generate document overview from representative samples."""
        if not chunks:
            return "No content available."
        
        # Sample beginning, middle, end for coverage
        total = len(chunks)
        if total <= 10:
            samples = chunks
        else:
            samples = chunks[:3] + chunks[total//3:total//3+3] + chunks[-3:]
        
        sample_text = " ".join([c['text'] for c in samples])[:5000]
        
        prompt = f"""Provide a 5-6 sentence overview of this document.

Document: {doc_name}
Content: {sample_text}

Requirements:
- Be factual and specific
- Cover main topics and scope
- Use information from content only
- No subjective interpretation

Overview:"""
        
        try:
            return self.llm_engine.generate_response(prompt, temperature=0.1, max_tokens=500).strip()
        except:
            return f"Document contains {len(chunks)} sections covering multiple topics."
    
    def _extract_claims(self, chunks: List[Dict], max_claims: int = 15) -> List[Dict]:
        """Extract key claims from document chunks."""
        claims = []
        for i, chunk in enumerate(chunks[:max_claims]):
            text = chunk['text']
            
            # Extend context if chunk is short
            if len(text) < 500 and i + 1 < len(chunks):
                text += " " + chunks[i + 1]['text'][:500]
            
            claims.append({
                'id': f"claim_{i+1}",
                'text': text[:3000],
                'metadata': chunk['metadata'],
                'chunk_index': i
            })
        
        return claims
    
    def _retrieve_references(self, claim_text: str, ref_collection: str, top_k: int) -> List[Dict]:
        """Get relevant reference passages for comparison."""
        try:
            if self.hybrid_search:
                return self.hybrid_search.search(claim_text, ref_collection, top_k)
            
            # Fallback to semantic search
            collection = self.manager.get_collection(ref_collection)
            if not collection:
                return []
            
            query_emb = self.embedding_model.encode([claim_text]).tolist()
            results = collection.query(query_embeddings=query_emb, n_results=top_k)
            
            return [{
                'text': results['documents'][0][i],
                'metadata': results['metadatas'][0][i],
                'score': 1.0 - results['distances'][0][i]
            } for i in range(len(results['documents'][0]))]
        
        except Exception as e:
            print(f"Reference retrieval error: {e}")
            return []
    
    def _check_alignment(self, claim: Dict, refs: List[Dict], 
                        target_doc: str, ref_collection: str) -> Dict[str, Any]:
        """Check if claim aligns with or conflicts with references."""
        if not refs:
            return {
                'has_misalignment': True,
                'claim_id': claim['id'],
                'claim_text': claim['text'][:1000],
                'misalignment_type': 'no_reference_found',
                'explanation': 'No relevant reference material found.',
                'severity': 'medium',
                'references': []
            }
        
        # Build reference context
        ref_context = "\n\n".join([
            f"Ref {i+1} ({ref['metadata'].get('filename', 'Unknown')}, "
            f"Page {ref['metadata'].get('page_num', 'N/A')}): {ref['text'][:800]}"
            for i, ref in enumerate(refs[:3])
        ])
        
        # LLM analysis
        prompt = f"""Compare target statement against reference regulations.

TARGET: {target_doc}
STATEMENT: {claim['text'][:2000]}

REFERENCES ({ref_collection}):
{ref_context}

Task: Identify MISALIGNMENTS or CONFLICTS only. If aligned, respond "ALIGNED".

Requirements:
- Base analysis ONLY on provided references
- Quote specific phrases from both target and references
- Focus on substance, not just wording
- State clearly if insufficient context

Format:
STATUS: [ALIGNED or MISALIGNED]
SEVERITY: [high/medium/low - if MISALIGNED]
EXPLANATION: [Detailed if MISALIGNED, else "No misalignment detected."]

Response:"""
        
        try:
            response = self.llm_engine.generate_response(prompt, temperature=0.1)
            
            if 'status: aligned' in response.lower():
                return {'has_misalignment': False}
            
            # Extract severity
            severity = 'medium'
            if 'severity: high' in response.lower():
                severity = 'high'
            elif 'severity: low' in response.lower():
                severity = 'low'
            
            # Extract explanation
            explanation_match = re.search(r'explanation[:\s]+(.*)', response, re.IGNORECASE | re.DOTALL)
            explanation = explanation_match.group(1).strip() if explanation_match else response.strip()
            
            # Format references
            formatted_refs = [{
                'filename': ref['metadata'].get('filename', 'Unknown'),
                'page_num': ref['metadata'].get('page_num', 'N/A'),
                'collection': ref_collection,
                'text_snippet': ref['text'][:600]
            } for ref in refs[:3]]
            
            return {
                'has_misalignment': True,
                'claim_id': claim['id'],
                'claim_text': claim['text'][:1000],
                'claim_source': {
                    'filename': target_doc,
                    'page_num': claim['metadata'].get('page_num', 'N/A'),
                    'chunk_id': claim['metadata'].get('chunk_id', 'N/A')
                },
                'misalignment_type': 'regulatory_conflict',
                'severity': severity,
                'explanation': explanation,
                'references': formatted_refs
            }
        
        except Exception as e:
            print(f"Alignment analysis error: {e}")
            return {'has_misalignment': False}
    
    def _calculate_confidence(self, misalignments: List[Dict], total_claims: int) -> float:
        """Calculate overall analysis confidence."""
        if total_claims == 0:
            return 0.0
        
        coverage_score = min(total_claims / 15, 1.0)
        detection_score = 0.7 if misalignments else 0.5
        
        if misalignments:
            severity_map = {'high': 0.9, 'medium': 0.7, 'low': 0.6}
            avg_severity = np.mean([severity_map.get(m.get('severity', 'medium'), 0.7) 
                                   for m in misalignments])
        else:
            avg_severity = 0.8
        
        return round(coverage_score * 0.3 + detection_score * 0.3 + avg_severity * 0.4, 3)
    
    def _build_report(self, target_doc: str, ref_collection: str, overview: str,
                     misalignments: List[Dict], confidence: float, 
                     total_claims: int) -> Dict[str, Any]:
        """Build final comparison report."""
        report = {
            'title': f"Cross-Reference Comparison: {target_doc} vs {ref_collection}",
            'target_document': target_doc,
            'reference_collection': ref_collection,
            'overview': overview,
            'analysis_summary': {
                'total_sections_analyzed': total_claims,
                'misalignments_found': len(misalignments),
                'confidence_score': confidence,
                'confidence_level': 'High' if confidence > 0.75 else 'Medium' if confidence > 0.5 else 'Low'
            }
        }
        
        if misalignments:
            report['results'] = [{
                'section': f"Misalignment #{i+1}",
                'severity': m.get('severity', 'medium').upper(),
                'target_location': m['claim_source'],
                'target_statement': m['claim_text'],
                'explanation': m['explanation'],
                'reference_citations': m.get('references', [])
            } for i, m in enumerate(misalignments)]
        else:
            report['results'] = [{
                'section': 'Overall Assessment',
                'message': 'No areas of misalignment detected',
                'details': f'Analyzed {total_claims} sections. All content aligns with {ref_collection}.'
            }]
        
        return report
    
    def _map_collection(self, reference_text: str) -> str:
        """Map natural language reference to collection name."""
        mappings = {
            'pra': 'pra_rules',
            'rules': 'pra_rules',
            'regulations': 'pra_rules',
            'earnings': 'earnings_transcripts',
            'transcript': 'earnings_transcripts'
        }
        
        ref_lower = reference_text.lower()
        for key, collection in mappings.items():
            if key in ref_lower:
                return collection
        
        return reference_text

print("✅ Cross-Reference Comparison System initialized!")
print("   • Document-to-collection comparison")
print("   • Misalignment detection with severity scoring")
print("   • Query intent detection")
print("   • Confidence-scored reports")

✅ Cross-Reference Comparison System initialized!
   • Document-to-collection comparison
   • Misalignment detection with severity scoring
   • Query intent detection
   • Confidence-scored reports


#### 6.1 Enhanced Metadata Search and Summarization Systems

In [51]:
# Advanced Hybrid Search Engine with BM25 Index Management
class AdvancedHybridSearchEngine:
    """Enhanced hybrid search with automatic BM25 index loading"""
    
    def __init__(self, collections_manager):
        self.manager = collections_manager
        self.fusion_engine = RRFFusionEngine()
        self.loaded_collections = set()
    
    def ensure_bm25_indices_loaded(self, collection_names: List[str]):
        """Ensure BM25 indices are loaded for specified collections"""
        for col_name in collection_names:
            if col_name not in self.loaded_collections:
                # Try to load existing index
                if not self.manager.load_bm25_index(col_name):
                    # Build new index if load fails
                    print(f"Building BM25 index for {col_name}...")
                    self.manager.build_bm25_index(col_name)
                self.loaded_collections.add(col_name)
    
    def multi_collection_search(self, query: str, collection_names: List[str], 
                               top_k: int = 10) -> List[Dict]:
        """Search across multiple collections with fusion"""
        # Ensure indices loaded
        self.ensure_bm25_indices_loaded(collection_names)
        
        all_results = []
        expanded_k = int(top_k * 2.5)
        
        for col_name in collection_names:
            # Get BM25 results
            bm25_results = self.manager.search_bm25(query, col_name, expanded_k)
            
            # Get semantic results
            semantic_results = self.manager.semantic_search(query, col_name, expanded_k)
            
            # Fuse results
            if bm25_results or semantic_results:
                fused = self.fusion_engine.fuse(semantic_results, bm25_results)
                
                # Tag with source collection
                for result in fused:
                    result['source_collection'] = col_name
                
                all_results.extend(fused)
        
        # Re-sort all results
        all_results.sort(key=lambda x: x.get('final_score', 0), reverse=True)
        
        return all_results[:top_k * len(collection_names)]

# Add this cell before Section 6.4.4 (where SpecCompliantRAGSystem is defined)

# ============================================================================
# Enhanced Metadata Search System
# ============================================================================

class EnhancedMetadataSearch:
    """Handle metadata queries about files and collections"""
    
    def __init__(self, collections_manager, llm_engine):
        self.manager = collections_manager
        self.llm_engine = llm_engine
    
    def execute_metadata_query(self, query: str, collections: List[str]) -> Dict[str, Any]:
        """Execute metadata query and return formatted results"""
        query_lower = query.lower()
        
        # Detect query type
        if any(word in query_lower for word in ['count', 'how many']):
            return self._count_files(query, collections)
        elif any(word in query_lower for word in ['list', 'show', 'display']):
            return self._list_files(query, collections)
        else:
            return self._general_metadata_query(query, collections)
    
    def _count_files(self, query: str, collections: List[str]) -> Dict[str, Any]:
        """Count files matching criteria"""
        results = []
        total_files = 0
        
        for col_name in collections:
            collection = self.manager.get_collection(col_name)
            if not collection:
                continue
            
            try:
                data = collection.get(include=['metadatas'])
                filenames = set()
                
                for metadata in data.get('metadatas', []):
                    if metadata and metadata.get('filename'):
                        filenames.add(metadata['filename'])
                
                count = len(filenames)
                total_files += count
                results.append(f"• {col_name}: {count} files")
                
            except Exception as e:
                results.append(f"• {col_name}: Error ({str(e)[:50]})")
        
        answer = f"Found {total_files} unique files across {len(collections)} collection(s):\n\n" + "\n".join(results)
        
        return {
            'query': query,
            'search_type': 'Metadata Count Query',
            'answer': answer,
            'references': [],
            'confidence': 1.0,
            'collections_searched': collections
        }
    
    def _list_files(self, query: str, collections: List[str]) -> Dict[str, Any]:
        """List files in collections"""
        results = []
        
        for col_name in collections:
            collection = self.manager.get_collection(col_name)
            if not collection:
                continue
            
            try:
                data = collection.get(include=['metadatas'])
                filenames = sorted(set(
                    m.get('filename') for m in data.get('metadatas', [])
                    if m and m.get('filename')
                ))
                
                if filenames:
                    results.append(f"\n{col_name} ({len(filenames)} files):")
                    for i, filename in enumerate(filenames, 1):
                        results.append(f"  {i}. {filename}")
                
            except Exception as e:
                results.append(f"\n{col_name}: Error - {str(e)[:50]}")
        
        answer = "Files in selected collections:\n" + "\n".join(results) if results else "No files found"
        
        return {
            'query': query,
            'search_type': 'File List Query',
            'answer': answer,
            'references': [],
            'confidence': 1.0,
            'collections_searched': collections
        }
    
    def _general_metadata_query(self, query: str, collections: List[str]) -> Dict[str, Any]:
        """Handle general metadata questions"""
        # Gather metadata statistics
        stats = []
        
        for col_name in collections:
            collection = self.manager.get_collection(col_name)
            if not collection:
                continue
            
            try:
                data = collection.get(include=['metadatas'])
                metadatas = data.get('metadatas', [])
                
                filenames = set(m.get('filename') for m in metadatas if m and m.get('filename'))
                doc_types = set(m.get('doc_type') for m in metadatas if m and m.get('doc_type'))
                
                stats.append({
                    'collection': col_name,
                    'total_chunks': len(metadatas),
                    'unique_files': len(filenames),
                    'doc_types': list(doc_types)
                })
                
            except Exception as e:
                stats.append({'collection': col_name, 'error': str(e)})
        
        # Format answer
        answer_lines = ["Database Statistics:\n"]
        for stat in stats:
            if 'error' in stat:
                answer_lines.append(f"• {stat['collection']}: Error")
            else:
                answer_lines.append(f"• {stat['collection']}:")
                answer_lines.append(f"  - Files: {stat['unique_files']}")
                answer_lines.append(f"  - Chunks: {stat['total_chunks']}")
                answer_lines.append(f"  - Types: {', '.join(stat['doc_types'])}")
        
        return {
            'query': query,
            'search_type': 'Metadata Query',
            'answer': "\n".join(answer_lines),
            'references': [],
            'confidence': 1.0,
            'collections_searched': collections
        }


# ============================================================================
# Enhanced Summarizer System
# ============================================================================

class EnhancedSummarizer:
    """Generate focused summaries with citation tracking"""
    
    def __init__(self, llm_engine, citation_system):
        self.llm_engine = llm_engine
        self.citation_system = citation_system
    
    def generate_summary(self, search_results: List[Dict], query: str, 
                        word_count: int = 300) -> Dict[str, Any]:
        """Generate summary from search results with citations"""
        
        if not search_results:
            return {
                'summary': 'No content available to summarize.',
                'confidence': 0.0,
                'word_count': 0,
                'target_words': word_count,
                'citations_used': [],
                'sources_count': 0
            }
        
        # Generate citations
        citations = self.citation_system.generate_precise_citations(search_results[:10])
        
        # Build context from results
        context_parts = []
        for i, result in enumerate(search_results[:10]):
            if not result.get('text'):
                continue
            
            citation = citations[i] if i < len(citations) else {'id': f'cite_{i+1}'}
            filename = citation.get('filename', 'Unknown')
            page = citation.get('page_num', 'N/A')
            
            context_parts.append(
                f"[{citation['id']} - {filename}, Page {page}]:\n{result['text'][:800]}"
            )
        
        context = "\n\n".join(context_parts)[:5000]
        
        # Generate summary
        prompt = f"""Provide a {word_count}-word summary of the following content.

CONTENT:
{context}

QUERY FOCUS: {query}

REQUIREMENTS:
- Target length: {word_count} words (±20%)
- Use citation format: (cite_X)
- Focus on query-relevant information
- Use professional, clear language
- Cite sources for key facts

SUMMARY:"""
        
        try:
            summary = self.llm_engine.generate_response(prompt, temperature=0.1, max_tokens=word_count*2)
            
            # Calculate statistics
            actual_words = len(summary.split())
            confidence = min(actual_words / word_count, 1.0) if word_count > 0 else 0.5
            
            return {
                'summary': summary,
                'confidence': confidence,
                'word_count': actual_words,
                'target_words': word_count,
                'citations_used': citations,
                'sources_count': len(search_results),
                'query_focus': query
            }
            
        except Exception as e:
            return {
                'summary': f'Error generating summary: {str(e)}',
                'confidence': 0.0,
                'word_count': 0,
                'target_words': word_count,
                'citations_used': [],
                'sources_count': 0
            }


print("✅ EnhancedMetadataSearch and EnhancedSummarizer defined!")
print("   • Metadata queries (count, list, stats)")
print("   • Document summarization with citations")
print("   • Query-focused summaries")

print("✅ AdvancedHybridSearchEngine defined")

✅ EnhancedMetadataSearch and EnhancedSummarizer defined!
   • Metadata queries (count, list, stats)
   • Document summarization with citations
   • Query-focused summaries
✅ AdvancedHybridSearchEngine defined


#### 6.2 Reliability Enforcement System

In [52]:
# Reliability Enforcement System

class ReliabilityEnforcer:
    """Validate response grounding using semantic similarity"""

    def __init__(self, llm_engine):
        self.llm_engine = llm_engine

    def validate_response_grounding(self, query: str, answer: str, 
                                   references: List[Dict]) -> Dict[str, Any]:
        """Validate that response is grounded in source materials"""

        if not references:
            return {
                'is_grounded': False,
                'grounding_score': 0.0,
                'issues': ['No source references provided'],
                'confidence': 0.0,
                'total_claims_analyzed': 0
            }

        # Extract claims
        claims = self._extract_key_claims(answer)
        if not claims:
            return {
                'is_grounded': True,
                'grounding_score': 1.0,
                'issues': [],
                'confidence': 1.0,
                'total_claims_analyzed': 0
            }

        # Check grounding for each claim
        grounding_scores = []
        issues = []

        for claim in claims:
            result = self._check_claim_grounding(claim, references)
            score = result['grounding_score']
            grounding_scores.append(score)

            if score < 0.5:
                issues.append(f"Weak grounding: {claim[:60]}...")

        # Calculate overall score
        overall_score = np.mean(grounding_scores) if grounding_scores else 0.0
        is_grounded = overall_score >= 0.6 and len(issues) == 0

        return {
            'is_grounded': is_grounded,
            'grounding_score': overall_score,
            'issues': issues,
            'confidence': min(overall_score, 1.0),
            'total_claims_analyzed': len(claims)
        }

    def _extract_key_claims(self, answer: str) -> List[str]:
        """Extract key factual claims from answer"""
        # Split on sentence boundaries with lookbehind/lookahead
        sentences = re.split(r'(?<=[.!?])\s+(?=[A-Z])', answer)

        claims = []
        transition_phrases = ('however', 'therefore', 'moreover', 'furthermore', 
                            'additionally', 'in conclusion', 'as a result')
        
        for sentence in sentences:
            sentence = sentence.strip()
            if (len(sentence) > 25 and 
                not sentence.lower().startswith(transition_phrases)):
                claims.append(sentence)

        return claims[:5]  # Limit to top 5 for performance

    def _check_claim_grounding(self, claim: str, references: List[Dict]) -> Dict[str, Any]:
        """Check if claim is grounded using semantic similarity"""
        
        # Try semantic similarity first
        if 'embedding_model' in globals():
            try:
                return self._semantic_grounding(claim, references)
            except Exception:
                pass  # Fall through to word overlap
        
        # Fallback: word overlap
        return self._word_overlap_grounding(claim, references)
    
    def _semantic_grounding(self, claim: str, references: List[Dict]) -> Dict[str, Any]:
        """Semantic similarity-based grounding check"""
        claim_emb = embedding_model.encode([claim])[0]
        
        max_similarity = 0.0
        best_ref = None
        
        for ref in references:
            if 'text' not in ref or not ref.get('text'):
                continue
            
            ref_emb = embedding_model.encode([ref['text'][:500]])[0]
            similarity = np.dot(claim_emb, ref_emb) / (
                np.linalg.norm(claim_emb) * np.linalg.norm(ref_emb)
            )
            
            if similarity > max_similarity:
                max_similarity = similarity
                best_ref = ref
        
        return {
            'claim': claim,
            'grounding_score': float(max_similarity),
            'best_reference': best_ref.get('metadata', {}).get('filename', 'Unknown') if best_ref else None
        }
    
    def _word_overlap_grounding(self, claim: str, references: List[Dict]) -> Dict[str, Any]:
        """Fallback word overlap grounding check"""
        claim_words = set(claim.lower().split())
        
        max_overlap = 0.0
        best_ref = None
        
        for ref in references:
            if 'text' not in ref or not ref.get('text'):
                continue
            
            ref_words = set(ref['text'].lower().split())
            overlap = len(claim_words.intersection(ref_words))
            overlap_ratio = overlap / len(claim_words) if claim_words else 0
            
            if overlap_ratio > max_overlap:
                max_overlap = overlap_ratio
                best_ref = ref
        
        return {
            'claim': claim,
            'grounding_score': max_overlap,
            'best_reference': best_ref.get('metadata', {}).get('filename', 'Unknown') if best_ref else None
        }

print("✅ Reliability Enforcement System initialized!")
print("   • Semantic similarity-based grounding validation")
print("   • Fallback to word overlap if needed")
print("   • Claim extraction with transition phrase filtering")

✅ Reliability Enforcement System initialized!
   • Semantic similarity-based grounding validation
   • Fallback to word overlap if needed
   • Claim extraction with transition phrase filtering


#### 6.3 Enhanced Multi-Collection Selector UI Component

In [53]:
# Section 6.3: Enhanced Multi-Collection Selector UI Component

# ============================================================================
# 1. Advanced Collection Selector (KEEP - under 100 lines)
# ============================================================================

class AdvancedCollectionSelector:
    """Advanced collection selector with file counts and metadata"""

    def __init__(self, collections_manager):
        self.manager = collections_manager
        self.selected_collections = []

    def create_selector_widget(self):
        """Create advanced collection selector with real-time stats"""
        available_collections = self.manager.get_available_collections()
        collection_checkboxes = {}
        collection_info = {}

        for collection_name in available_collections:
            try:
                collection = self.manager.get_collection(collection_name)
                count = collection.count()

                # Get sample filenames
                data = collection.get(include=['metadatas'], limit=5)
                filenames = set()
                for metadata in data.get('metadatas', []):
                    if metadata and metadata.get('filename'):
                        filenames.add(metadata['filename'])

                info_text = f"{collection_name} ({count:,} chunks, {len(filenames)} files)"
                collection_info[collection_name] = {
                    'count': count,
                    'files': len(filenames),
                    'sample_files': list(filenames)[:3]
                }

            except Exception as e:
                info_text = f"{collection_name} (error: {str(e)[:50]})"
                collection_info[collection_name] = {'error': str(e)}

            checkbox = widgets.Checkbox(
                value=False,
                description=info_text,
                style={'description_width': 'initial'},
                layout=widgets.Layout(width='100%')
            )
            collection_checkboxes[collection_name] = checkbox

        selection_summary = widgets.HTML(value="<p><em>No collections selected</em></p>")

        def update_selection(*args):
            selected = [name for name, cb in collection_checkboxes.items() if cb.value]
            self.selected_collections = selected

            if selected:
                total_chunks = sum(collection_info[name].get('count', 0) for name in selected)
                total_files = sum(collection_info[name].get('files', 0) for name in selected)

                summary_html = f"""
                <div style='background: #f0f9ff; padding: 10px; border-radius: 5px; border: 1px solid #0ea5e9;'>
                    <strong>Selected Collections ({len(selected)}):</strong><br/>
                    • Total chunks: {total_chunks:,}<br/>
                    • Total files: {total_files}<br/>
                    • Collections: {', '.join(selected)}
                </div>
                """
            else:
                summary_html = "<p><em>No collections selected</em></p>"

            selection_summary.value = summary_html

        for cb in collection_checkboxes.values():
            cb.observe(update_selection, names='value')

        selector_ui = widgets.VBox([
            widgets.HTML("<h4>📂 Select Collections for RAG Queries:</h4>"),
            widgets.VBox(list(collection_checkboxes.values())),
            selection_summary
        ])

        return selector_ui, collection_checkboxes


# ============================================================================
# 2. Progress Tracker (KEEP - under 100 lines)
# ============================================================================

class RAGProgressTracker:
    """Real-time progress tracking for RAG operations"""

    def __init__(self):
        self.progress_bar = widgets.FloatProgress(
            value=0, min=0, max=100,
            description='Progress:',
            bar_style='info',
            layout=widgets.Layout(width='100%')
        )

        self.status_label = widgets.HTML(value="Ready")

        self.cancel_flag = threading.Event()
        self.cancel_button = widgets.Button(
            description='Cancel',
            button_style='danger',
            disabled=True,
            layout=widgets.Layout(width='100px')
        )

        self.cancel_button.on_click(self._cancel_operation)

    def _cancel_operation(self, button):
        """Cancel current operation"""
        self.cancel_flag.set()
        self.update_status("Cancelling...", 0)
        button.disabled = True

    def start_operation(self, operation_name: str):
        """Start tracking an operation"""
        self.cancel_flag.clear()
        self.cancel_button.disabled = False
        self.update_status(f"Starting {operation_name}...", 10)

    def update_progress(self, progress: float, message: str = ""):
        """Update progress (0-100)"""
        self.progress_bar.value = progress
        if message:
            self.status_label.value = f"<span style='color: #0066cc;'>{message}</span>"

    def update_status(self, message: str, progress: float = None):
        """Update status message"""
        self.status_label.value = f"<span style='color: #0066cc;'>{message}</span>"
        if progress is not None:
            self.progress_bar.value = progress

    def complete_operation(self, message: str = "Operation completed"):
        """Mark operation as complete"""
        self.progress_bar.value = 100
        self.cancel_button.disabled = True
        self.status_label.value = f"<span style='color: #008800;'>✅ {message}</span>"

    def error_operation(self, message: str = "Operation failed"):
        """Mark operation as failed"""
        self.cancel_button.disabled = True
        self.status_label.value = f"<span style='color: #cc0000;'>❌ {message}</span>"

    def get_widgets(self):
        """Get progress widgets for display"""
        return widgets.VBox([
            self.progress_bar,
            widgets.HBox([self.status_label, self.cancel_button])
        ])


# ============================================================================
# 3. Results Formatter (NEW - pure formatting logic)
# ============================================================================

class ResultsFormatter:
    """Format RAG results as text (no widget logic)"""
    
    @staticmethod
    def format_page_info(ref: Dict) -> str:
        """Format page number/range for display"""
        page = ref.get('page', 'N/A')
        page_range = ref.get('page_range', '')
        
        if page_range:
            return f" 📄 {page_range}"
        elif page != 'N/A':
            return f" 📄 Page {page}"
        return ""
    
    @staticmethod
    def format_scores(ref: Dict) -> str:
        """Format confidence and relevance scores"""
        scores = []
        if 'confidence_score' in ref:
            scores.append(f"Confidence: {ref['confidence_score']:.1%}")
        if 'relevance_score' in ref:
            scores.append(f"Relevance: {ref['relevance_score']:.3f}")
        elif 'scores' in ref and ref['scores'].get('final_score'):
            scores.append(f"Score: {ref['scores']['final_score']:.3f}")
        
        return f" ({', '.join(scores)})" if scores else ""
    
    @staticmethod
    def format_reference(ref: Dict, index: int) -> str:
        """Format single reference with all details"""
        filename = ref.get('filename', 'Unknown')
        collection = ref.get('collection', 'Unknown')
        citation_id = ref.get('citation_id', f'cite_{index}')
        
        # Build reference line with page info
        page_info = ResultsFormatter.format_page_info(ref)
        line = f"{index}. [{citation_id}]{page_info} - {filename}"
        
        # Add collection and scores on next line
        if collection != 'Unknown':
            scores = ResultsFormatter.format_scores(ref)
            line += f"\n   Collection: {collection}{scores}"
        
        # Add snippet if available
        snippet = ref.get('text_snippet', '')
        if snippet:
            line += f"\n   └─ {snippet}"
        
        return line
    
    @staticmethod
    def format_result(result: Dict) -> str:
        """Format complete RAG result as text"""
        lines = []
        
        # Answer section
        lines.append("🎯 ANSWER")
        lines.append("=" * 80)
        lines.append(result.get('answer', 'No answer generated'))
        lines.append("")
        
        # Confidence & reliability
        confidence = result.get('confidence', 0)
        lines.append(f"📊 CONFIDENCE: {confidence:.1%}")
        
        reliability = result.get('reliability_validation', {})
        if reliability:
            grounding_score = reliability.get('grounding_score', 0)
            lines.append(f"🔗 GROUNDING: {grounding_score:.1%}")
            if not reliability.get('is_grounded', True):
                lines.append("⚠️  WARNING: Response may not be fully grounded in sources")
        lines.append("")
        
        # Search metadata
        search_type = result.get('search_type', 'Unknown')
        collections = result.get('collections_searched', [])
        lines.append(f"🔍 SEARCH TYPE: {search_type}")
        lines.append(f"📂 COLLECTIONS: {', '.join(collections)}")
        
        # Processing stats (if available)
        stats = result.get('processing_stats', {})
        if stats:
            total = stats.get('total_found', 0)
            processed = stats.get('total_processed', 0)
            lines.append(f"📈 RESULTS: {processed} processed from {total} found")
            
            dist = stats.get('confidence_distribution', {})
            if dist:
                lines.append(f"🎯 CONFIDENCE DIST: High: {dist.get('high', 0)}, "
                           f"Medium: {dist.get('medium', 0)}, Low: {dist.get('low', 0)}")
        
        lines.append("")
        
        # References
        references = result.get('references', [])
        if references:
            lines.append(f"📎 SOURCES ({len(references)}):")
            lines.append("=" * 50)
            
            for i, ref in enumerate(references[:15], 1):
                lines.append(ResultsFormatter.format_reference(ref, i))
                lines.append("")
        
        # Summary stats (if available)
        summary_stats = result.get('summary_stats', {})
        if summary_stats:
            lines.append("📝 SUMMARY STATS:")
            actual = summary_stats.get('actual_words', 0)
            target = summary_stats.get('target_words', 0)
            lines.append(f"   Words: {actual}/{target}")
            lines.append(f"   Sources: {summary_stats.get('sources_used', 0)}")
            
            if summary_stats.get('unique_pages'):
                lines.append(f"   Unique Pages: {summary_stats['unique_pages']}")
            if summary_stats.get('query_focus'):
                lines.append(f"   Query Focus: {summary_stats['query_focus']}")
        
        return "\n".join(lines)

# ============================================================================
# 3.5. Response Formatter (NEW - for structured responses)
# ============================================================================

class ResponseFormatter:
    """Format RAG responses with consistent structure"""
    
    @staticmethod
    def format_search_response(query: str, answer: str, references: List[Dict], 
                               confidence: float, stats: Dict) -> Dict[str, Any]:
        """Format standard search response"""
        return {
            'query': query,
            'search_type': 'Enhanced Search',
            'answer': answer,
            'references': references,
            'confidence': confidence,
            'processing_stats': stats,
            'collections_searched': stats.get('collections', [])
        }
    
    @staticmethod
    def format_comparison_report(report: Dict) -> str:
        """Format cross-reference comparison report as readable text"""
        lines = []
        
        # Title and overview
        lines.append(f"📋 {report.get('title', 'Comparison Report')}")
        lines.append("=" * 80)
        lines.append("")
        
        # Document overview
        if report.get('overview'):
            lines.append("📄 DOCUMENT OVERVIEW:")
            lines.append(report['overview'])
            lines.append("")
        
        # Analysis summary
        summary = report.get('analysis_summary', {})
        if summary:
            lines.append("📊 ANALYSIS SUMMARY:")
            lines.append(f"   • Sections analyzed: {summary.get('total_sections_analyzed', 0)}")
            lines.append(f"   • Misalignments found: {summary.get('misalignments_found', 0)}")
            lines.append(f"   • Confidence: {summary.get('confidence_score', 0):.1%} ({summary.get('confidence_level', 'Unknown')})")
            lines.append("")
        
        # Results
        results = report.get('results', [])
        if results:
            lines.append("🔍 FINDINGS:")
            lines.append("-" * 80)
            
            for i, result in enumerate(results, 1):
                section = result.get('section', f'Finding {i}')
                lines.append(f"\n{i}. {section}")
                
                if result.get('severity'):
                    lines.append(f"   Severity: {result['severity']}")
                
                if result.get('explanation'):
                    lines.append(f"   {result['explanation']}")
                
                if result.get('target_statement'):
                    stmt = result['target_statement'][:300]
                    lines.append(f"   Target: {stmt}{'...' if len(result['target_statement']) > 300 else ''}")
                
                # Reference citations
                refs = result.get('reference_citations', [])
                if refs:
                    lines.append(f"   References ({len(refs)}):")
                    for ref in refs[:3]:
                        fn = ref.get('filename', 'Unknown')
                        pg = ref.get('page_num', 'N/A')
                        lines.append(f"     • {fn} (Page {pg})")
        
        return "\n".join(lines)



# ============================================================================
# 4. Enhanced Results Display (REFACTORED - widget management only)
# ============================================================================

class EnhancedResultsDisplay:
    """Widget manager for displaying RAG results"""
    
    def __init__(self):
        self.results_area = widgets.Output(
            layout=widgets.Layout(
                width='100%',
                border='1px solid #e2e8f0',
                padding='15px',
                border_radius='8px',
                background_color='#f8fafc',
                overflow='auto',
                max_height='600px'
            )
        )
        
        # Show placeholder
        with self.results_area:
            print("💡 Ready to search! Enter a query and click Search to see results here.")
    
    def display_results(self, result: Dict[str, Any]):
        """Display formatted results in widget"""
        formatted_text = ResultsFormatter.format_result(result)
        
        with self.results_area:
            clear_output(wait=False)
            print(formatted_text)
    
    def get_widget(self):
        """Get results display widget"""
        return self.results_area


print("✅ Additional RAG UI Components defined!")
print("   • AdvancedCollectionSelector: Multi-select with stats")
print("   • RAGProgressTracker: Progress tracking with cancellation")
print("   • ResultsFormatter: Reusable result formatting")
print("   • EnhancedResultsDisplay: Simplified widget manager")

✅ Additional RAG UI Components defined!
   • AdvancedCollectionSelector: Multi-select with stats
   • RAGProgressTracker: Progress tracking with cancellation
   • ResultsFormatter: Reusable result formatting
   • EnhancedResultsDisplay: Simplified widget manager


#### 6.4.1 Main Enhanced RAG System

In [54]:
# LLM-Powered Query Router 
class LLMQueryRouter:
    """Use LLM to understand query intent - fails explicitly if LLM unavailable"""
    
    def __init__(self, llm_engine, collections_manager):
        self.llm = llm_engine
        self.manager = collections_manager
    
    def analyze_query(self, query: str, collections: List[str]) -> Dict[str, Any]:
        """Use LLM to analyze query - raises exception if LLM fails"""
        
        # Check LLM availability
        if not self.llm or not hasattr(self.llm, 'generate_response'):
            raise RuntimeError("LLM engine not available or not properly initialized")
        
        # Get available metadata context
        metadata_context = self._get_metadata_context(collections)
        
        prompt = f"""Analyze this query and output JSON only.

Available files: {metadata_context['sample_files'][:10]}
Available years: {metadata_context['years']}
Available companies: {metadata_context['companies'][:5]}

Query: "{query}"

Determine:
1. Intent: "database_query" (asking about files/database) OR "search" (asking about content) OR "summarization" OR "comparison"
2. If database_query: extract filters (filename, year, company, author)
3. If summarization: extract target document and word count
4. If comparison: extract target and reference documents

JSON format:
{{
  "intent": "database_query|search|summarization|comparison",
  "confidence": 0.0-1.0,
  "filters": {{"filename": "partial name or null", "year": "2020 or null", "company": "name or null"}},
  "target_document": "filename or null",
  "reference_document": "filename or null",
  "word_count": 300
}}

JSON:"""
        
        # Call LLM - let exceptions propagate
        response = self.llm.generate_response(prompt, temperature=0.1, max_tokens=200)
        result = self._parse_llm_response(response)
        
        # Apply metadata-based document retrieval if needed
        if result['intent'] == 'database_query' and result.get('filters'):
            result['matched_documents'] = self._retrieve_by_metadata(
                result['filters'], collections
            )
        
        return result
    
    def _get_metadata_context(self, collections: List[str]) -> Dict:
        """Extract available metadata for LLM context"""
        files, years, companies = set(), set(), set()
        
        for col_name in collections[:2]:  # Limit to avoid token overflow
            collection = self.manager.get_collection(col_name)
            if not collection:
                continue
            
            try:
                data = collection.get(include=['metadatas'], limit=50)
                for m in data.get('metadatas', []):
                    if m:
                        if m.get('filename'):
                            files.add(m['filename'])
                        if m.get('year'):
                            years.add(m['year'])
                        if m.get('company'):
                            companies.add(m['company'])
            except:
                pass
        
        return {
            'sample_files': sorted(files)[:10],
            'years': sorted(years, reverse=True)[:5],
            'companies': sorted(companies)[:5]
        }
    
    def _parse_llm_response(self, response: str) -> Dict:
        """Parse JSON from LLM response - raises ValueError if invalid"""
        import json
        match = re.search(r'\{[^{}]*(?:\{[^{}]*\}[^{}]*)*\}', response, re.DOTALL)
        if not match:
            raise ValueError(f"LLM did not return valid JSON. Response: {response[:200]}")
        
        try:
            data = json.loads(match.group(0))
        except json.JSONDecodeError as e:
            raise ValueError(f"LLM returned invalid JSON: {e}")
        
        return {
            'intent': data.get('intent', 'search'),
            'confidence': data.get('confidence', 0.5),
            'filters': data.get('filters', {}),
            'target_document': data.get('target_document'),
            'reference_document': data.get('reference_document'),
            'word_count': data.get('word_count', 300)
        }
    
    def _retrieve_by_metadata(self, filters: Dict, collections: List[str]) -> List[Dict]:
        """Retrieve documents using metadata filters - improved year matching"""
        matched = []
        
        for col_name in collections:
            collection = self.manager.get_collection(col_name)
            if not collection:
                continue
            
            try:
                data = collection.get(include=['metadatas', 'documents'])
                for i, metadata in enumerate(data.get('metadatas', [])):
                    if not metadata:
                        continue
                    
                    filename = metadata.get('filename', '')
                    
                    # Apply filename filter
                    if filters.get('filename'):
                        fname = filters['filename'].lower()
                        if fname not in filename.lower():
                            continue
                    
                    # Apply year filter - check metadata AND filename
                    if filters.get('year'):
                        year_str = str(filters['year'])
                        # Extract last 2 digits for short year (2022 -> 22)
                        year_short = year_str[2:] if len(year_str) == 4 else year_str
                        
                        # Check metadata year OR filename patterns
                        metadata_year = str(metadata.get('year', ''))
                        has_year = (
                            year_str in metadata_year or 
                            year_short in metadata_year or
                            year_str in filename or
                            f"_{year_short}" in filename or  # Match _22 in barclays_1Q22
                            f"{year_short}_" in filename or  # Match 22_ pattern
                            f"Q{year_short}" in filename     # Match Q22 pattern
                        )
                        
                        if not has_year:
                            continue
                    
                    # Apply company filter - check metadata AND filename
                    if filters.get('company'):
                        comp = filters['company'].lower()
                        company_meta = metadata.get('company', '').lower()
                        if comp not in company_meta and comp not in filename.lower():
                            continue
                    
                    # Document matches all filters
                    matched.append({
                        'filename': filename,
                        'collection': col_name,
                        'year': metadata.get('year'),
                        'company': metadata.get('company'),
                        'text': data['documents'][i][:300] if i < len(data.get('documents', [])) else ''
                    })
            except Exception as e:
                print(f"⚠️ Filter error in {col_name}: {e}")
                pass
        
        return matched


print("✅ LLM-Powered Query Router - No Fallbacks")
print("   • LLM required for all query analysis")
print("   • Explicit errors when LLM fails")
print("   • Metadata-based document retrieval with year extraction from filenames")


✅ LLM-Powered Query Router - No Fallbacks
   • LLM required for all query analysis
   • Explicit errors when LLM fails
   • Metadata-based document retrieval with year extraction from filenames


#### 6.4.2 Main Enhanced RAG System

In [55]:
# Query Router - LLM-Dependent (No Pattern Matching Fallback)
class QueryRouter:
    """Simple query router that delegates to LLM - no fallback logic"""
    
    def __init__(self, llm_router, cross_reference_comparator):
        self.llm_router = llm_router
        self.comparator = cross_reference_comparator
    
    def detect_query_type(self, query: str, collections: List[str] = None) -> Dict[str, Any]:
        """Detect query type using LLM - raises exception if LLM fails"""
        
        # Check cross-reference comparison first (uses pattern detection, not LLM)
        comparison_intent = self.comparator.detect_query_intent(query)
        if comparison_intent['is_comparison_query']:
            return {
                'type': 'cross_reference_comparison',
                'confidence': 0.9,
                **comparison_intent
            }
        
        # Use LLM for all other query analysis - no fallback
        if not collections:
            raise ValueError("Collections parameter required for LLM query analysis")
        
        # LLM analysis - exceptions will propagate
        analysis = self.llm_router.analyze_query(query, collections)
        return {'type': analysis['intent'], **analysis}


# Search Results Processor
class SearchResultsProcessor:
    """Process and enrich search results with statistics"""
    
    def __init__(self):
        self.stats = {
            'total_found': 0,
            'high_confidence': 0,
            'medium_confidence': 0,
            'low_confidence': 0,
            'both_methods': 0,
            'semantic_only': 0,
            'bm25_only': 0
        }
    
    def process_results(self, results: List[Dict], min_confidence: float = 0.3) -> List[Dict]:
        """Filter and enrich results, track statistics"""
        self.stats['total_found'] = len(results)
        processed = []
        
        for result in results:
            confidence = result.get('confidence', 0)
            
            # Track confidence levels
            if confidence >= 0.7:
                self.stats['high_confidence'] += 1
            elif confidence >= 0.5:
                self.stats['medium_confidence'] += 1
            else:
                self.stats['low_confidence'] += 1
            
            # Track search methods
            has_semantic = result.get('semantic_score', 0) > 0
            has_bm25 = result.get('bm25_score', 0) > 0
            
            if has_semantic and has_bm25:
                self.stats['both_methods'] += 1
            elif has_semantic:
                self.stats['semantic_only'] += 1
            elif has_bm25:
                self.stats['bm25_only'] += 1
            
            # Apply confidence filter
            if confidence >= min_confidence:
                result['confidence_level'] = self._get_confidence_level(confidence)
                processed.append(result)
        
        return processed
    
    def _get_confidence_level(self, confidence: float) -> str:
        if confidence >= 0.7:
            return 'high'
        elif confidence >= 0.5:
            return 'medium'
        return 'low'
    
    def get_stats(self) -> Dict:
        """Return accumulated statistics"""
        return {
            **self.stats,
            'total_processed': self.stats['high_confidence'] + 
                              self.stats['medium_confidence'] + 
                              self.stats['low_confidence'],
            'confidence_distribution': {
                'high': self.stats['high_confidence'],
                'medium': self.stats['medium_confidence'],
                'low': self.stats['low_confidence']
            }
        }


print("✅ Query Router - LLM-Dependent (No Fallbacks)")
print("   • All queries processed by LLM")
print("   • Explicit errors when LLM unavailable")
print("   • Cross-reference detection still uses patterns (not LLM-dependent)")


✅ Query Router - LLM-Dependent (No Fallbacks)
   • All queries processed by LLM
   • Explicit errors when LLM unavailable
   • Cross-reference detection still uses patterns (not LLM-dependent)


#### 6.4.3 Advanced Hybrid Search Engine with BM25 Index Management

In [56]:
# Advanced Hybrid Search Engine with BM25 Index Management
class AdvancedHybridSearchEngine:
    """Enhanced hybrid search with automatic BM25 index loading"""
    
    def __init__(self, collections_manager):
        self.manager = collections_manager
        self.fusion_engine = RRFFusionEngine()
        self.loaded_collections = set()
    
    def ensure_bm25_indices_loaded(self, collection_names: List[str]):
        """Ensure BM25 indices are loaded for specified collections"""
        for col_name in collection_names:
            if col_name not in self.loaded_collections:
                # Try to load existing index
                if not self.manager.load_bm25_index(col_name):
                    # Build new index if load fails
                    print(f"Building BM25 index for {col_name}...")
                    self.manager.build_bm25_index(col_name)
                self.loaded_collections.add(col_name)
    
    def multi_collection_search(self, query: str, collection_names: List[str], 
                               top_k: int = 10) -> List[Dict]:
        """Search across multiple collections with fusion"""
        # Ensure indices loaded
        self.ensure_bm25_indices_loaded(collection_names)
        
        all_results = []
        expanded_k = int(top_k * 2.5)
        
        for col_name in collection_names:
            # Get BM25 results
            bm25_results = self.manager.search_bm25(query, col_name, expanded_k)
            
            # Get semantic results
            semantic_results = self.manager.semantic_search(query, col_name, expanded_k)
            
            # Fuse results
            if bm25_results or semantic_results:
                fused = self.fusion_engine.fuse(semantic_results, bm25_results)
                
                # Tag with source collection
                for result in fused:
                    result['source_collection'] = col_name
                
                all_results.extend(fused)
        
        # Re-sort all results
        all_results.sort(key=lambda x: x.get('final_score', 0), reverse=True)
        
        return all_results[:top_k * len(collection_names)]

print("✅ AdvancedHybridSearchEngine defined")

✅ AdvancedHybridSearchEngine defined


#### 6.4.4 Main Enhanced RAG System

In [57]:
# Main Enhanced RAG System
class SpecCompliantRAGSystem:
    """Simplified RAG orchestrator - delegates to specialized components"""
    
    def __init__(self, collections_manager, llm_engine, embedding_model, cross_encoder=None):
        self.collections_manager = collections_manager
        self.llm_engine = llm_engine
        self.embedding_model = embedding_model
        self.cross_encoder = cross_encoder
        
        # Initialize hybrid search
        self.hybrid_search = self._init_hybrid_search()
        
        # Initialize LLM router for intelligent query processing
        self.llm_router = LLMQueryRouter(llm_engine, collections_manager)
        
        # Initialize specialized components
        self.citation_system = EnhancedCitationSystem(collections_manager)
        self.cross_doc_analyzer = CrossDocumentAnalyzer(llm_engine, collections_manager)
        self.metadata_search = EnhancedMetadataSearch(collections_manager, self.llm_router)
        self.enhanced_summarizer = EnhancedSummarizer(llm_engine, self.citation_system)
        self.reliability_enforcer = ReliabilityEnforcer(llm_engine)
        self.cross_reference_comparator = CrossReferenceComparator(
            llm_engine, collections_manager, embedding_model, self.hybrid_search
        )
        
        # Initialize helper utilities
        self.query_router = QueryRouter(self.llm_router, self.cross_reference_comparator)
        self.results_processor = SearchResultsProcessor()
        self.formatter = ResponseFormatter()
        
        self.enhanced_mode = True
    
    def _init_hybrid_search(self):
        """Initialize hybrid search engine"""
        try:
            return globals().get('enhanced_hybrid_search') or \
                   AdvancedHybridSearchEngine(self.collections_manager)
        except:
            return None
    
    def process_query(self, query: str, collections: List[str], 
                     use_reranking: bool = True) -> Dict[str, Any]:
        """Main query processing with intelligent routing"""
        
        if not collections:
            return self._error_response(query, 'No collections selected', collections)
        
        try:
            # Detect query type and route (pass collections for LLM context)
            analysis = self.query_router.detect_query_type(query, collections)
            
            # Route to appropriate handler
            if analysis['type'] == 'database_query':
                result = self.metadata_search.execute_metadata_query(query, collections)
            
            elif analysis['type'] == 'cross_reference_comparison':
                result = self._handle_comparison(query, collections, analysis)
            
            elif analysis['type'] == 'summarization':
                result = self._handle_summarization(query, collections, analysis)
            
            else:
                result = self._handle_search(query, collections, use_reranking)
            
            # Apply reliability validation (skip for metadata/summaries)
            if result.get('search_type') not in [
                'Metadata Count Query', 'File List Query', 
                'Enhanced Document Summarization', 'Cross-Reference Comparison'
            ]:
                validation = self.reliability_enforcer.validate_response_grounding(
                    query, result.get('answer', ''), result.get('references', [])
                )
                result['reliability_validation'] = validation
                
                if not validation['is_grounded']:
                    result['confidence'] *= 0.5
                    result['answer'] = "⚠️ RELIABILITY WARNING: Response may not be fully grounded.\n\n" + \
                                      result['answer']
            
            # Add metadata
            result['enhanced_mode'] = self.enhanced_mode
            result['processing_timestamp'] = time.strftime('%Y-%m-%d %H:%M:%S')
            
            return result
        
        except Exception as e:
            return self._error_response(query, str(e), collections)
    
    def _handle_search(self, query: str, collections: List[str], 
                      use_reranking: bool) -> Dict[str, Any]:
        """Handle standard hybrid search with reranking"""
        
        if not self.hybrid_search:
            return self._error_response(query, 'Hybrid search not available', collections)
        
        # Ensure BM25 indices loaded
        self.hybrid_search.ensure_bm25_indices_loaded(collections)
        
        # Execute search
        search_results = self.hybrid_search.multi_collection_search(query, collections, 15)
        
        if not search_results:
            return {
                'query': query,
                'search_type': 'Enhanced Search',
                'answer': 'No relevant information found.',
                'references': [],
                'confidence': 0.0,
                'collections_searched': collections
            }
        
        # Process results with statistics
        processed_results = self.results_processor.process_results(search_results)
        
        # Apply reranking if requested
        if use_reranking and self.cross_encoder and len(processed_results) > 1:
            try:
                processed_results = self.cross_encoder.rerank(query, processed_results, 
                                                             len(processed_results))
            except Exception as e:
                print(f"Reranking failed: {e}")
        
        # Limit to top 10
        final_results = processed_results[:10]
        
        # Generate citations and response
        citations = self.citation_system.generate_precise_citations(final_results)
        cross_analysis = self.cross_doc_analyzer.analyze_cross_document_patterns(
            final_results, query
        )
        answer = self._generate_response(query, final_results, citations)
        
        # Build references
        references = self._build_references(final_results, citations)
        
        # Calculate confidence
        confidence = np.mean([r.get('confidence', 0) for r in final_results[:5]]) \
                     if final_results else 0.0
        
        return self.formatter.format_search_response(
            query, answer, references, confidence, 
            self.results_processor.get_stats()
        )
    
    def _handle_summarization(self, query: str, collections: List[str], 
                             analysis: Dict) -> Dict[str, Any]:
        """Handle document summarization"""
        
        filename = analysis.get('filename')
        word_count = analysis.get('word_count', 300)
        
        if filename:
            # Specific document summary
            doc_finder = DocumentFinder(self.collections_manager)
            documents = doc_finder.find_documents_by_name(filename, collections)
            
            if not documents:
                return {
                    'query': query,
                    'search_type': 'Document Summarization',
                    'answer': f'Document "{filename}" not found.',
                    'references': [],
                    'confidence': 0.0,
                    'collections_searched': collections
                }
            
            search_results = documents
        else:
            # Topic summary
            if not self.hybrid_search:
                return self._error_response(query, 'Hybrid search not available', collections)
            
            topic_query = re.sub(r'\bsummariz\w*\b|\b\d+\s*words?\b', '', query, 
                                flags=re.IGNORECASE).strip() or "main topics"
            search_results = self.hybrid_search.multi_collection_search(topic_query, collections, 12)
        
        if not search_results:
            return {
                'query': query,
                'search_type': 'Summarization',
                'answer': 'No content found to summarize.',
                'references': [],
                'confidence': 0.0,
                'collections_searched': collections
            }
        
        # Generate summary
        summary_result = self.enhanced_summarizer.generate_summary(
            search_results, query, word_count
        )
        
        # Build references (single document with page range)
        references = self._build_summary_reference(search_results, summary_result, collections)
        
        return {
            'query': query,
            'search_type': 'Enhanced Document Summarization',
            'answer': summary_result['summary'],
            'references': references,
            'confidence': summary_result['confidence'],
            'collections_searched': collections,
            'summary_stats': self._build_summary_stats(summary_result, references)
        }
    
    def _handle_comparison(self, query: str, collections: List[str], 
                          analysis: Dict) -> Dict[str, Any]:
        """Handle cross-reference comparison"""
        
        target_doc = analysis.get('target_document')
        reference_collection = analysis.get('reference_collection')
        
        if not target_doc or not reference_collection:
            return {
                'query': query,
                'search_type': 'Cross-Reference Comparison',
                'answer': 'Please specify target document and reference collection.',
                'references': [],
                'confidence': 0.0,
                'collections_searched': collections,
                'error': 'Missing target or reference'
            }
        
        # Perform comparison
        comparison_report = self.cross_reference_comparator.compare_documents(
            target_doc=target_doc,
            reference_collection=reference_collection,
            target_collection=collections[0] if collections else None
        )
        
        if comparison_report.get('status') == 'error':
            return {
                'query': query,
                'search_type': 'Cross-Reference Comparison',
                'answer': f"Error: {comparison_report.get('message')}",
                'references': [],
                'confidence': 0.0,
                'collections_searched': collections
            }
        
        # Format report
        answer = self.formatter.format_comparison_report(comparison_report)
        
        # Extract references
        references = []
        for result in comparison_report.get('results', []):
            for ref_cite in result.get('reference_citations', []):
                references.append({
                    'filename': ref_cite.get('filename', 'Unknown'),
                    'page_num': ref_cite.get('page', 'N/A'),
                    'collection': ref_cite.get('collection', reference_collection),
                    'text': ref_cite.get('excerpt', '')
                })
        
        return {
            'query': query,
            'search_type': 'Cross-Reference Comparison',
            'answer': answer,
            'references': references,
            'confidence': comparison_report.get('analysis_summary', {}).get('confidence_score', 0.0),
            'collections_searched': collections,
            'comparison_report': comparison_report
        }
    
    def _generate_response(self, query: str, results: List[Dict], 
                          citations: List[Dict]) -> str:
        """Generate LLM response with citations"""
        
        context_parts = []
        for i, result in enumerate(results[:5]):
            if not result.get('text'):
                continue
            
            citation = citations[i] if i < len(citations) else {'id': f'cite_{i+1}'}
            filename = citation.get('filename', 'Unknown')
            context_parts.append(f"[{citation['id']} - {filename}]: {result['text'][:600]}")
        
        context = "\n\n".join(context_parts)[:4000]
        
        prompt = f"""Answer using ONLY the provided context with proper citations.

CONTEXT: {context}

QUESTION: {query}

REQUIREMENTS:
- Cite every fact: (cite_X)
- Professional paragraphs
- Use only context information
- If insufficient, state: "Limited information in sources"

YOUR ANSWER:"""
        
        return self.llm_engine.generate_response(prompt, temperature=0.0)
    
    def _build_references(self, results: List[Dict], citations: List[Dict]) -> List[Dict]:
        """Build reference list from results and citations"""
        references = []
        
        for i, result in enumerate(results):
            if not result.get('text'):
                continue
            
            metadata = result.get('metadata', {})
            citation = citations[i] if i < len(citations) else {}
            
            references.append({
                'filename': metadata.get('filename', 'Unknown'),
                'collection': result.get('source_collection', metadata.get('collection', 'Unknown')),
                'chunk_id': metadata.get('chunk_id'),
                'text_snippet': result['text'][:200] + '...' if len(result['text']) > 200 else result['text'],
                'relevance_score': result.get('final_score', 0),
                'confidence': result.get('confidence', 0),
                'citation_id': citation.get('id', f'cite_{i+1}')
            })
        
        return references
    
    def _build_summary_reference(self, results: List[Dict], summary_result: Dict, 
                                 collections: List[str]) -> List[Dict]:
        """Build single document reference with page range for summaries"""
        if not results:
            return []
        
        filename = results[0].get('metadata', {}).get('filename', 'Unknown')
        collection = results[0].get('collection', collections[0])
        
        # Collect unique pages
        pages = []
        for citation in summary_result.get('citations_used', []):
            page = citation.get('page')
            if page and page != 'N/A' and page not in pages:
                pages.append(page)
        
        # Sort pages
        try:
            pages.sort(key=int)
        except:
            pages.sort()
        
        # Create page range string
        if not pages:
            page_range = "Pages not available"
        elif len(pages) == 1:
            page_range = f"Page {pages[0]}"
        else:
            page_range = f"Pages {pages[0]}-{pages[-1]} ({len(pages)} pages)"
        
        return [{
            'filename': filename,
            'collection': collection,
            'page_range': page_range,
            'pages': pages,
            'citation_id': 'doc_1',
            'confidence_score': summary_result['confidence']
        }]
    
    def _build_summary_stats(self, summary_result: Dict, references: List[Dict]) -> Dict:
        """Build summary statistics"""
        pages = references[0]['pages'] if references else []
        
        return {
            'target_words': summary_result['target_words'],
            'actual_words': summary_result['word_count'],
            'sources_used': summary_result.get('sources_count', 0),
            'unique_pages': len(pages),
            'query_focus': summary_result.get('query_focus', 'general')
        }
    
    def _error_response(self, query: str, error: str, collections: List[str]) -> Dict[str, Any]:
        """Create standardized error response"""
        return {
            'query': query,
            'search_type': 'Error',
            'answer': f'Error: {error}',
            'references': [],
            'confidence': 0.0,
            'collections_searched': collections,
            'enhanced_mode': self.enhanced_mode,
            'processing_timestamp': time.strftime('%Y-%m-%d %H:%M:%S')
        }


# Initialize system
try:
    required_components = ['collections_manager', 'llm_engine', 'embedding_model']
    missing_components = [comp for comp in required_components if comp not in globals()]
    
    if missing_components:
        print(f"❌ Missing: {missing_components}")
    else:
        spec_rag = SpecCompliantRAGSystem(
            collections_manager=collections_manager,
            llm_engine=llm_engine,
            embedding_model=embedding_model,
            cross_encoder=cross_encoder if 'cross_encoder' in globals() else None
        )
        
        print("✅ Enhanced RAG System initialized!")
        print("   • Query routing with specialized handlers")
        print("   • Hybrid search with reranking")
        print("   • Reliability validation")
        print("   • Cross-reference comparison")
        print("   • Enhanced summarization")
        print(f"   • Collections: {len(collections_manager.get_available_collections())}")

except Exception as e:
    print(f"❌ Initialization failed: {e}")

✅ Enhanced RAG System initialized!
   • Query routing with specialized handlers
   • Hybrid search with reranking
   • Reliability validation
   • Cross-reference comparison
   • Enhanced summarization
   • Collections: 2


## 7. Launch the complete RAG system interface

#### 7.1 Complete Functional RAG System UI

In [58]:
# Section 7.1: Complete Functional RAG System UI

class ComprehensiveRAGInterface:
    """Complete RAG system interface with all capabilities"""

    def __init__(self, spec_rag_system):
        self.rag_system = spec_rag_system
        self.formatter = ResultsFormatter()  # Use existing formatter
        self.setup_ui_components()
        self.animation_active = False

    def setup_ui_components(self):
        """Initialize all UI components"""
        
        # 1. Collection Selector
        collections = self.rag_system.collections_manager.get_available_collections()
        self.collection_checkboxes = []
        self.collection_map = {}

        for col_name in collections:
            cb = widgets.Checkbox(
                value=True,
                description=col_name,
                layout=widgets.Layout(width='auto'),
                style={'description_width': 'initial'}
            )
            self.collection_checkboxes.append(cb)
            self.collection_map[cb] = col_name

        self.compact_selector = widgets.HBox(
            self.collection_checkboxes,
            layout=widgets.Layout(justify_content='center', margin='10px 0')
        )

        # 2. Query Input
        self.query_input = widgets.Textarea(
            value='',
            placeholder='Enter your query here...',
            layout=widgets.Layout(width='95%', height='60px', margin='10px auto')
        )

        # 3. Answer Output (using HTML widget for formatted display)
        self.answer_output = widgets.HTML(
            value=self._get_placeholder_html(),
            layout=widgets.Layout(width='95%', margin='6px auto')
        )

        # 4. Search Controls
        self.search_button = widgets.Button(
            description='⏳ Search',
            button_style='primary',
            layout=widgets.Layout(width='100px', height='30px')
        )

        self.progress_bar = widgets.FloatProgress(
            value=0, min=0, max=100,
            layout=widgets.Layout(width='400px', height='25px')
        )

        self.status_label = widgets.HTML(value="Ready")

        self.cancel_button = widgets.Button(
            description='Cancel',
            button_style='danger',
            layout=widgets.Layout(width='80px', height='30px'),
            disabled=True
        )

        # 5. Advanced Options
        self.rerank_checkbox = widgets.Checkbox(
            value=True,
            description='Use Cross-Encoder Reranking',
            style={'description_width': 'initial'}
        )
        
        self.expand_checkbox = widgets.Checkbox(
            value=True,
            description='Use Query Expansion',
            style={'description_width': 'initial'}
        )

        self.advanced_options = widgets.Accordion([
            widgets.VBox([self.rerank_checkbox, self.expand_checkbox])
        ])
        self.advanced_options.set_title(0, '⚙️ Advanced Options')
        self.advanced_options.selected_index = None

        # 6. Instructions
        self.instructions = widgets.HTML(value="""
        <div style='background-color: #f8fafc; padding: 10px; border-radius: 6px; font-size: 12px; margin-top: 10px;'>
            <b>💡 Tips:</b><br/>
            • <b>Summarize:</b> "Summarize barclays_1Q20_analyst_meeting_transcript.pdf in 300 words"<br/>
            • <b>Database:</b> "List all files in each collection for year 2022"<br/>
            • <b>Cross-Reference:</b> "Compare barclays_1Q20_analyst_meeting_transcript.pdf against PRA rules"
        </div>
        """)

        # Setup event handlers
        self.search_button.on_click(self._on_search_clicked)
        self.cancel_button.on_click(self._on_cancel_clicked)
        self.cancel_flag = threading.Event()

    def _get_placeholder_html(self) -> str:
        """Get placeholder HTML for answer output"""
        return """
        <div style='max-height:800px; overflow:auto; font-family: monospace; white-space: pre-wrap;
                    color: #334155; padding: 10px; border: 1px solid #e2e8f0; border-radius: 6px;'>
        💡 Results will appear here after search
        </div>
        """

    def _on_cancel_clicked(self, button):
        """Handle cancel button click"""
        self.cancel_flag.set()
        self.animation_active = False
        self.cancel_button.disabled = True
        self.search_button.disabled = False
        self.search_button.description = '⏳ Search'
        self.status_label.value = "<span style='color: #cc6600;'>⚠️ Search cancelled</span>"

    def _on_search_clicked(self, button):
        """Handle search button click"""
        query = self.query_input.value.strip()
        if not query:
            self.status_label.value = "<span style='color: #cc0000;'>⚠️ Enter a query</span>"
            return

        # Get selected collections
        collections = [col for cb, col in self.collection_map.items() if cb.value]

        if not collections:
            self.status_label.value = "<span style='color: #cc0000;'>⚠️ Select at least one collection</span>"
            return

        # Reset UI state
        self._reset_ui_for_search()

        # Start search in background thread
        search_thread = threading.Thread(
            target=self._execute_search,
            args=(query, collections, self.rerank_checkbox.value)
        )
        search_thread.daemon = True
        search_thread.start()

        # Start button animation
        animation_thread = threading.Thread(target=self._animate_button)
        animation_thread.daemon = True
        animation_thread.start()

    def _reset_ui_for_search(self):
        """Reset UI state before starting search"""
        self.answer_output.value = "<div style='white-space: pre-wrap; font-family: monospace;'>Running search...</div>"
        self.search_button.disabled = True
        self.animation_active = True
        self.cancel_button.disabled = False
        self.progress_bar.value = 0
        self.status_label.value = "<span style='color: #0066cc;'>Starting search...</span>"
        self.cancel_flag.clear()

    def _animate_button(self):
        """Animate search button with spinning hourglass"""
        import time
        spinners = ['⏳', '⌛']
        idx = 0
        while self.animation_active:
            self.search_button.description = f'{spinners[idx]} Searching...'
            idx = (idx + 1) % len(spinners)
            time.sleep(0.5)

    def _execute_search(self, query: str, collections: List[str], use_reranking: bool):
        """Execute search in background thread"""
        try:
            self._update_progress(20, "Initializing search...")
            
            if self.cancel_flag.is_set():
                return

            self._update_progress(40, "Searching collections...")
            
            # Execute query with reranking parameter
            result = self.rag_system.process_query(query, collections, use_reranking)

            if self.cancel_flag.is_set():
                return

            self._update_progress(80, "Processing results...")

            # Format and display results
            self._display_results(result)

            self._update_progress(100, "✅ Search completed successfully", success=True)
            self.cancel_button.disabled = True

        except Exception as e:
            self._handle_search_error(e)
        
        finally:
            self._finalize_search()

    def _update_progress(self, value: int, message: str, success: bool = False):
        """Update progress bar and status"""
        self.progress_bar.value = value
        color = '#008800' if success else '#0066cc'
        self.status_label.value = f"<span style='color: {color};'>{message}</span>"

    def _display_results(self, result: Dict[str, Any]):
        """Display formatted results using ResultsFormatter"""
        formatted_text = self.formatter.format_result(result)
        
        # Escape HTML and wrap in styled div
        from html import escape
        escaped_text = escape(formatted_text)
        
        html = f"""
        <div style='max-height:800px; overflow:auto; font-family: monospace; white-space: pre-wrap;
                    color: #334155; padding: 10px; border: 1px solid #e2e8f0; border-radius: 6px;'>
        {escaped_text}
        </div>
        """
        
        self.answer_output.value = html

    def _handle_search_error(self, error: Exception):
        """Handle search errors"""
        error_msg = f"Search failed: {str(error)}"
        self.progress_bar.value = 0
        self.status_label.value = f"<span style='color: #cc0000;'>❌ {error_msg}</span>"
        self.cancel_button.disabled = True
        
        from html import escape
        self.answer_output.value = f"<div style='color:#cc0000;'>{escape(error_msg)}</div>"

    def _finalize_search(self):
        """Finalize search state"""
        self.animation_active = False
        self.search_button.disabled = False
        self.search_button.description = '⏳ Search'

    def display(self):
        """Build and display complete interface"""
        header = widgets.HTML("""
        <div style='background: linear-gradient(135deg, #002147 0%, #003d82 100%);
                    color: white; padding: 10px; border-radius: 8px; margin-bottom: 20px; text-align: center;'>
            <h2 style='margin: 0; font-size: 24px;'>RAG System Interface</h2>
        </div>
        """)

        search_controls = widgets.HBox([
            self.search_button,
            widgets.HTML("<div style='width: 20px;'></div>"),
            self.progress_bar,
            widgets.HTML("<div style='width: 10px;'></div>"),
            self.cancel_button,
        ], layout=widgets.Layout(justify_content='center', margin='10px 0'))

        main_content = widgets.VBox([
            self.compact_selector,
            widgets.HTML("<div style='height: 10px;'></div>"),
            search_controls,
            widgets.HBox([self.status_label], layout=widgets.Layout(justify_content='center')),
            widgets.HTML("<div style='height: 10px;'></div>"),
            self.query_input,
            self.answer_output,
            widgets.HTML("<div style='height: 10px;'></div>"),
            self.advanced_options,
            self.instructions
        ], layout=widgets.Layout(align_items='center'))

        complete_interface = widgets.VBox([
            header,
            main_content
        ], layout=widgets.Layout(
            width='95%',
            max_width='1000px',
            margin='0 auto',
            padding='20px',
            background_color='white',
            border_radius='12px',
            box_shadow='0 2px 8px rgba(0,0,0,0.1)'
        ))

        return complete_interface


print("✅ Complete RAG System UI initialized!")

✅ Complete RAG System UI initialized!


#### 7.2 Launch the complete RAG system interface

In [47]:
# Initialize and Launch Complete RAG Interface
if 'spec_rag' not in globals():
    print("spec_rag system not available. Please run Section 7 first.")
else:
    # Initialize the comprehensive interface
    comprehensive_rag_interface = ComprehensiveRAGInterface(spec_rag)

    # Display the interface ONLY (no print statements after)
    display(comprehensive_rag_interface.display())